In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import sys

from astropy import units as u
from astropy.modeling.models import Voigt1D

from matplotlib import colors

from specutils.spectra import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum
from specutils.fitting import fit_continuum
import os
from astropy.io import ascii
from astropy.io import fits

from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation

from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy import interpolate
import scipy
import matplotlib.colors as mcol
import matplotlib.cm as cm

from astropy.convolution import convolve, Box1DKernel, Gaussian1DKernel

from PyAstronomy import pyasl

from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import plotly
import plotly.graph_objs as go

from rascal.calibrator import Calibrator
from rascal.util import refine_peaks

import pickle as pk

from astroquery.irsa_dust import IrsaDust
import astropy.coordinates as coord
from dust_extinction.parameter_averages import CCM89
from PyAstronomy import pyasl

import warnings
warnings.simplefilter('ignore', UserWarning)

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams['font.size'] = 20
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.markersize'] = 17
plt.rcParams['axes.labelsize'] = 30
# plt.rcParams['axes.formatter.useoffset'] = False

In [ ]:
import spectrum_functions as spec

In [ ]:
def bad_pixel_rejection(wavelength,flux,clip_value=3):
    ################## 3 sigma rejection ###############
    p = np.polyfit(wavelength,flux,1)
    running_mean = p[0]*wavelength+p[1]
    sigma = np.std(flux,axis=0)
    flux = np.where(flux < running_mean + clip_value*sigma,flux,0)
    flux = np.where(flux > running_mean - clip_value*sigma,flux,0)
    func = scipy.interpolate.interp1d(wavelength[np.where(flux!=0)],flux[np.where(flux!=0)],
                                      bounds_error=False,fill_value='extrapolate')
    flux = func(wavelength)
    ################## telluric rejection ##############
    left_of_telluric_lam = wavelength[(wavelength < 7575)]
    left_of_telluric_flux = flux[(wavelength < 7575)]
    right_of_telluric_lam = wavelength[(wavelength > 7700)]
    right_of_telluric_flux = flux[(wavelength > 7700)]
    wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
    flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))
    
    return wavelength,flux

In [ ]:
master_w_spec, master_store_lam, master_store_flux, master_store_cont, master_store_type, all_models = spec.initialize_RCB_spectra()

In [ ]:
for i in range(0,len(master_store_type)):
    if master_w_spec['Name'][i] in ['ASAS-RCB-6','HdCcand-A166']:
        print(master_store_type[i])
        master_store_type[i] = 'Cool'
        print(master_store_type[i])

In [ ]:
store_rv, store_names, store_types, store_rest_wavelength, store_rest_flux, store_ra, store_dec = spec.redshift_RCB_spectra(master_w_spec, 
                                                                                                                         master_store_lam, 
                                                                                                                         master_store_flux, 
                                                                                                                         master_store_type,
                                                                                                                           region='Ca trip')#,
#                                                                                                                            suppress_output=False)

In [ ]:
store_Av_values = spec.read_RCB_extinction_values(master_w_spec,store_names)
# store_Av_values = spec.initialize_RCB_extinction_values(store_names,store_ra,store_dec)

In [ ]:
for i in range(0,len(store_names)):
    print(store_names[i],store_Av_values[i])

In [ ]:
# # # ########### SAVES ALL DEREDDENNED SPECTRA TO ONE FOLDER #############

# os.chdir('/Users/crawford/Desktop/Confirmed_RCB/all_spec_dereddened')

for i in range(0,len(store_names)):
#     if store_names[i] in ['J054221.91-690259.3']:
#     print(store_Av_values[i])
    if store_names[i] in ['UX Ant','UW Cen','dLS17','MACHO-81.8394.1358','MACHO-80.6956.207',
                         'W Men','ASAS J050232-7218.9','J173737.07-072828.1','J171908.50-435044.6',
                         'MACHO 308.38099.66','MACHO 401.48170.2237','RT Nor','RZ Nor',
                         'J181836.38-181732.8','WISE-ToI-150','J181252.50-233304.4','A166',
                         'EROS2-CG-RCB-3','EROS2-CG-RCB-13','KDM5651','M37','M38','P12','WISE-ToI-5031']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])


        line = store_names[i]
        for char in line:
            if char in "?.!/;:":
                line = line.replace(char,'-')
        f,ax = plt.subplots(2, 1, figsize=[12,8], constrained_layout=True)

        ax[0].plot(wavelength,flux,color='k')
        ax[0].set_xlim(3000,6500)

        ax[1].plot(wavelength,flux,color='k')
        ax[1].set_xlim(6500,10000)

        plt.suptitle(store_names[i])
        ax[1].set_xlabel(r'Wavelength ($\AA$)')
        ax[0].set_ylabel('Flux')
        ax[1].set_ylabel('Flux')
#         plt.savefig('%s_dereddened' % (line))
        plt.show()

# os.chdir('/Users/crawford/Desktop')

In [ ]:
# norm_spectra(folder,redfile,bluefile,toi,suppress_output=True)
# norm_spectra_picky(folder,bluefile,redfile,toi,suppress_output=True)
# norm_spectra_AB(folder,redfile1,bluefile1,redfile2,bluefile2,toi,suppress_output=True)

os.chdir('/Users/crawford/Desktop')
# lam,flux,cont = spec.norm_spectra_picky('WISE_271.4603768_-15.2171411',
#                                         'WISE_J180550.49-151301.7_July2012_Blue_118_unsmooth.fits',
#                                         'WISE_J180550.49-151301.7_July2012_Red_118_unsmooth.fits',
#                                         'J180550.49-151301.7')
# lam,flux,cont = spec.norm_spectra_AB('ASAS_RCB_7','ASAS-RCB-7_July2010_Red-A_075_unsmooth.fits',
#                                      'ASAS-RCB-7_July2010_Blue-A_075_unsmooth.fits',
#                                      'ASAS-RCB-7_July2010_Red-B_075_unsmooth.fits',
#                                      'ASAS-RCB-7_July2010_Blue-B_075_unsmooth.fits',
#                                      'ASAS-RCB-7')
lam,flux,cont = spec.norm_spectra_one('EROS2-CG-RCB-6',
                                        'EROS2-CG-RCB-6__fnttwfs17red_ccd00035.fits',
                                        'EROS2-CG-RCB-6')
os.chdir('/Users/crawford/Desktop')

wavelength, flux = spec.apply_extinction(lam,flux,6.555,'EROS2-CG-RCB-6')

# f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
# ax[0].plot(wavelength,flux,label='Old Spectrum')
# ax[0].legend()
plt.plot(wavelength,flux,label='Old Spectrum')

for i in range(0,len(store_names)):
    if store_names[i] == 'EROS2-CG-RCB-6':
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
# ax[1].plot(wavelength,flux,label='New Spectrum')
# ax[1].legend()
plt.plot(wavelength,flux,label='New Spectrum')
plt.legend()
plt.title('EROS2-CG-RCB-6')
# plt.savefig('EROS2-CG-RCB-6_same')
plt.show()

In [ ]:
# norm_spectra(folder,redfile,bluefile,toi,suppress_output=True)
# norm_spectra_picky(folder,bluefile,redfile,toi,suppress_output=True)
# norm_spectra_AB(folder,redfile1,bluefile1,redfile2,bluefile2,toi,suppress_output=True)

os.chdir('/Users/crawford/Desktop')
lam,flux,cont = spec.norm_spectra_picky('J174111.80-281955.3',
                                        'WISE_J174111.80-281955.3_T2m3wb_July2011_0052_unsmooth.fits',
                                        'WISE_J174111.80-281955.3_T2m3wr_July2011_0052_unsmooth.fits',
                                        'J174111.80-281955.3')
# lam,flux,cont = spec.norm_spectra_AB('ASAS_RCB_7','ASAS-RCB-7_July2010_Red-A_075_unsmooth.fits',
#                                      'ASAS-RCB-7_July2010_Blue-A_075_unsmooth.fits',
#                                      'ASAS-RCB-7_July2010_Red-B_075_unsmooth.fits',
#                                      'ASAS-RCB-7_July2010_Blue-B_075_unsmooth.fits',
#                                      'ASAS-RCB-7')
os.chdir('/Users/crawford/Desktop')

wavelength, flux = spec.apply_extinction(lam,flux,7.58,'J174111.80-281955.3')

# f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
# ax[0].plot(wavelength,flux,label='Old Spectrum')
# ax[0].legend()
plt.plot(wavelength,flux,label='Old Spectrum')

for i in range(0,len(store_names)):
    if store_names[i] == 'J174111.80-281955.3':
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
# ax[1].plot(wavelength,flux,label='New Spectrum')
# ax[1].legend()
plt.plot(wavelength,flux,label='New Spectrum')
plt.legend()
plt.title('J174111.80-281955.3')
# plt.savefig('J17411180-2819553_same')
plt.show()

In [ ]:
# norm_spectra(folder,redfile,bluefile,toi,suppress_output=True)
# norm_spectra_picky(folder,bluefile,redfile,toi,suppress_output=True)
# norm_spectra_AB(folder,redfile1,bluefile1,redfile2,bluefile2,toi,suppress_output=True)

os.chdir('/Users/crawford/Desktop')
lam,flux,cont = spec.norm_spectra_picky('WMen',
                                        'WMen__T2m3wb-20220219.103652-0733.spectrum.p11.fits.gz',
                                        'WMen__T2m3wr-20220219.103652-0733.spectrum.p11.fits.gz',
                                        'W Men')
# lam,flux,cont = spec.norm_spectra_AB('ASAS_RCB_10','ASAS-RCB-10_July2010_Red-A_071_unsmooth.fits',
#                                      'ASAS-RCB-10_July2010_Blue-A_071_unsmooth.fits',
#                                      'ASAS-RCB-10_July2010_Red-B_071_unsmooth.fits',
#                                      'ASAS-RCB-10_July2010_Blue-B_071_unsmooth.fits',
#                                      'ASAS-RCB-10')
os.chdir('/Users/crawford/Desktop')

wavelength, flux = spec.apply_extinction(lam,flux,0.284,'W Men')

# f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
# ax[0].plot(wavelength,flux,label='Old Spectrum')
# ax[0].legend()
plt.plot(wavelength,flux,label='Old Spectrum')
# flux1 = flux

for i in range(0,len(store_names)):
    if store_names[i] == 'W Men':
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
#         wavelength, flux = bad_pixel_rejection(wavelength,flux,1.5)
# ax[1].plot(wavelength,flux,label='New Spectrum')
# ax[1].legend()
plt.plot(wavelength,flux,label='New Spectrum')
# plt.plot(wavelength,flux/flux1)
plt.legend()
plt.title('W Men')
# plt.savefig('WMen_same')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in ['ASAS-RCB-21','J132354.47-673720.8','U Aqr']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        greater = np.where(wavelength>4200) 
        plt.plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i])
# plt.title('ASAS-RCB-4')
plt.legend()
plt.xlim(3800,4300)
plt.axvline(4077,color='k')
plt.axvline(4215,color='k')
plt.axvline(3908,color='r')
# plt.axvline(6498,color='r')
# plt.savefig('asas4_srlines')
plt.show()

In [ ]:
strontium_list = ['HE 1015-2050','U Aqr','EROS2-LMC-RCB-3','A249','A166','C539','ASAS-RCB-4','J132354.47-673720.8']

# ca_k_area = []
sr_4077_area = []

# for i in range(0,6):
for i in range(0,len(store_names)):
#     if store_names[i] in strontium_list:
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])

    flux = flux/np.max(flux)

    lines = [4077]

    if np.min(wavelength) > lines[0]-75:
        print(store_names[i], 'not enough spectrum, setting EW to -99')
        eq_width_array[0] = -99
#             eq_width_array[1] = -99
#             index = -99
    else:
        try:
            eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
                                                                          center_tolerance=3,
                                                                          width_tolerance=20,
                                                                          continuum_region=75,
                                                                          continuum_width=0)


        except TypeError:
            print(store_names[i], 'not enough spectrum, setting EW to -99')
            eq_width_array[0] = -99
#                 eq_width_array[1] = -99
#                 index = -99


#         ca_k_area.append(eq_width_array[0])
    sr_4077_area.append(eq_width_array[0])

In [ ]:
ca_h_area = []
# sr_4077_area = []

# for i in range(0,6):
for i in range(0,len(store_names)):
#     if store_names[i] in strontium_list:
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])

    flux = flux/np.max(flux)

    lines = [3968]#,4077]

    if np.min(wavelength) > lines[0]-75:
        print(store_names[i], 'not enough spectrum, setting EW to -99')
        eq_width_array[0] = -99
#             eq_width_array[1] = -99
#             index = -99
    else:
        try:
            eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
                                                                          center_tolerance=3,
                                                                          width_tolerance=30,
                                                                          continuum_region=75,
                                                                          continuum_width=0)


        except TypeError:
            print(store_names[i], 'not enough spectrum, setting EW to -99')
            eq_width_array[0] = -99
#                 eq_width_array[1] = -99
#                 index = -99


    ca_h_area.append(eq_width_array[0])
#         sr_4077_area.append(eq_width_array[1])

In [ ]:
plt.plot(ca_h_area,sr_4077_area,'o', markersize=15,color='r',alpha=0.65)
plt.xlabel('Pseudo-EW of Ca II H')
plt.ylabel('Pseudo-EW of Sr II 4077')
plt.ylim((-2,19))
plt.xlim((-2,30))
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] != 'C1004':
        plt.plot(ca_h_area[i],sr_4077_area[i],'o', markersize=15,color='k',alpha=0.5)
#         if ca_h_area[i] > 0 and ca_h_area[i] < 4:
#             plt.annotate(store_names[i],xy=(ca_h_area[i]+0.45,sr_4077_area[i]-0.25),annotation_clip=False)
        if store_names[i] in strontium_list:
            plt.plot(ca_h_area[i],sr_4077_area[i],'o', markersize=15,color='r',alpha=0.65)
            if store_names[i] == 'C539':
                plt.annotate(store_names[i],xy=(ca_h_area[i]+.25,sr_4077_area[i]+0.25),annotation_clip=False)
            elif store_names[i] == 'EROS2-LMC-RCB-3':
                plt.annotate(store_names[i],xy=(ca_h_area[i]+.45,sr_4077_area[i]-0.25),annotation_clip=False)
            else:
                plt.annotate(store_names[i],xy=(ca_h_area[i]+0.45,sr_4077_area[i]-0.25),annotation_clip=False)
    #     if ca_h_area[i] > 25:
    #         plt.annotate(store_names[i],xy=(ca_h_area[i],sr_4077_area[i]),annotation_clip=False)

plt.xlabel('Pseudo-EW of Ca II H')
plt.ylabel('Pseudo-EW of Sr II 4077')
plt.ylim((-2,19))
plt.xlim((-2,30))
# plt.savefig('sr_newstars')
plt.show()

In [ ]:
for i in range(0,len(strontium_list)):
    print(strontium_list[i],ca_h_area[i],sr_4077_area[i],np.round(ca_h_area[i]/sr_4077_area[i],2))

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in ['ASAS-RCB-4','U Aqr','J132354.47-673720.8','ASAS-RCB-14','V517 Oph','ES Aql',
                         'J173202.75-432906.1','EROS2-CG-RCB-13','ASAS-RCB-21','V2331 Sgr',
                         'EROS2-CG-RCB-6','J174111.80-281955.3','ASAS-RCB-10','W Men','ASAS-RCB-7',
                         'J175558.51-164744.3','ASAS-RCB-3','IRAS1813.5-2419','V2552 Oph',
                         'J174645.90-250314.1','J182235.25-033213.2','J171815.36-341339.9','J173819.81-203632.1',
                         'J175749.98-182522.8','J180550.49-151301.7','EROS2-CG-RCB-1']:
        #EROS2-CG-RCB-1
        
        plt.plot(store_rest_wavelength[i],store_rest_flux[i],label=store_names[i])
        plt.axvline(3600,color='r')
        plt.legend()
        plt.show()
        
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux,label=store_names[i])
        plt.axvline(3600,color='r')
        plt.legend()
        plt.show()
# plt.title('ASAS-RCB-4')
# plt.legend()
# plt.xlim(3800,4300)
# plt.axvline(4077,color='k')
# plt.axvline(4215,color='k')
# plt.axvline(3908,color='r')
# # plt.axvline(6498,color='r')
# # plt.savefig('asas4_srlines')
# plt.show()

In [ ]:
# for i in range(0,len(master_w_spec)):
#     if type(master_w_spec['Bluefile'][i]) == str:
#         print('cp',master_w_spec['Folder'][i]+'/'+master_w_spec['Bluefile'][i],'may3_2022_spectra')
#     if type(master_w_spec['Alt_Bluefile'][i]) == str:
#         print('cp',master_w_spec['Folder'][i]+'/'+master_w_spec['Alt_Bluefile'][i],'may3_2022_spectra')
#     if type(master_w_spec['Redfile'][i]) == str:
#         print('cp',master_w_spec['Folder'][i]+'/'+master_w_spec['Redfile'][i],'may3_2022_spectra')
#     if type(master_w_spec['Alt_Redfile'][i]) == str:
#         print('cp',master_w_spec['Folder'][i]+'/'+master_w_spec['Alt_Redfile'][i],'may3_2022_spectra')

In [ ]:
# for i in range(0,len(master_w_spec)):     
#     print(master_w_spec['Name'][i]+','+str(master_w_spec['Bluefile'][i])+','+
#           str(master_w_spec['Alt_Bluefile'][i])+','+str(master_w_spec['Redfile'][i])+','+
#           str(master_w_spec['Alt_Redfile'][i]))

# MK Standard Stars

In [ ]:
os.chdir('/Users/crawford/Desktop')

In [ ]:
standards = pd.read_csv('MK_standards.csv')

In [ ]:
standards

In [ ]:
os.chdir('/Users/crawford/Desktop/')
stand_store_lam = [[]]
stand_store_flux = [[]]
stand_store_cont = [[]]
for i in range(0,len(standards)):
    if standards['Spectra_Type'][i] in ['picky','picky_cs']:
        a,b,c = spec.norm_spectra_picky(standards['Folder'][i],standards['Bluefile'][i],
                                  standards['Redfile'][i],standards['Name'][i])
        stand_store_lam.append(a)
        stand_store_flux.append(b)
        stand_store_cont.append(c)
    else:
        pass
stand_store_lam.pop(0)
stand_store_flux.pop(0)
stand_store_cont.pop(0)
os.chdir('/Users/crawford/Desktop/')

In [ ]:
stand_coords = {'HD 95444':['11h00m31.35s', '-44d10m47.78s'], 'HD 101947':['11h43m31.19s','-62d29m21.83s'], 
                'Iota Vir':['14h16m00.86s','-06d00m01.96s'], 'HD 133774':['15h06m37.59s','-16d15m24.56s'], 
                '1 Lup':['15h14m37.32s','-31d31m08.84s'], 
               'HD 141767':['15h55m29.60s','-68d36m10.80s'], 'HD 145544':['16h15m26.27s','-63d41m08.45s'], 
                'HD 146143':['16h17m00.93s','-50d04m05.23s'], 'HD 155603':['17h14m27.65s','-39d45m59.94s'], 
                'HR 6724':['18h02m51.10s','-24d16m56.87s'], 
               'HD 181475':['19h20m48.31s','-04d30m09.01s'], 'HD 188603':['19h56m56.83s','-27d10m11.64s'], 
                'Alpha Cap':['20h18m03.26s','-12d32m41.47s']}

stand_order = {'HD 95444':9, 'HD 101947':5, 'Iota Vir':3, 'HD 133774':12, '1 Lup':1, 
               'HD 141767':8, 'HD 145544':6, 'HD 146143':4, 'HD 155603':10, 
                'HR 6724':2, 'HD 181475':13, 'HD 188603':11, 'Alpha Cap':7}

In [ ]:
stand_Av_values = []
for i in range(0,len(standards['Name'])):
    rastring = stand_coords[standards['Name'][i]][0]
    decstring = stand_coords[standards['Name'][i]][1]
    coordstring = rastring + ' ' + decstring
    coo = coord.SkyCoord(coordstring)
    table = IrsaDust.get_extinction_table(coo)
    if standards['Name'][i] == 'HD 155603':
        stand_Av_values.append(4.464)
    elif standards['Name'][i] == 'HD 146143':
        stand_Av_values.append(0.806)
    elif standards['Name'][i] == 'HR 6724':
        stand_Av_values.append(0.527)
    elif standards['Name'][i] == 'HD 101947':
        stand_Av_values.append(0.620)
    else:
        stand_Av_values.append(table['A_SandF'][2])

In [ ]:
for i in range(0,len(standards['Name'])):
    print(standards['Name'][i],stand_Av_values[i])

In [ ]:
ext = CCM89(Rv=3.1)

plt.figure(figsize=(10,17))
# plt.figure(figsize=(15,10))
for i in range(0,len(standards)):
    x=stand_order[standards['Name'][i]]*2
    wavelength_w_units = stand_store_lam[i]*u.AA
    dereddened_flux = np.asarray(stand_store_flux[i])/ext.extinguish(wavelength_w_units,Av=stand_Av_values[i])
    plt.plot(wavelength_w_units,(dereddened_flux*2/np.max(dereddened_flux))-x)
#     plt.plot(stand_store_lam[i],(stand_store_flux[i]*2/np.max(stand_store_flux[i]))-x)
    plt.annotate(standards['Name'][i] + ' ' + standards['temp_type'][i] + standards['lum_type'][i],
                 xy=(4500,-x+1),annotation_clip=False)
            
# plt.axvline(4326,color='k')
# plt.axvline(4290,color='k')

        
plt.yticks([])
plt.xlim(4000,4500)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel('Flux')
plt.savefig('standard_stars',bbox_inches='tight')
plt.show()

In [ ]:
ext = CCM89(Rv=3.1)

plt.figure(figsize=(12,5))
# plt.figure(figsize=(15,10))
for i in range(0,len(standards)):
    if standards['Name'][i] in ['1 Lup','HR 6724','Iota Vir']:
        x=stand_order[standards['Name'][i]]
        wavelength_w_units = stand_store_lam[i]*u.AA
        dereddened_flux = np.asarray(stand_store_flux[i])/ext.extinguish(wavelength_w_units,Av=stand_Av_values[i])
        plt.plot(wavelength_w_units,(dereddened_flux/np.max(dereddened_flux))-x)#,color='tab:red')
    #     plt.plot(stand_store_lam[i],(stand_store_flux[i]*2/np.max(stand_store_flux[i]))-x)
#         plt.annotate(standards['Name'][i] + ' ' + standards['temp_type'][i] + standards['lum_type'][i],
#                      xy=(4510,-x),annotation_clip=False)
            
# plt.axvline(4326,color='k')
# plt.axvline(4215,color='k')

# for i in range(0,len(barnbaum_stars_lam)):
#     if barnbaum_spec['Name'][i] == 'NQ Cas':#'BD+53 379':#'BD+23 123':#'BD+20 5071': #RV Sct, NQ Cas
#         plt.plot(barnbaum_stars_lam[i],barnbaum_stars_flux[i]*1.5/np.max(barnbaum_stars_flux[i]),color='tab:red')


for i in range(0,len(store_names)):
    if store_names[i] in ['V3795 Sgr']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))
        
        greater = np.where(wavelength>6900)                
#         plt.plot(wavelength,flux/(np.average(flux[greater][:50]))-order_pca[store_names[i]])#,label=store_names[i],color='tab:blue')
        plt.plot(wavelength,flux/np.max(flux)-4)#-order_pca[store_names[i]])
#         plt.annotate('HdC' + str(order_pca[store_names[i]]),xy=(5510,-order_pca[store_names[i]]+6+0.2),annotation_clip=False)

        
# plt.axvline(4215,color='k')
# plt.axvline(4226,color='k')
# plt.axvline(4233,color='k')
        
plt.yticks([])
plt.xlim(5000,5500)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel('Flux')
# plt.savefig('standard_stars',bbox_inches='tight')
plt.show()

In [ ]:
from astropy.modeling.models import BlackBody

plt.figure(figsize=(12,5))
wavelengths = np.linspace(4000,7000, num=7000) * u.AA

# BlackBody provides the results in ergs/(cm^2 Hz s sr) when scale has no units
bb1 = BlackBody(temperature=3800*u.K, scale=1.0)
bb_result1 = bb1(wavelengths)

bb2 = BlackBody(temperature=4100*u.K, scale=1.0)
bb_result2 = bb2(wavelengths)

greater = np.where(wavelengths/u.AA>6800)
plt.plot(wavelengths, bb_result1/(bb_result1[greater][0]),color='tab:red',label='3800 K')
plt.plot(wavelengths, bb_result2/(bb_result2[greater][0]),color='tab:blue',label='4100 K')

#         greater = np.where(wavelength>6800)                
#         plt.plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i],color='tab:blue')

plt.legend()
plt.show()

In [ ]:
ext = CCM89(Rv=3.1)

plt.figure(figsize=(12,5))
# plt.figure(figsize=(15,10))
# for i in range(0,len(standards)):
#     if standards['Name'][i] in ['1 Lup','HR 6724','Iota Vir']:
#         x=stand_order[standards['Name'][i]]
#         wavelength_w_units = stand_store_lam[i]*u.AA
#         dereddened_flux = np.asarray(stand_store_flux[i])/ext.extinguish(wavelength_w_units,Av=stand_Av_values[i])
#         plt.plot(wavelength_w_units,(dereddened_flux/np.max(dereddened_flux))-x)#,color='tab:red')
#     #     plt.plot(stand_store_lam[i],(stand_store_flux[i]*2/np.max(stand_store_flux[i]))-x)
# #         plt.annotate(standards['Name'][i] + ' ' + standards['temp_type'][i] + standards['lum_type'][i],
#                      xy=(4510,-x),annotation_clip=False)
            
# plt.axvline(4326,color='k')
# plt.axvline(4215,color='k')

for i in range(0,len(store_names)):
    if store_names[i] in ['ASAS-RCB-4']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))
        
        greater = np.where(wavelength>6800)                
        plt.plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i],color='tab:blue')

for i in range(0,len(barnbaum_stars_lam)):
    if barnbaum_spec['Name'][i] == 'BD+34 1929': #in ['TV Lac','V Ari']:
        print(barnbaum_Av_values[i])
        wavelength_w_units = barnbaum_stars_lam[i]*u.AA
        dereddened_flux = np.asarray(barnbaum_stars_flux[i])/ext.extinguish(wavelength_w_units,
                                                                            Av=barnbaum_Av_values[i])
        plt.plot(barnbaum_stars_lam[i],dereddened_flux*1.1/np.max(dereddened_flux),color='tab:red',label='barnbaum_star')



plt.yticks([])
plt.xlim(3800,7000)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel('Flux')
plt.legend()
# plt.savefig('standard_stars',bbox_inches='tight')
plt.show()

# HdC Standards Plots

In [ ]:
order_pca = {'V3795 Sgr':0,'VZ Sgr':1,'HD173409':2,'A223':3,'B42':4,
             'ASAS-RCB-2':5,'V517 Oph':6,'ASAS-RCB-18':7}#,
#              'V517 Oph':8,'J182334.24-282957.1':9}

In [ ]:
order_pca = {'V3795 Sgr':0,'VZ Sgr':1,'HD173409':2,'A223':3,'B42':4,
             'ASAS-RCB-2':5,'V517 Oph':6,'ASAS-RCB-18':7}#,
#              'V517 Oph':8,'J182334.24-282957.1':9}#'J174328.50-375029.0':6

plt.figure(figsize=(15,20))
# f,ax = plt.subplots(1, 2, figsize=[10,10], constrained_layout=True)
for i in range(0,len(store_names)):
    if store_names[i] in list(order_pca.keys()):
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))
        greater = np.where(wavelength>7000)                
        plt.plot(wavelength,flux/(np.average(flux[greater][:50]))-order_pca[store_names[i]],color='tab:blue')
#         plt.plot(wavelength,flux*2/np.max(flux)-order_pca[store_names[i]],color='tab:blue')
        plt.annotate('HdC' + str(order_pca[store_names[i]]),xy=(7010,-order_pca[store_names[i]]+0.92),annotation_clip=False)
        
plt.axvline(6191,ymin=0.04,ymax=0.07,color='tab:orange',linewidth=2)
plt.axvline(6122,ymin=0.04,ymax=0.07,color='tab:orange',linewidth=2)
plt.axvline(6059,ymin=0.04,ymax=0.07,color='tab:orange',linewidth=2)
plt.annotate(r'C$_2$',xy=(6045,-7.8),annotation_clip=False,size=15,color='tab:orange')
plt.axvline(6530,ymin=0.04,ymax=0.07,color='tab:orange',linewidth=2)
plt.axvline(6600,ymin=0.04,ymax=0.07,color='tab:orange',linewidth=2)
plt.axvline(6675,ymin=0.04,ymax=0.07,color='tab:orange',linewidth=2)
plt.axvline(6760,ymin=0.04,ymax=0.07,color='tab:orange',linewidth=2)
plt.axvline(6855,ymin=0.04,ymax=0.07,color='tab:orange',linewidth=2)

plt.axvline(6245,ymin=0.93,ymax=0.96,color='r',linewidth=2)
plt.axvline(6013,ymin=0.93,ymax=0.96,color='r',linewidth=2)
plt.annotate(r'C I',xy=(6005,2.7),annotation_clip=False,size=15,color='r')

plt.axvline(6156,ymin=0.93,ymax=0.96,color='b',linewidth=2)
plt.annotate(r'O I',xy=(6145,2.7),annotation_clip=False,size=15,color='b')
plt.axvline(6455,ymin=0.93,ymax=0.96,color='b',linewidth=2)


plt.axvline(6206,ymin=0.04,ymax=0.07,color='tab:green',linewidth=2)
plt.annotate(r'CN',xy=(6195,-7.8),annotation_clip=False,size=15,color='tab:green')
plt.axvline(6332,ymin=0.04,ymax=0.07,color='tab:green',linewidth=2)
plt.axvline(6945,ymin=0.04,ymax=0.07,color='tab:green',linewidth=2)

# plt.axvline(6548,ymin=0.93,ymax=0.96,color='tab:green',linewidth=2) #N


plt.axvline(5890,ymin=0.93,ymax=0.96,color='k',linewidth=2)
plt.annotate(r'Na',xy=(5880,2.7),annotation_clip=False,size=15)
plt.axvline(5890,ymin=0.04,ymax=0.07,color='k',linewidth=2)
plt.annotate(r'Na',xy=(5880,-7.8),annotation_clip=False,size=15)

plt.axvline(6563,ymin=0.93,ymax=0.96,color='k',linewidth=2)
plt.annotate(r'H$\alpha$',xy=(6550,2.7),annotation_clip=False,size=15)

plt.axvline(6707,ymin=0.93,ymax=0.96,color='k',linewidth=2)
plt.annotate(r'Li',xy=(6697,2.7),annotation_clip=False,size=15)
plt.axvline(6707,ymin=0.04,ymax=0.07,color='k',linewidth=2)
plt.annotate(r'Li',xy=(6697,-7.8),annotation_clip=False,size=15)

plt.axvline(6498,ymin=0.93,ymax=0.96,color='k',linewidth=2)
plt.annotate(r'Ba II',xy=(6478,2.7),annotation_clip=False,size=15)
plt.axvline(6498,ymin=0.04,ymax=0.07,color='k',linewidth=2)
plt.annotate(r'Ba II',xy=(6478,-7.8),annotation_clip=False,size=15)

plt.axvline(6347,ymin=0.93,ymax=0.96,color='k',linewidth=2)
plt.annotate(r'Si II',xy=(6340,2.7),annotation_clip=False,size=15)
plt.axvline(6371,ymin=0.93,ymax=0.96,color='k',linewidth=2)
# plt.annotate(r'Si II',xy=(6371,2.7),annotation_clip=False,size=15)

# plt.axvline(6140,color='k')



plt.yticks([])
# plt.ylim(-8,1)
plt.xlim(5800,7000)
plt.ylim(-8,3)
plt.ylabel('Flux + Offset')
plt.xlabel(r'Wavelength ($\AA$)')
# plt.savefig('class_sequence_zoom_labeled.eps',bbox_inches='tight')
plt.show()

In [ ]:
order_pca = {'V3795 Sgr':0,'VZ Sgr':1,'HD173409':2,'A223':3,'B42':4,
             'ASAS-RCB-2':5,'V517 Oph':6,'ASAS-RCB-18':7}#,
#              'V517 Oph':8,'J182334.24-282957.1':9}

plt.figure(figsize=(15,18))
# f,ax = plt.subplots(1, 2, figsize=[10,10], constrained_layout=True)
for i in range(0,len(store_names)):
    if store_names[i] in list(order_pca.keys()):
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))
#         greater = np.where(wavelength>7000)                
#         plt.plot(wavelength,flux/(np.average(flux[greater][:50]))-order_pca[store_names[i]],color='tab:blue')
        plt.plot(wavelength,flux/np.max(flux)-order_pca[store_names[i]],color='tab:blue')
        plt.annotate('HdC' + str(order_pca[store_names[i]]),xy=(10050,-order_pca[store_names[i]]+0.25),annotation_clip=False)


plt.yticks([])
# plt.ylim(-8,1)
# plt.xlim(3600,9500)
plt.ylim(-7.5,1.5)
plt.ylabel('Flux + Offset')
plt.xlabel(r'Wavelength ($\AA$)')
# plt.savefig('class_sequence.eps',bbox_inches='tight')
plt.show()

# Initializing Classes

In [ ]:
# known_temps_dict = {'SV Sge':5000, 'Z Umi':5250, 'NSV11154':5200, 'S Aps':5400, 'WX CrA':5300, 'U Aqr':6000,
#               'HD182040':5590, 'HD175893':5640, 'HD137613':5400, 'HD173409':6100, 'RT Nor':7000, 'HD148839':5625,
#               'GU Sgr':6250, 'RS Tel':6750, 'V CrA':6250, 'R CrB':6750, 'FH Sct':6250, 'UX Ant':7000, 'RZ Nor':6750,
#               'V2552 Oph':6750, 'VZ Sgr':7000, 'RY Sgr':7250, 'Y Mus':7250, 'V3795 Sgr':8000}

# temperature_order = {'SV Sge':1, 'Z Umi':2, 'NSV11154':3, 'S Aps':4, 'WX CrA':5, 'U Aqr':6,
#               'HD182040':7, 'HD175893':8, 'HD137613':9, 'HD173409':10, 'RT Nor':11, 'HD148839':12,
#               'GU Sgr':13, 'RS Tel':14, 'V CrA':15, 'R CrB':16, 'FH Sct':17, 'UX Ant':18, 'RZ Nor':19,
#               'V2552 Oph':20, 'VZ Sgr':21, 'RY Sgr':22, 'Y Mus':23, 'V3795 Sgr':24}

# ############ only full spectra ###############
# clean_known_temps_dict = {'SV Sge':5000, 'S Aps':5400, 'WX CrA':5300, 'U Aqr':6000,
#               'HD182040':5590, 'HD175893':5640, 'HD137613':5400, 'HD173409':6100, 'RT Nor':7000, 'HD148839':5625,
#               'GU Sgr':6250, 'RS Tel':6750, 'V CrA':6250, 'FH Sct':6250, 'UX Ant':7000, 'RZ Nor':6750,
#               'V2552 Oph':6750, 'VZ Sgr':7000, 'RY Sgr':7250, 'Y Mus':7250, 'V3795 Sgr':8000}

# clean_temperature_order = {'SV Sge':1, 'S Aps':2, 'WX CrA':3, 'U Aqr':4,
#               'HD182040':5, 'HD175893':6, 'HD137613':7, 'HD173409':9, 'RT Nor':8, 'HD148839':10,
#               'GU Sgr':11, 'RS Tel':12, 'V CrA':13, 'FH Sct':14, 'UX Ant':15, 'RZ Nor':16,
#               'V2552 Oph':17, 'VZ Sgr':18, 'RY Sgr':19, 'Y Mus':20, 'V3795 Sgr':21}

# estimated_temp = {'SV Sge':5250, 'S Aps':5250, 'WX CrA':5250, 'U Aqr':5500,
#               'HD182040':5500, 'HD175893':5750, 'HD137613':5750, 'RT Nor':6000, 'HD173409':6000, 'HD148839':6000,
#               'GU Sgr':6250, 'RS Tel':6250, 'V CrA':6250, 'FH Sct':6500, 'UX Ant':6500, 'RZ Nor':6750,
#               'V2552 Oph':7000, 'VZ Sgr':7000, 'RY Sgr':7250, 'Y Mus':7500, 'V3795 Sgr':8000}

In [ ]:
# final_sets = {#  8: ['UW Cen', 'UX Ant', 'C528'],
# 7: ['AO Her',
#   'ASAS-RCB-18',
#   'ASAS-RCB-19',
#   'ASAS-RCB-4',
#   'ASAS-RCB-7',
# #     'EROS2-SMC-RCB-3',
#   #'ES Aql',
#   'J132354.47-673720.8',
# #   'J182334.24-282957.1',
#   'J182723.38-200830.1',
#   'J183649.54-113420.7',
#     'MACHO 401.48170.2237',
#   'J184246.26-125414.7'],
#   #'V517 Oph'],
# 6: ['ASAS-RCB-15',
# #   'ASAS-RCB-3',
#   'ASAS-RCB-5',
#   'IRAS1813.5-2419',
#   'J173819.81-203632.1',
#   'J174138.87-161546.4',
#   'J174328.50-375029.0',
#   'J194218.38-203247.5',
#   'SV Sge',
#   'V1157 Sgr',
#     'MACHO-81.8394.1358',
# #     'MACHO-80.6956.207',
#     'EROS2-CG-RCB-13',
#     'WISE-ToI-5031',
#     'EROS2-CG-RCB-3',
# #         'ASAS-RCB-6',
# #         'ASAS-RCB-3',
# #         'A166',
# #     'Z Umi',
#   'EROS2-LMC-RCB-4',
#   'EROS2-SMC-RCB-1'],
# 5: ['ASAS-RCB-1',
#   'ASAS-RCB-16',
#   'ASAS-RCB-17',
#   'ASAS-RCB-2',
#   'ASAS-RCB-9',
#   'J160205.48-552741.6',
#   'J161156.23-575527.1',
#   'J182943.83-190246.2',
#   'OGLE-GC-RCB-1',
#   'S Aps',
#   'U Aqr',
#   'V1783 Sgr',
#   'V2331 Sgr',
#   'V739 Sgr',
#   'WX CrA',
#     'NSV11154',
#     'Z Umi',
#   'EROS2-LMC-RCB-1',
#   'EROS2-LMC-RCB-5',
#   'EROS2-SMC-RCB-2'],
# 4: ['ASAS-RCB-11',
# #   'HE 1015-2050',
#   #'J175749.76-075314.9',
#   'HD137613',
#   'HD175893',
#   'EROS2-LMC-RCB-2',
#   'EROS2-LMC-RCB-3',
#   'J054123.49-705801.8',
#     'ASAS J050232-7218.9',
#     'J175749.76-075314.9',
#     'HE 1015-2050',
#     'J181252.50-233304.4',
# #     'J181836.38-181732.8',
#   'B42',
#   'A770',
# #     'R CrB',
#   'A977',
# #     'ASAS-RCB-12',
# #   'FH Sct',
#   'J184158.40-054819.2',
#   'V4017 Sgr',
# #   'J005010.67-694357.7',
# #   'C542',
#   'A814'],
# 3: [#'HD182040', 
#     'C27',
# #     'B566',
#     'GU Sgr',
#   'J174119.57-250621.2',
# #   'J174851.29-330617.0',
#   'RS Tel',
# #   'V CrA',
#   'J055643.56-715510.7',
#   'A223',
#   'A183',
#   'C38',
#   'C20',
#   'A811',
#   'C539',
#     'dLS17',
# #   'B565',
#   'B563'],
# 2: [#'J174111.80-281955.3',
# #   'RZ Nor',
#   'HD148839',
#   'HD173409',
#     'HD182040',
#   'A182',
#   'A226',
#     'B565',
#     'RZ Nor',
#     'RT Nor',
#     'B566',
#     'J174851.29-330617.0',
#   'C17',
#   'C105'],
# #   'A249'],
# 1: ['J172447.52-290418.6',
#   'J175558.51-164744.3',
# #   'V854 Cen',
#   'VZ Sgr',
#     'ASAS-RCB-8','RY Sgr',
#   'F75',
#   'C526',
#   'J185525.52-025145.7',
#   'V2552 Oph',
#   'V391 Sct',
#   'B567',
#     'A249',
#     'UX Ant',
# #   'F152',
#   'V854 Cen'],
# 0: ['J172951.80-101715.9', 
#     'V3795 Sgr', 
#     'W Men',
#     'A798',
#     'Y Mus', 
#     'B564', 
#     'A980']}
# #  2: ['ASAS-RCB-6', 'A166'],
# #  12: ['ASAS-RCB-8','RY Sgr']}

In [ ]:
# final_set_dict = {}
# final_set_arr = []
# for i in range(0,len(store_names)):
#     if store_names[i] in final_sets[0]:
#         final_set_dict[store_names[i]] = 0
#         final_set_arr.append(7)
#     elif store_names[i] in final_sets[1]:
#         final_set_dict[store_names[i]] = 1
#         final_set_arr.append(6)
#     elif store_names[i] in final_sets[2]:
#         final_set_dict[store_names[i]] = 2
#         final_set_arr.append(5)
#     elif store_names[i] in final_sets[3]:
#         final_set_dict[store_names[i]] = 3
#         final_set_arr.append(4)
#     elif store_names[i] in final_sets[4]:
#         final_set_dict[store_names[i]] = 4
#         final_set_arr.append(3)
#     elif store_names[i] in final_sets[5]:
#         final_set_dict[store_names[i]] = 5
#         final_set_arr.append(2)
#     elif store_names[i] in final_sets[6]:
#         final_set_dict[store_names[i]] = 6
#         final_set_arr.append(1)
#     elif store_names[i] in final_sets[7]:
#         final_set_dict[store_names[i]] = 7
#         final_set_arr.append(0)

In [ ]:
# plot_order_final_sets = {
#   'ASAS-RCB-18':4,
#   'ASAS-RCB-19':6,
#   'ASAS-RCB-4':3,
#   'ASAS-RCB-7':1,
#   'J132354.47-673720.8':2,
#   'J182723.38-200830.1':8,
#   'J183649.54-113420.7':9,
# #     'EROS2-SMC-RCB-3':3,
#     'AO Her':0,
#     'MACHO 401.48170.2237':5,
#   'J184246.26-125414.7':7, ###########
#   'ASAS-RCB-15':4,
#   'ASAS-RCB-5':15,
#   'IRAS1813.5-2419':14,
#   'J173819.81-203632.1':5,
#   'J174138.87-161546.4':12,
#   'J174328.50-375029.0':7,
#   'J194218.38-203247.5':9,
#   'SV Sge':6,
#   'V1157 Sgr':0,
#   'EROS2-LMC-RCB-4':2,
#     'MACHO-80.6956.207':1,
#     'MACHO-81.8394.1358':8,
#     'EROS2-CG-RCB-13':13,
#     'WISE-ToI-5031':10,
#     'EROS2-CG-RCB-3':11,
#   'EROS2-SMC-RCB-1':3, #############
#   'ASAS-RCB-1':8,
#   'ASAS-RCB-16':3,
#   'ASAS-RCB-17':6,
#   'ASAS-RCB-2':11,
#   'ASAS-RCB-9':12,
#   'J160205.48-552741.6':13,
#   'J161156.23-575527.1':7,
#   'J182943.83-190246.2':14,
#   'OGLE-GC-RCB-1':2,
#   'S Aps':15,
#   'U Aqr':16,
#   'V1783 Sgr':17,
#   'V2331 Sgr':9,
#   'V739 Sgr':10,
#   'WX CrA':18,
#   'EROS2-LMC-RCB-1':19,
#   'EROS2-LMC-RCB-5':1,
#     'NSV11154':4,
#     'Z Umi':5,
#   'EROS2-SMC-RCB-2':0, ##########
#   'ASAS-RCB-11':9,
#   'HD137613':14,
#   'HD175893':13,
#   'EROS2-LMC-RCB-2':3,
#   'EROS2-LMC-RCB-3':4,
#   'J054123.49-705801.8':0,
#   'B42':8,
#   'A770':1,
#   'A977':7,
#   'J184158.40-054819.2':12,
#   'V4017 Sgr':15,
#     'HE 1015-2050':5,
#     'ASAS J050232-7218.9':2,
#     'J175749.76-075314.9':6,
#     'J181252.50-233304.4':10,
# #     'WISE-ToI-222 G-IR':15,
#   'A814':11, ##############
#   'C27':2, 
#   'GU Sgr':10,
#   'J174119.57-250621.2':12,
#   'RS Tel':9,
#   'J055643.56-715510.7':3,
#   'A223':6,
#   'A183':8,
#   'C38':7,
#   'C20':5, 
#   'A811':4,
#   'C539':11,
#     'dLS17':1,
#   'B563':0, ############
#   'RZ Nor':10,
#   'HD182040':0,
#   'B566':9,
#   'J174851.29-330617.0':11,
#   'HD148839':7,
#   'HD173409':6,
#   'A182':8,
#   'A226':5,
#   'C17':3,
#     'B565':1,
#     'RT Nor':4,
#   'C105':2, ############
#   'VZ Sgr':7,
#   'ASAS-RCB-8':2,
#   'RY Sgr':3,
#   'F75':8,
#   'C526':9,
#   'J185525.52-025145.7':5,
#   'V2552 Oph':6,
#   'V391 Sct':4,
#   'B567':10,
#   'V854 Cen':11,
#     'UX Ant':1,
#     'A249':0,
#     'J175558.51-164744.3':12,
#     'J172447.52-290418.6':13,
# #   'F152':2, ###########
#   'J172951.80-101715.9':5,
#   'V3795 Sgr':4, 
#   'Y Mus':1,
#   'B564':3, 
#     'A798':6,
#     'W Men':0,
#   'A980':2}

In [ ]:
# carbon_index_final_sets = {
#   'ASAS-RCB-18':6,
#   'ASAS-RCB-19':6,
#   'ASAS-RCB-4':6,
#   'ASAS-RCB-7':6,
#   'J132354.47-673720.8':6,
# #   'J182334.24-282957.1':5.5,
#   'J182723.38-200830.1':5.5,
#   'J183649.54-113420.7':5.5,
# #   'ASAS-RCB-5':5,
#     'EROS2-SMC-RCB-3':6,
#     'AO Her':6,
#     'MACHO 401.48170.2237':6,
#   'J184246.26-125414.7':5.5, ###########
#   'ASAS-RCB-15':5.5,
#   'ASAS-RCB-5':5,
#   'IRAS1813.5-2419':5,
#   'J173819.81-203632.1':5.5,
#   'J174138.87-161546.4':5.5,
#   'J174328.50-375029.0':5.5,
#   'J194218.38-203247.5':5.5,
#   'SV Sge':5.5,
#   'V1157 Sgr':5.5,
#   'EROS2-LMC-RCB-4':5.5,
#     'MACHO-80.6956.207':5.5,
#     'MACHO-81.8394.1358':5.5,
#     'EROS2-CG-RCB-13':5.5,
#     'EROS2-CG-RCB-3':5.5,
#     'WISE-ToI-5031':5.5,
#   'EROS2-SMC-RCB-1':5.5, #############
#   'ASAS-RCB-1':5,
#   'ASAS-RCB-16':5.5,
#   'ASAS-RCB-17':5,
#   'ASAS-RCB-2':5,
#   'ASAS-RCB-9':5,
#   'J160205.48-552741.6':5,
#   'J161156.23-575527.1':5,
#   'J182943.83-190246.2':5,
#   'OGLE-GC-RCB-1':5.5,
#   'S Aps':5,
#   'U Aqr':5,
#   'V1783 Sgr':5,
#   'V2331 Sgr':5,
#   'V739 Sgr':5,
#   'WX CrA':5,
#   'EROS2-LMC-RCB-1':5,
#   'EROS2-LMC-RCB-5':5.5,
#     'NSV11154':5.5,
#     'Z Umi':5.5,
#   'EROS2-SMC-RCB-2':5.5, ##########
#   'ASAS-RCB-11':4.5,
#   'HD137613':3.5,
#   'HD175893':3.5,
#   'EROS2-LMC-RCB-2':5,
#   'EROS2-LMC-RCB-3':5,
#   'J054123.49-705801.8':5,
#   'B42':4.5,
#   'A770':5,
#   'A977':4.5,
#   'J184158.40-054819.2':4,
#   'V4017 Sgr':3,
#     'HE 1015-2050':5,
#     'ASAS J050232-7218.9':5,
#     'J175749.76-075314.9':5,
#     'J181836.38-181732.8':5,
#     'J181252.50-233304.4':4.5,
#   'A814':4, ##############
#   'HD182040':4, 
#   'C27':4, 
#   'B566':3,
#   'GU Sgr':3,
#   'J174119.57-250621.2':2,
#   'J174851.29-330617.0':2.5,
#   'RS Tel':3,
#   'J055643.56-715510.7':4,
#   'A223':3,
#   'A183':3,
#   'C38':3,
#   'C20':3.5, 
#   'A811':3.5,
#   'C539':3,
#     'dLS17':4.5,
#   'B563':4.5, ############
#   'RZ Nor':2.5,
#   'HD148839':3,
#   'HD173409':3,
#   'A182':3,
#   'A226':3,
#   'C17':3,
#     'B565':3.5,
#     'RT Nor':3,
#   'C105':3, ############
# #   'J172447.52-290418.6':7,
# #   'J175558.51-164744.3':0,
#   'VZ Sgr':0.5,
#   'ASAS-RCB-8':1,
#   'RY Sgr':1,
#   'F75':0.5,
#   'C526':0.5,
#   'J185525.52-025145.7':0.5,
#   'V2552 Oph':0.5,
#   'V391 Sct':0.5,
#   'B567':0.5,
#   'V854 Cen':0.5,
#     'UX Ant':2,
#     'A249':2.5,
#     'J175558.51-164744.3':0.5,
#     'J172447.52-290418.6':0.5,
# #   'F152':1, ###########
#   'J172951.80-101715.9':0,
#   'V3795 Sgr':0.5, 
#   'Y Mus':0.5,
#   'B564':0.5, 
#   'A980':0.5,
#     'A798':0,
#     'W Men':1}

# HdC IR Colors

In [ ]:
hdc_ir_colors = pd.read_csv('more_hdc_ir_colors.csv')
hdc_ir_colors

In [ ]:
hdc_j_min_w2 = dict()
hdc_v_min_w2 = dict()
# hdc_i_min_w2 = dict()
for i in range(0,len(hdc_ir_colors)):
    hdc_j_min_w2[hdc_ir_colors['Name'][i]] = hdc_ir_colors['J0'][i] - hdc_ir_colors['W20'][i]
    hdc_v_min_w2[hdc_ir_colors['Name'][i]] = hdc_ir_colors['V0'][i] - hdc_ir_colors['W20'][i]
    print(hdc_ir_colors['Name'][i],np.round(hdc_ir_colors['V0'][i] - hdc_ir_colors['W20'][i],3))
#     hdc_i_min_w2[hdc_ir_colors['Name'][i]] = hdc_ir_colors['I0'][i] - hdc_ir_colors['W20'][i]

In [ ]:
hdc_colors = pd.read_csv('HdC_colors.csv')
hdc_colors

In [ ]:
hdc_colors_dict = dict()
for i in range(0,len(hdc_colors)):
    hdc_colors_dict[hdc_colors['Name'][i]] = hdc_colors['V_minus_I'][i]

# hdc_colors_dict

In [ ]:
hdc_colors_dict['ES Aql']

In [ ]:
# temperature_dict = {0:8100,1:7200,2:6200,3:5400,4:4750,5:4300,6:4000,7:3850}
# bad_list = ['A977','A814','HD175893','EROS2-LMC-RCB-5','U Aqr','V1783 Sgr','ASAS-RCB-1',
#            'SV Sge','A166','AO Her','NSV11154','W Men','Y Mus','V482 Cyg','J005010.67-694357.7','ASAS-RCB-3']
bad_list = ['SV Sge','A166','AO Her','W Men','Y Mus','V482 Cyg','J005010.67-694357.7']

for i in range(0,len(store_names)):
    if store_names[i] in final_set_v2_dict.keys() and store_names[i] in hdc_colors_dict.keys() and store_names[i] not in ['ES Aql','V517 Oph']:
#         if store_names[i] in bad_list:
#             pass
#             plt.scatter(final_set_v2_dict[store_names[i]],hdc_colors_dict[store_names[i]],color='tab:red',edgecolors='k')
#             if store_names[i] == 'A814':
#                 plt.annotate(store_names[i],xy=(final_set_v2_dict[store_names[i]]+0.15,hdc_colors_dict[store_names[i]]+0.01),
#                          annotation_clip=False,size=12)
#             elif store_names[i] == 'NSV11154':
#                 plt.annotate(store_names[i],xy=(final_set_v2_dict[store_names[i]]+0.15,hdc_colors_dict[store_names[i]]-0.035),
#                          annotation_clip=False,size=12)
#             elif store_names[i] == 'V1783 Sgr':
#                 plt.annotate(store_names[i],xy=(final_set_v2_dict[store_names[i]]+0.15,hdc_colors_dict[store_names[i]]-0.035),
#                          annotation_clip=False,size=12)
#             else:
#                 plt.annotate(store_names[i],xy=(final_set_v2_dict[store_names[i]]+0.15,hdc_colors_dict[store_names[i]]-0.01),
#                          annotation_clip=False,size=12)
#         else:
        plt.scatter(final_set_v2_dict[store_names[i]],hdc_colors_dict[store_names[i]],color='tab:blue',edgecolors='k')
#         plt.annotate(store_names[i],xy=(final_set_v2_dict[store_names[i]],hdc_colors_dict[store_names[i]]),
#                      annotation_clip=False,size=10)
#         plt.scatter(temperature_dict[final_set_dict[store_names[i]]],hdc_colors_dict[store_names[i]],color='tab:blue')

plt.xlabel('HdC Temperature type')
plt.ylabel(r'Intrinsic V-I$_C$')
# plt.ylim(0.5,1)
# plt.savefig('hdccolors_plot.eps')
plt.show()

In [ ]:
mk_colors = pd.read_csv('color_calibration.csv')
mk_colors

In [ ]:
plt.scatter(mk_colors['Teff'],mk_colors['V-Ic'])

plt.xlabel('MK Temperature')
plt.xlim(9000,3000)
# plt.ylim(0,2.5)
plt.ylabel('(V-I)')
# plt.ylim(1.5,2)
plt.show()

In [ ]:
temperature_dict = {0:8100,1:7200,2:6200,3:5400,4:4750,5:4300,6:4000,7:3850}
for i in range(0,len(store_names)):
    if store_names[i] in final_set_dict.keys() and store_names[i] in hdc_colors_dict.keys():
        plt.scatter(temperature_dict[final_set_dict[store_names[i]]],hdc_colors_dict[store_names[i]],color='tab:blue')
plt.scatter(mk_colors['Teff'],mk_colors['V-Ic'],color='tab:red')

plt.xlabel('Temperature')
plt.xlim(9000,3500)
plt.ylim(0,2.1)
plt.ylabel('(V-I)')
# plt.ylim(1.5,2)
plt.show()

In [ ]:
from scipy import integrate
import scipy

store_v = []
store_i = []
store_v_min_i = []

v_bandpass = np.loadtxt('Bessel_V-1.txt')
i_bandpass = np.loadtxt('Bessel_I-1.txt')
v_wavelength = v_bandpass[:,0]*10
v_wavelength = np.flip(v_wavelength)
i_wavelength = i_bandpass[:,0]*10
i_wavelength = np.flip(i_wavelength)

for i in range(0,len(store_names)):
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])
    left_of_telluric_lam = wavelength[(wavelength < 7575)]
    left_of_telluric_flux = flux[(wavelength < 7575)]
    right_of_telluric_lam = wavelength[(wavelength > 7700)]
    right_of_telluric_flux = flux[(wavelength > 7700)]
    wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
    flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))

    interpolated = interpolate.interp1d(wavelength, flux, bounds_error = False, fill_value = 0)
    
    resampled_flux_v = interpolated(v_wavelength)
    convolution_v = resampled_flux_v * np.flip(v_bandpass[:,1])
    v_integrated = scipy.integrate.trapz(convolution_v,x=v_wavelength)
    v_mag = -2.5*np.log10(v_integrated)
    
    resampled_flux_i = interpolated(i_wavelength)
    convolution_i = resampled_flux_i * np.flip(i_bandpass[:,1])
    i_integrated = scipy.integrate.trapz(convolution_i,x=i_wavelength)
    i_mag = -2.5*np.log10(i_integrated)
    
    store_v.append(v_mag)
    store_i.append(i_mag)
    store_v_min_i.append(v_mag-i_mag)

In [ ]:
bad_list = ['A977','A814','HD175893','EROS2-LMC-RCB-5','U Aqr','V1783 Sgr','ASAS-RCB-1',
           'SV Sge','A166','AO Her','NSV11154','HD137613','W Men','Y Mus','ASAS-RCB-3']

for i in range(0,len(store_names)):
    if store_names[i] in final_set_dict.keys() and store_names[i] in hdc_colors_dict.keys():# or store_names[i] == 'A166':
        if store_names[i] in bad_list:
            plt.scatter(hdc_colors_dict[store_names[i]],store_v_min_i[i],color='tab:red')
        else:
            plt.scatter(hdc_colors_dict[store_names[i]],store_v_min_i[i],color='tab:blue')
        plt.annotate(store_names[i],xy=(hdc_colors_dict[store_names[i]],store_v_min_i[i]),
                     annotation_clip=False,size=10)
#         plt.scatter(temperature_dict[final_set_dict[store_names[i]]],hdc_colors_dict[store_names[i]],color='tab:blue')

plt.xlabel('(V-I)0')
plt.ylabel('pseudo (V-I)')
# plt.ylim(0.5,1)
# plt.savefig('hdccolors_compare')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in final_set_dict.keys() and store_names[i] in hdc_colors_dict.keys():
        if store_names[i] in bad_list:
            plt.scatter(final_set_dict[store_names[i]],store_v_min_i[i],color='tab:red')
        else:
            plt.scatter(final_set_dict[store_names[i]],store_v_min_i[i],color='tab:blue')
        plt.annotate(store_names[i],xy=(final_set_dict[store_names[i]],store_v_min_i[i]),
                     annotation_clip=False,size=10)

plt.xlabel('HdC Temperature type')
plt.ylabel('Pseudo (V-I)')
# plt.ylim(0.5,1)
# plt.savefig('hdccolors_pseudo')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in final_set_dict.keys() and store_names[i] in hdc_j_min_w2.keys():
        if store_names[i] in bad_list:
            plt.scatter(final_set_dict[store_names[i]],hdc_j_min_w2[store_names[i]],color='tab:red')
        else:
            plt.scatter(final_set_dict[store_names[i]],hdc_j_min_w2[store_names[i]],color='tab:blue')
        plt.annotate(store_names[i],xy=(final_set_dict[store_names[i]],hdc_j_min_w2[store_names[i]]),
                     annotation_clip=False,size=10)

plt.ylabel('(J-W2)0')
plt.xlabel('HdC Temperature Class')
plt.savefig('J-W2_v_class')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in hdc_colors_dict.keys() and store_names[i] in hdc_j_min_w2.keys():
        if store_names[i] in bad_list:
            plt.scatter(hdc_colors_dict[store_names[i]],hdc_j_min_w2[store_names[i]],color='tab:red')
        else:
            plt.scatter(hdc_colors_dict[store_names[i]],hdc_j_min_w2[store_names[i]],color='tab:blue')
        plt.annotate(store_names[i],xy=(hdc_colors_dict[store_names[i]],hdc_j_min_w2[store_names[i]]),
                     annotation_clip=False,size=10)

plt.ylabel('(J-W2)0')
plt.xlabel('(V-I)0')
plt.savefig('J-W2_v_V-I')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in final_set_dict.keys() and store_names[i] in hdc_v_min_w2.keys():
        if not np.isnan(hdc_v_min_w2[store_names[i]]):
            if store_names[i] in bad_list:
                plt.scatter(final_set_dict[store_names[i]],hdc_v_min_w2[store_names[i]],color='tab:red')
            else:
                plt.scatter(final_set_dict[store_names[i]],hdc_v_min_w2[store_names[i]],color='tab:blue')
            plt.annotate(store_names[i],xy=(final_set_dict[store_names[i]],hdc_v_min_w2[store_names[i]]),
                         annotation_clip=False,size=10)

plt.ylabel('(V-W2)0')
plt.xlabel('HdC Temperature Class')
plt.savefig('V-W2_v_class')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in hdc_colors_dict.keys() and store_names[i] in hdc_v_min_w2.keys():
        if not np.isnan(hdc_v_min_w2[store_names[i]]):
            if store_names[i] in bad_list:
                plt.scatter(hdc_colors_dict[store_names[i]],hdc_v_min_w2[store_names[i]],color='tab:red')
            else:
                plt.scatter(hdc_colors_dict[store_names[i]],hdc_v_min_w2[store_names[i]],color='tab:blue')
            plt.annotate(store_names[i],xy=(hdc_colors_dict[store_names[i]],hdc_v_min_w2[store_names[i]]),
                         annotation_clip=False,size=10)

plt.ylabel('(V-W2)0')
plt.xlabel('(V-I)0')
plt.savefig('V-W2_v_V-I')
plt.show()

In [ ]:
temperature_dict = {0:8100,1:7200,2:6200,3:5400,4:4750,5:4300,6:4000,7:3850}

for i in range(0,len(mk_colors)):
    if mk_colors['Ks-W1'][i] != '...' and mk_colors['W1-W2'][i] != '...':
        plt.scatter(mk_colors['Teff'][i],mk_colors['V-Ks'][i]-float(mk_colors['Ks-W1'][i])-float(mk_colors['W1-W2'][i]),
                   color='tab:red')
        
        
for i in range(0,len(store_names)):
    if store_names[i] in hdc_colors_dict.keys() and store_names[i] in hdc_v_min_w2.keys() and store_names[i] in final_set_dict.keys():
        if not np.isnan(hdc_v_min_w2[store_names[i]]):
            plt.scatter(temperature_dict[final_set_dict[store_names[i]]],hdc_v_min_w2[store_names[i]],color='tab:blue')
            plt.annotate(store_names[i],xy=(temperature_dict[final_set_dict[store_names[i]]],hdc_v_min_w2[store_names[i]]),
                         annotation_clip=False,size=10)
            
            
plt.xlabel('MK Temperature')
plt.xlim(9000,3000)
# plt.ylim(0,2.5)
plt.ylim(top=7.5)
plt.ylabel('(V-W2)')
plt.show()

In [ ]:
for i in range(0,len(mk_colors)):
    if mk_colors['Ks-W1'][i] != '...' and mk_colors['W1-W2'][i] != '...' and mk_colors['J-H'][i] != '...' and mk_colors['H-Ks'][i] != '...':
        plt.scatter(mk_colors['Teff'][i],float(mk_colors['J-H'][i])-float(mk_colors['H-Ks'][i])-float(mk_colors['Ks-W1'][i])-float(mk_colors['W1-W2'][i]),
                   color='tab:red')
        
for i in range(0,len(store_names)):
    if store_names[i] in hdc_colors_dict.keys() and store_names[i] in hdc_j_min_w2.keys() and store_names[i] in final_set_dict.keys():
        if not np.isnan(hdc_j_min_w2[store_names[i]]):
            plt.scatter(temperature_dict[final_set_dict[store_names[i]]],hdc_j_min_w2[store_names[i]],color='tab:blue')
            plt.annotate(store_names[i],xy=(temperature_dict[final_set_dict[store_names[i]]],hdc_j_min_w2[store_names[i]]),
                         annotation_clip=False,size=10)

plt.xlabel('MK Temperature')
plt.xlim(9000,3000)
# plt.ylim(0,2.5)
plt.ylabel('(J-W2)')
plt.show()

In [ ]:
bad_list = ['A977','A814','HD175893','EROS-LMC-RCB-5','U Aqr','V1783 Sgr','ASAS-RCB-1',
           'SV Sge','A166','AO Her','NSV11154','HD137613','W Men','Y Mus','ASAS-RCB-3']

bergeat_temps = {'SV Sge':4010,'XX Cam':6900,'U Aqr':5000,'SU Tau':6900,'RY Sgr':6900,'R CrB':6100,
                'HD137613':4725,'HD182040':5590,'HD148839':5625,'HD175893':5640}

literature_temps = {'ES Aql':5000,'FH Sct':6250,'GU Sgr':6250,'NSV11154':5200,
                    'R CrB':6750,'RS Tel':6750,'RT Nor':7000,'RY Sgr':7250,'RZ Nor':6750,
                    'S Aps':5400,'SU Tau':6500,'SV Sge':5000,'U Aqr':6000,'UV Cas':7250,
                    'UW Cen':7500,'UX Ant':7000,'V CrA':6250,'V2552 Oph':6750,'V3795 Sgr':8000,
                    'V482 Cyg':6500,'V854 Cen':6750,'VZ Sgr':7000,'WX CrA':5300,'XX Cam':7250,
                    'Y Mus':7250,'Z Umi':5250,'HD137613':5400,'HD148839':5625,'HD173409':6100,
                    'HD175893':5640,'HD182040':5590,'W Men':6700,'HV 12842':6800}


temperature_dict = {0:8100,1:7200,2:6200,3:5400,4:4750,5:4300,6:4000,7:3850}

for i in range(0,len(store_names)):
    if store_names[i] in literature_temps.keys() and store_names[i] in hdc_colors_dict.keys() and store_names[i] in final_set_dict.keys():
        if store_names[i] in bad_list:
            plt.scatter(temperature_dict[final_set_dict[store_names[i]]],literature_temps[store_names[i]],color='tab:red')
        else:
            plt.scatter(temperature_dict[final_set_dict[store_names[i]]],literature_temps[store_names[i]],color='tab:blue')
        plt.annotate(store_names[i],xy=(temperature_dict[final_set_dict[store_names[i]]],literature_temps[store_names[i]]),
                     annotation_clip=False,size=10)

plt.xlabel('V-I temp estimate')
plt.ylabel('literature temps')
plt.show()

In [ ]:
bad_list = ['A977','A814','HD175893','EROS2-LMC-RCB-5','U Aqr','V1783 Sgr','ASAS-RCB-1',
           'SV Sge','A166','AO Her','NSV11154','HD137613','W Men','Y Mus','ASAS-RCB-3']

for i in range(0,len(store_names)):
    if store_names[i] in bad_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux)
        plt.title(store_names[i])
        plt.show()

In [ ]:
declinedata = pd.read_csv('rcbchart.csv')
declinedata

In [ ]:
for i in range(0,len(declinedata['Star'])):
    if declinedata['Star'][i] in final_set_dict.keys() and declinedata['Star'][i] in hdc_colors_dict.keys():
        plt.scatter(hdc_colors_dict[declinedata['Star'][i]],declinedata['decline_p_year'][i],color='tab:blue')
        plt.annotate(declinedata['Star'][i],xy=(hdc_colors_dict[declinedata['Star'][i]],declinedata['decline_p_year'][i]),
                     annotation_clip=False,size=10)

plt.xlabel('(V-I)0')
plt.ylabel('Declines per year')
# plt.ylim(0.5,1)
# plt.savefig('hdccolors_plot')
plt.show()

# Carbon Stars

In [ ]:
os.chdir('/Users/crawford/Desktop/Barnbaum_spectra')
sp_table = pd.read_csv('table_4.csv')
os.chdir('/Users/crawford/Desktop')

barnbaum_spec = sp_table[sp_table['SpFile1'].notnull()]
barnbaum_spec.reset_index(drop=True,inplace=True)
barnbaum_spec

In [ ]:
barnbaum_Av_values = []
for i in range(0,len(barnbaum_spec)):
    rastring = barnbaum_spec['RA'][i]
    decstring = barnbaum_spec['Dec'][i]
    coordstring = rastring + ' ' + decstring + ' 00'
#     print(':',coordstring,':')
    coo = coord.SkyCoord(coordstring, unit=(u.hourangle, u.deg))
    table = IrsaDust.get_extinction_table(coo)
    barnbaum_Av_values.append(table['A_SandF'][2])

# barnbaum_Av_values

In [ ]:
for i in range(0,len(barnbaum_spec)):
    print(barnbaum_spec['Name'][i],barnbaum_Av_values[i])

In [ ]:
os.chdir('/Users/crawford/Desktop/Barnbaum_spectra')
os.chdir('sp')
barnbaum_stars_lam = []
barnbaum_stars_flux = []
for i in range(0,len(barnbaum_spec)):
#     f,ax = plt.subplots(2, 1, figsize=[12,8], constrained_layout=True)
#     if barnbaum_spec['Name'][i] == 'Z Psc':
    f,ax = plt.subplots(2, 1, figsize=[12,8], constrained_layout=True)
    if type(barnbaum_spec['SpFile2'][i]) == float:
        spec1 = np.loadtxt(barnbaum_spec['SpFile1'][i])
        lam = spec1[:,0]
        flux = spec1[:,1]
#         plt.plot(lam,flux)
        ax[0].plot(lam,flux)
        ax[0].set_xlim(3000,6500)

        ax[1].plot(lam,flux)
        ax[1].set_xlim(6500,10000)
#         plt.title(barnbaum_spec['SpFile1'][i])
#         plt.show()
        store_lam = lam
        store_flux = flux
        barnbaum_stars_lam.append(store_lam)
        barnbaum_stars_flux.append(store_flux)
    else:
        spec1 = np.loadtxt(barnbaum_spec['SpFile1'][i])
        spec2 = np.loadtxt(barnbaum_spec['SpFile2'][i])
        lam_1 = spec1[:,0]
        flux_1 = spec1[:,1]
        lam_2 = spec2[:,0]
        flux_2 = spec2[:,1]

        overlap = np.trim_zeros(np.where(lam_1>lam_2[0],lam_1,0))
        scale_factor = flux_2[0]/flux_1[-len(overlap)]

#         plt.plot(lam1,flux1*scale_factor)
#         plt.plot(lam2,flux2)

        ax[0].plot(lam_1,flux_1*scale_factor)
        ax[0].plot(lam_2,flux_2)
        ax[0].set_xlim(3000,6500)

        ax[1].plot(lam_1,flux_1*scale_factor)
        ax[1].plot(lam_2,flux_2)
        ax[1].set_xlim(6500,10000)
        store_lam = np.concatenate((np.asarray(lam_1),np.asarray(lam_2)))
        store_flux = np.concatenate((np.asarray(flux_1*scale_factor),np.asarray(flux_2)))
        barnbaum_stars_lam.append(store_lam)
        barnbaum_stars_flux.append(store_flux)
    #         plt.title(barnbaum_spec['SpFile1'][i])
    #         plt.show()
    plt.suptitle(barnbaum_spec['SpFile1'][i])
    ax[1].set_xlabel(r'Wavelength ($\AA$)')
    ax[0].set_ylabel('Flux')
    ax[1].set_ylabel('Flux')
    #     plt.savefig('%s_stretch' % barnbaum_spec['Name'][i])
    plt.show()
        
        
#         store_lam = np.concatenate((np.asarray(lam_1),np.asarray(lam_2)))
#         store_flux = np.concatenate((np.asarray(flux_1*scale_factor),np.asarray(flux_2)))
#         store_lam = lam
#         store_flux = flux
        
# else:
#     pass

os.chdir('/Users/crawford/Desktop')

In [ ]:
# ext = CCM89(Rv=3.1)

# store_flux = np.asarray(store_flux/ext.extinguish(store_lam*u.AA,Av=1.15))
# plt.plot(store_lam,store_flux*2/np.max(store_flux))
# plt.axvline(4737,color='k')
# plt.axvline(4861,color='k')
# plt.axvline(4990,color='k')

# plt.xlim(4500,5300)
# plt.show()

# Carbon Index Calibration

In [ ]:
barn_5635 = []
for i in range(0,len(barnbaum_stars_lam)):
    print(barnbaum_spec['Name'][i],barnbaum_spec['C2'][i])
    eq_width, index = spec.calculate_band_eq_width_observations(barnbaum_stars_lam[i],
                                                                barnbaum_stars_flux[i]/np.max(barnbaum_stars_flux[i]),
                                                                barnbaum_spec['Name'][i],
                                                               [5320,5640],[5175,5245],[5640,5675])
    barn_5635.append(eq_width)

In [ ]:
barn_5165 = []
for i in range(0,len(barnbaum_stars_lam)):
    print(barnbaum_spec['Name'][i],barnbaum_spec['C2'][i])
    eq_width, index = spec.calculate_band_eq_width_observations(barnbaum_stars_lam[i],
                                                                barnbaum_stars_flux[i]/np.max(barnbaum_stars_flux[i]),
                                                                barnbaum_spec['Name'][i],
                                                               [4900,5175],[4740,4835],[5175,5200])
    barn_5165.append(eq_width)

In [ ]:
barn_4737 = []
for i in range(0,len(barnbaum_stars_lam)):
    print(barnbaum_spec['Name'][i],barnbaum_spec['C2'][i])
    eq_width, index = spec.calculate_band_eq_width_observations(barnbaum_stars_lam[i],
                                                                barnbaum_stars_flux[i]/np.max(barnbaum_stars_flux[i]),
                                                                barnbaum_spec['Name'][i],
                                                               [4450,4745],[4400,4450],[4740,4835])
    barn_4737.append(eq_width)

In [ ]:
C2_index = [5.9,1.5,5,5,4.5,4.5,5,4,4.5,5.5,5.5,5.5,5.9,5.5,4.5,5.1,
            6,5.5,6,5.9,5.1,6,5.1,4.1,4.9,5.5,4.1,5.9,5.9,5,4.9,4.9,
            5.5,5.9,5.5,5.9,4,4,4.9,5.9,6,1.5,1.5,2.5,3,5,3,4.5,5.5,
            5,5.5,5,5.5,4,5,3,2.9,2,4.5,4.5,5,5,4]

plt.scatter(C2_index,barn_5635,label='5635')
plt.scatter(C2_index,barn_5165,label='5165')
plt.scatter(C2_index,barn_4737,label='4737')
# plt.yscale('log')
plt.legend()
plt.xlabel('C2 index')
plt.ylabel(r'Band Pseudo-Eq. Width ($\AA$)')
plt.show()

In [ ]:
for i in range(0,len(barnbaum_stars_lam)):
    print(barnbaum_spec['Name'][i],C2_index[i],barn_5635[i],barn_5165[i],barn_4737[i])

In [ ]:
eq_1 = []
eq_2 = []
eq_3 = []
for i in range(0,len(barnbaum_stars_lam)):
    if C2_index[i] < 1.8 and C2_index[i] > 1.3:
        eq_1.append(barn_5635[i])
        eq_2.append(barn_5165[i])
        eq_3.append(barn_4737[i])
print(np.round(np.mean(eq_1),2), '(', np.min(eq_1), '-', np.max(eq_1), ')')
print(np.round(np.mean(eq_2),2), '(', np.min(eq_2), '-', np.max(eq_2), ')')
print(np.round(np.mean(eq_3),2), '(', np.min(eq_3), '-', np.max(eq_3), ')')
# print(np.round(np.mean(eq_1),2),np.round(np.mean(eq_2),2),np.round(np.mean(eq_3),2))
# print(np.min(eq_1),np.max(eq_1))
# print(np.min(eq_2),np.max(eq_2))
# print(np.min(eq_3),np.max(eq_3))

In [ ]:
obs_C2_5635_band_area = []

for i in range(0,len(store_names)):
#     if store_names[i] in good_names:
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])

    flux = flux/np.max(flux)

    if np.min(wavelength) > 4600:
        print(store_names[i], 'not enough spectrum, setting EW to -99')
        eq_width = -99
        index = -99
    else:
        try:
            eq_width, index = spec.calculate_band_eq_width_observations(wavelength,flux,store_names[i],
                                                                   [5320,5640],[5175,5245],[5640,5675])
        except TypeError:
            print(store_names[i], 'not enough spectrum, setting EW to -99')
            eq_width = -99
            index = -99

    obs_C2_5635_band_area.append(eq_width)
#     else:
#         obs_C2_5635_band_area.append(-99)

In [ ]:
obs_C2_5165_band_area = []

for i in range(0,len(store_names)):
#     if store_names[i] in good_names:
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])

    flux = flux/np.max(flux)

    if np.min(wavelength) > 4600:
        print(store_names[i], 'not enough spectrum, setting EW to -99')
        eq_width = -99
        index = -99
    else:
        try:
            eq_width, index = spec.calculate_band_eq_width_observations(wavelength,flux,store_names[i],
                                                                   [4900,5175],[4740,4835],[5175,5200])
        except TypeError:
            print(store_names[i], 'not enough spectrum, setting EW to -99')
            eq_width = -99
            index = -99

    obs_C2_5165_band_area.append(eq_width)
#     else:
#         obs_C2_5165_band_area.append(-99)


In [ ]:
obs_C2_4737_band_area = []

for i in range(0,len(store_names)):
#     if store_names[i] in good_names:
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])

    flux = flux/np.max(flux)

    if np.min(wavelength) > 4600:
        print(store_names[i], 'not enough spectrum, setting EW to -99')
        eq_width = -99
        index = -99
    else:
        try:
            eq_width, index = spec.calculate_band_eq_width_observations(wavelength,flux,store_names[i],
                                                                   [4450,4745],[4400,4450],[4740,4835])
        except TypeError:
            print(store_names[i], 'not enough spectrum, setting EW to -99')
            eq_width = -99
            index = -99

    obs_C2_4737_band_area.append(eq_width)
#     else:
#         obs_C2_4737_band_area.append(-99)



In [ ]:
# new_classes = ['ASAS-RCB-12','ASAS-RCB-14','ASAS-RCB-20','EROS2-CG-RCB-11',
#  'EROS2-CG-RCB-4','EROS2-CG-RCB-5','J150104.50-563325.1','J173202.75-432906.1',
#  'J173553.02-364104.3','J175521.75-281131.2','J181538.25-203845.7',
#  'J182334.24-282957.1','MACHO 135.27132.51','MACHO 301.45783.9',
#  'MACHO 308.38099.66','R CrB','V517 Oph','EROS2-SMC-RCB-3',
#  'J005010.67-694357.7','J005113.58-731036.3','J054221.91-690259.3',
#  'C528','C542','F152','M37','M38','P12','ASAS-RCB-3','ASAS-RCB-6',
#  'EROS2-CG-RCB-14','EROS2-CG-RCB-6','EROS2-CG-RCB-7','EROS2-CG-RCB-9',
#  'ES Aql','FH Sct','J110008.77-600303.6','J163450.35-380218.5',
#  'J164704.67-470817.8','J170343.87-385126.6','J171815.36-341339.9',
#  'J171908.50-435044.6','J172553.80-312421.1','J174111.80-281955.3',
#  'J174257.19-362052.1','J174645.90-250314.1','J175031.70-233945.7',
#  'J175107.12-242357.3','J175749.98-182522.8','J180550.49-151301.7',
#  'UV Cas','UW Cen','V CrA','V482 Cyg','EROS2-LMC-RCB-6','EROS2-SMC-RCB-3',
#  'J005010.67-694357.7','J173737.07-072828.1','J181836.38-181732.8','A166','dLS114']

In [ ]:
# for i in range(0,len(store_names)):
#     if store_names[i] in new_classes:
#         print(store_names[i], obs_C2_5635_band_area[i], obs_C2_5165_band_area[i], obs_C2_4737_band_area[i])

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classing_set_names:
        if store_names[i] in final_sets[0]:
            plt.scatter(0,obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(0,obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(0,obs_C2_4737_band_area[i],color='tab:green')
        elif store_names[i] in final_sets[1]:
            plt.scatter(1,obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(1,obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(1,obs_C2_4737_band_area[i],color='tab:green')
        elif store_names[i] in final_sets[2]:
            plt.scatter(2,obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(2,obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(2,obs_C2_4737_band_area[i],color='tab:green')
        elif store_names[i] in final_sets[3]:
            plt.scatter(3,obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(3,obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(3,obs_C2_4737_band_area[i],color='tab:green')
        elif store_names[i] in final_sets[4]:
            plt.scatter(4,obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(4,obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(4,obs_C2_4737_band_area[i],color='tab:green')
        elif store_names[i] in final_sets[5]:
            plt.scatter(5,obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(5,obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(5,obs_C2_4737_band_area[i],color='tab:green')
        elif store_names[i] in final_sets[6]:
            plt.scatter(6,obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(6,obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(6,obs_C2_4737_band_area[i],color='tab:green')
        elif store_names[i] in final_sets[7]:
            plt.scatter(7,obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(7,obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(7,obs_C2_4737_band_area[i],color='tab:green')
            
plt.show()

In [ ]:
xx = 0
for i in range(0,len(store_names)):
    if store_names[i] in carbon_index_final_sets.keys():
        if xx == 0:
            plt.scatter(carbon_index_final_sets[store_names[i]],obs_C2_5635_band_area[i],color='tab:blue',label='5635')
            plt.scatter(carbon_index_final_sets[store_names[i]],obs_C2_5165_band_area[i],color='tab:orange',label='5165')
            plt.scatter(carbon_index_final_sets[store_names[i]],obs_C2_4737_band_area[i],color='tab:green',label='4737')
        else:
            plt.scatter(carbon_index_final_sets[store_names[i]],obs_C2_5635_band_area[i],color='tab:blue')
            plt.scatter(carbon_index_final_sets[store_names[i]],obs_C2_5165_band_area[i],color='tab:orange')
            plt.scatter(carbon_index_final_sets[store_names[i]],obs_C2_4737_band_area[i],color='tab:green')
        xx = xx+1
        
plt.legend()
plt.ylim(bottom=-20)
plt.xlabel('C2 index')
plt.ylabel(r'Band Pseudo-Eq. Width ($\AA$)')
plt.show()

In [ ]:
# for i in range(0,len(store_names)):
#     if store_names[i] in classing_set_names:
#         if store_names[i] in final_sets[0]:
#             plt.scatter(0,carbon_index_final_sets[store_names[i]],color='tab:blue')
#         elif store_names[i] in final_sets[1]:
#             plt.scatter(1,carbon_index_final_sets[store_names[i]],color='tab:blue')
#         elif store_names[i] in final_sets[2]:
#             plt.scatter(2,carbon_index_final_sets[store_names[i]],color='tab:blue')
#         elif store_names[i] in final_sets[3]:
#             plt.scatter(3,carbon_index_final_sets[store_names[i]],color='tab:blue')
#         elif store_names[i] in final_sets[4]:
#             plt.scatter(4,carbon_index_final_sets[store_names[i]],color='tab:blue')
#         elif store_names[i] in final_sets[5]:
#             plt.scatter(5,carbon_index_final_sets[store_names[i]],color='tab:blue')
#         elif store_names[i] in final_sets[6]:
#             plt.scatter(6,carbon_index_final_sets[store_names[i]],color='tab:blue')
#         elif store_names[i] in final_sets[7]:
#             plt.scatter(7,carbon_index_final_sets[store_names[i]],color='tab:blue')
            
# plt.show()

In [ ]:
carbon_indexes = [0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6]

for xx in carbon_indexes:
    try:
        eq_1 = []
        eq_2 = []
        eq_3 = []
        for i in range(0,len(store_names)):
            if store_names[i] in carbon_index_final_sets.keys() and carbon_index_final_sets[store_names[i]] == xx:
                if obs_C2_5635_band_area[i] != -99:
                    eq_1.append(obs_C2_5635_band_area[i])
                if obs_C2_5165_band_area[i] != -99:
                    eq_2.append(obs_C2_5165_band_area[i])
                if obs_C2_4737_band_area[i] != -99:
                    eq_3.append(obs_C2_4737_band_area[i])
        print('Carbon index',xx)
        print(np.round(np.mean(eq_1),2), '(', np.min(eq_1), '-', np.max(eq_1), ')')
        print(np.round(np.mean(eq_2),2), '(', np.min(eq_2), '-', np.max(eq_2), ')')
        print(np.round(np.mean(eq_3),2), '(', np.min(eq_3), '-', np.max(eq_3), ')')
    except ValueError:
        pass

In [ ]:
# V517_sed = np.asarray(((0.44, 2.0e-14),(0.55, 7.5e-14)))#,(0.64, 4.719e-14),
# #             (0.80, 5.123e-14),(1.22, 2.38e-14),(1.63, 1.09e-14),
# #             (2.19, 4.95e-15)))

In [ ]:
# for i in range(0,len(store_names)):
#     if store_names[i] in ['V517 Oph']:
#         wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
#                                                          0.0,store_names[i])
        
#         plt.scatter(V517_sed[:,0]*10000, V517_sed[:,1],color='tab:red')

#         plt.plot(wavelength,flux)

# plt.ylim(0,2.5e-13)
# # plt.xlim(3500,10000)
# plt.show()

In [ ]:
# Mv_dict = dict()
# for i in range(0,len(master_w_spec)):
#     if master_w_spec['Name'][i] in store_names:
#         mv = master_w_spec['Vmax'][i] - master_w_spec['Av'][i] - 5*np.log10(master_w_spec['Distance'][i]*1000) + 5
#         if not np.isnan(mv):
#             Mv_dict[master_w_spec['Name'][i]] = np.round(mv,2)

In [ ]:
# Mv_dict

In [ ]:
# group0_list = ['ASAS-RCB-20','ASAS-RCB-19','ASAS-RCB-14','J05422191-690259.3','J173202.75-432906.1',
#                 'J182334.24-282957.1','J184246.26-125414.7','J182723.38-200830.1','V517 Oph']

# group0half_list = ['ASAS-RCB-18','EROS2-SMC-RCB-3','J132354.47-673720.8','ASAS-RCB-4','ASAS-RCB-7',
#               'J174138.87-161546.4','J183649.54-113420.7','ASAS-RCB-15','ASAS-RCB-5','ASAS-RCB-3']

# group1_list = ['SV Sge','S Aps','WX CrA','V1783 Sgr','V1157 Sgr','OGLE-GC-RCB-1',
#               'J182943.83-190246.2','J174328.50-375029.0',
#               'J173819.81-203632.1','J161156.23-575527.1','J160205.48-552741.6',
#                'EROS2-LMC-RCB-1','ASAS-RCB-17','ASAS-RCB-16','ASAS-RCB-11','ASAS-RCB-9',
#               'ASAS-RCB-2','ASAS-RCB-1','V2331 Sgr']

# group1lowcn_list = ['EROS2-LMC-RCB-2','A770','A977']

# group1half_list = ['EROS2-SMC-RCB-1','EROS2-LMC-RCB-4','J194218.38-203247.5','V739 Sgr','IRAS1813.5-2419']

# group2_list = ['U Aqr','EROS2-SMC-RCB-2','EROS2-LMC-RCB-5','EROS2-LMC-RCB-3','J054123.49-705801.8']

# weird_group = ['ASAS-RCB-6','A166']

# group2half_list = ['HE 1015-2050','J055643.56-715510.7','HD182040','A223','C27','A811']

# group3_list = ['HD175893','HD137613','J184158.40-054819.2',
#               'B42','V4017 Sgr','C38']

# group3half_list = ['RT Nor','J184158.40-054819.2','C38','A814']

# group4half_list = ['ASAS-RCB-12','A814','C542']

# group4_list = ['HD173409','HD148839','B563','A226','A183',
#               'C17','C20','C105','B566','J174645.90-250314.1']

# group5_list = ['GU Sgr','RS Tel','V CrA','J005010.67-694357.7','J174851.29-330617.0',
#               'C539']

# group5half_list = ['B565','A249']

# group6_list = ['C528','UX Ant']

# group6half_list = ['J174119.57-250621.2','FH Sct','J175031.70-233945.7']

# group7_list = ['RZ Nor','A182']

# group7half_list = ['ASAS-RCB-8','B567','F152']

# group8_list = ['UW Cen','J185525.52-025145.7']

# group9_list = ['RY Sgr','V2552 Oph']

# group10_list = ['F75','C526','J175558.51-164744.3','VZ Sgr']

# group10half_list = ['Y Mus','B564','A980','C1004']

# group11_list = ['V3795 Sgr','J172447.52-290418.6','J172951.80-101715.9'] #A798 is an EHe

# Some Model Work

In [ ]:
model = np.loadtxt('models/MODELS_VIS_Courtney-Sept2020/'+all_models['O1618-10_C1213-inf_N80']['5000'],skiprows=3)
model_lam = model[:,0]
model_flux = model[:,1]
model_flux_1 = convolve(model_flux,Gaussian1DKernel(1))
plt.plot(model_lam,model_flux_1,label='Normal')



model = np.loadtxt('models/MODELS_VIS_Courtney-Jan2021-CN-or-C2-ONLY-WithoutBB/5000g1.0z+0.00m1.0t05ptin80_1000-25000_xit1.0_O1618-10.0_C1213-inf_N80__Only-C2.spec_0.3-1.0_res1ang'
                   ,skiprows=3)
model_lam = model[:,0]
model_flux = model[:,1]
model_flux_2 = convolve(model_flux,Gaussian1DKernel(1))
# plt.plot(model_lam,model_flux_2,label='C2')

model_no_c2 = (model_flux_1-model_flux_2)+1
model_10percent = model_no_c2/10 + model_flux_2

# plt.plot(model_lam,model_no_c2,label='subtracted')
plt.plot(model_lam,model_10percent,label='adjusted')



plt.xlim(6000,6200)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,25))
colors = plt.cm.viridis(np.linspace(0,0.8,19))
x=0
for temp in list(all_models['O1618-10_C1213-inf_N75'].keys()):
    model = np.loadtxt('models/MODELS_VIS_Courtney-Sept2020/'+all_models['O1618-10_C1213-inf_N75'][temp],skiprows=3)
    model_lam = model[:,0]
    model_flux = model[:,1]
    model_flux = convolve(model_flux,Gaussian1DKernel(1))
    
    greater = np.where(model_lam>6600)
    plt.plot(model_lam,model_flux/np.average(model_flux[greater][:100])-x,color=colors[int(x)])
    plt.annotate(temp,xy=(6510,-x+1),annotation_clip=False,size=15)
    x=x+1
    
plt.xlim(6000,6500)
plt.ylim(-18.5,3.5)
plt.yticks([])
plt.show()

# PCA and K-means

In [ ]:
# this set includes all full spectra besides a few known weird guys 
# (maybe not super useful anymore, but it's the original set used to make the PCA projection)

# this is used to make the kmeans plot as well, so I edited it to only call the classified stars
# ie only use those that are not in declines

general_wavelength = np.linspace(4500,9000,10000)
classified_spectra = []
smallspectra = []
classing_set_names = []
classing_indexes = []

for i in range(0,len(store_names)):
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])
#     if store_names[i] not in final_set_dict.keys():
    if np.min(wavelength) > 4500 or np.max(wavelength)<9000 or store_names[i] not in final_set_v2_dict.keys():
#     if np.min(wavelength) > 4500 or np.max(wavelength)<9000 or store_names[i] in ['A166','ASAS-RCB-6',
#                                                                                   'J171908.50-435044.6',
#                                                                                   'A208','dLS114',
#                                                                                   'MACHO-80.6956.207',
#                                                                                   'ASAS-DYPer-1','A798']:
#         smallspectra.append('square')
            pass
    else:
        if store_names[i] in order_pca.keys():
            smallspectra.append('square')
        else:
            smallspectra.append('circle')

        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

        interpolated = interpolate.interp1d(wavelength, flux, bounds_error = False, fill_value = 0)
        classified_spectra.append(interpolated(general_wavelength))
        classing_set_names.append(store_names[i])
        classing_indexes.append(i)

classified_spectra = np.asarray(classified_spectra)
smallspectra = np.asarray(smallspectra)

In [ ]:
X = preprocessing.normalize(classified_spectra)
mu = X.mean(0)
std = X.std(0)

In [ ]:
plt.plot(general_wavelength, mu, color='black')
plt.fill_between(general_wavelength, mu - std, mu + std, color='#CCCCCC')
plt.xlim(general_wavelength[0], general_wavelength[-1])
# plt.ylim(0, 0.06)
plt.xlabel('wavelength (Angstroms)')
plt.ylabel('scaled flux')
plt.title('Mean Spectrum')
plt.show()

In [ ]:
# rpca = PCA(n_components=50, svd_solver='randomized')
# pk.dump(rpca, open("store_pca.pkl","wb"))
# X_proj = rpca.fit_transform(X)
# X_proj.shape

In [ ]:
rpca = pk.load(open("store_pca.pkl",'rb'))
X_proj = rpca.fit_transform(X)
X_proj.shape

In [ ]:
# plt.figure()
# colors = plt.cm.rainbow(np.linspace(0,1,13))
# for i in range(0,len(classing_set_names)):
#     if class_index[i] == -5:
#         plt.scatter(X_proj[i, 0], X_proj[i, 1], s=50, lw=2,marker='o',edgecolors='k', facecolors='none')
#     elif class_index[i] == 17:
#         plt.scatter(X_proj[i, 0], X_proj[i, 1], s=50, lw=2,marker='s',edgecolors='k', facecolors='none')
#     else:
#         plt.scatter(X_proj[i, 0], X_proj[i, 1], s=75, lw=0,c=colors[class_index[i]],cmap=plt.cm.jet)
# # plt.colorbar()
# plt.xlabel('coefficient 1')
# plt.ylabel('coefficient 2')
# plt.title('PCA projection of Spectra')
# plt.show()

In [ ]:
# f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
# colors = plt.cm.rainbow(np.linspace(0,1,13))
# for i in range(0,len(classing_set_names)):
#     if class_index[i] == -5:
#         ax[0].scatter(X_proj[i, 0], X_proj[i, 1], s=50, lw=2,marker='o',edgecolors='k', facecolors='none')
#         ax[1].scatter(X_proj[i, 0], X_proj[i, 2], s=50, lw=2,marker='o',edgecolors='k', facecolors='none')
# #         ax[2].scatter(X_proj[i, 1], X_proj[i, 2], s=50, lw=2,marker='o',edgecolors='k', facecolors='none')
#     elif class_index[i] == 17:
#         ax[0].scatter(X_proj[i, 0], X_proj[i, 1], s=50, lw=2,marker='s',edgecolors='k', facecolors='none')
#         ax[1].scatter(X_proj[i, 0], X_proj[i, 2], s=50, lw=2,marker='s',edgecolors='k', facecolors='none')
# #         ax[2].scatter(X_proj[i, 1], X_proj[i, 2], s=50, lw=2,marker='s',edgecolors='k', facecolors='none')
#     else:
#         ax[0].scatter(X_proj[i, 0], X_proj[i, 1], s=75, lw=0,c=colors[class_index[i]])
#         ax[1].scatter(X_proj[i, 0], X_proj[i, 2], s=75, lw=0,c=colors[class_index[i]])
# #         ax[2].scatter(X_proj[i, 1], X_proj[i, 2], s=75, lw=0,c=colors[class_index[i]])

# # plt.colorbar()
# ax[0].set_xlabel('Component 1')
# ax[0].set_ylabel('Component 2')
# ax[1].set_xlabel('Component 1')
# ax[1].set_ylabel('Component 3')
# # ax[2].set_xlabel('Component 2')
# # ax[2].set_ylabel('Component 3')
# plt.savefig('orig_3projections')
# plt.show()

In [ ]:
plt.figure(figsize=(10,6))
l = plt.plot(general_wavelength, rpca.mean_*10 + 0.05,color='tab:blue')
c = l[0].get_color()
plt.text(9250, +0.15, "mean", color='k')
for i in range(5):
    l = plt.plot(general_wavelength, 1.5*rpca.components_[i] - 0.15 * i,color='tab:blue')
    plt.text(9250, -0.01 - 0.15 * i, "component %i" % (i + 1), color='k')
# plt.ylim(-0.2, 0.6)
# plt.axvline(6347,color='k')
# plt.axvline(6371,color='k')

# plt.xlim(6250,6750)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel('Scaled flux + offset')
plt.yticks([])
# plt.title('Mean Spectrum and Eigen-spectra')
# plt.savefig('5_eigenspectra.eps',bbox_inches='tight')
plt.show()

In [ ]:
expl_var = rpca.explained_variance_ratio_
np.cumsum(expl_var)

In [ ]:
dummy_x = np.linspace(1,51,50)
# plt.plot(expl_var,'-o', label="Explained Variance %")
plt.plot(dummy_x,np.cumsum(expl_var)*100,'-o', markersize=10, label = 'Cumulative variance %')
plt.xlabel("Component Number")
# plt.title('Explained Variance')
plt.ylabel('% Explained Variance')
# plt.savefig('explained_variance')
plt.show()

In [ ]:
# 3d plots all full spectra

# # Import dependencies
# import plotly
# import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=X_proj[:, 0],  # <-- Put your data instead
    y=X_proj[:, 1],  # <-- Put your data instead
    z=X_proj[:, 2],
    customdata=classing_set_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':'red',
        'colorscale':'Jet',
        'symbol':smallspectra
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]


plot_figure = go.Figure(data=data, layout=layout)

plot_figure.update_traces(
    hovertemplate="<br>".join([
        "Comp 1: %{x}",
        "Comp 2: %{y}",
        "Comp 3: %{z}",
        "Name: %{customdata}",
    ])
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
## lets try initializing my own centroids rather than letting it pick randomly
## choose them based off of some standards that I like.

## starting by trying just to pick the standards that I already have from my classes
# centroid_stars = ['A166','RZ Nor','V517 Oph','S Aps','B563'
#                   'J194218.38-203247.5','SV Sge','HD175893','RS Tel','UW Cen',
#                   'C528','RY Sgr','Y Mus','HD182040']
centroid_stars = ['RZ Nor','V517 Oph','S Aps','B563'
                  'J194218.38-203247.5','SV Sge','HD175893','RS Tel',
                  'C528','RY Sgr','Y Mus','HD182040']

centroid_init = []
for i in range(0,len(classing_set_names)):
    if classing_set_names[i] in centroid_stars:
        print(classing_set_names[i])
        centroid_init.append(X_proj[i,:])
        
centroid_init = np.asarray(centroid_init)
print(np.shape(centroid_init))

In [ ]:
# kmeans = KMeans(
#     init="random",
#     n_clusters=15,
#     n_init=20,
#     max_iter=300
# )

kmeans = KMeans(
    init=centroid_init,
    n_clusters=np.shape(centroid_init)[0],
    n_init=1,
    max_iter=300
)

kmeans.fit(X_proj)

In [ ]:
# plotly.offline.init_notebook_mode()

# # Configure the trace.
# trace = go.Scatter3d(
#     x=X_proj[:, 0],  # <-- Put your data instead
#     y=X_proj[:, 1],  # <-- Put your data instead
#     z=X_proj[:, 2],
#     customdata=classing_set_names,
#     mode='markers',
#     marker={
#         'size': 5,
#         'opacity': 0.8,
#         'color':kmeans.labels_,
#         'colorscale':'Jet',
#         'symbol':smallspectra
#     }
# )

# # Configure the layout.
# layout = go.Layout(
#     margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
# )

# data = [trace]


# plot_figure = go.Figure(data=data, layout=layout)

# plot_figure.update_traces(
#     hovertemplate="<br>".join([
#         "Comp 1: %{x}",
#         "Comp 2: %{y}",
#         "Comp 3: %{z}",
#         "Name: %{customdata}",
#     ])
# )

# # Render the plot.
# plotly.offline.iplot(plot_figure)

In [ ]:
#  # A list holds the silhouette coefficients for each k
# silhouette_coefficients = []

# # Notice you start at 2 clusters for silhouette coefficient
# for k in range(2, 20):
#     kmeans = KMeans(n_clusters=k,
#                     init="random",
#                     n_init=10,
#                     max_iter=300)
#     kmeans.fit(X_proj)
#     score = silhouette_score(X_proj, kmeans.labels_)
#     silhouette_coefficients.append(score)

In [ ]:
# plt.plot(range(2, 20), silhouette_coefficients)
# plt.xlabel("Number of Clusters")
# plt.ylabel("Silhouette Coefficient")
# plt.show()

In [ ]:
# from sklearn.cluster import DBSCAN

In [ ]:
# dbscan = DBSCAN(eps=0.3)

# dbscan.fit(X_proj)
# dbscan.labels_

In [ ]:
# plotly.offline.init_notebook_mode()

# # Configure the trace.
# trace = go.Scatter3d(
#     x=X_proj[:, 0],  # <-- Put your data instead
#     y=X_proj[:, 1],  # <-- Put your data instead
#     z=X_proj[:, 2],
#     customdata=store_names,
#     mode='markers',
#     marker={
#         'size': 5,
#         'opacity': 0.8,
#         'color':dbscan.labels_,
#         'colorscale':'Jet',
#         'symbol':smallspectra
#     }
# )

# # Configure the layout.
# layout = go.Layout(
#     margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
# )

# data = [trace]


# plot_figure = go.Figure(data=data, layout=layout)

# plot_figure.update_traces(
#     hovertemplate="<br>".join([
#         "Comp 1: %{x}",
#         "Comp 2: %{y}",
#         "Comp 3: %{z}",
#         "Name: %{customdata}",
#     ])
# )

# # Render the plot.
# plotly.offline.iplot(plot_figure)

In [ ]:
# order_labels = []
# for i in range(0,len(classing_set_names)):
#     if kmeans.labels_[i] == 6:
#         order_labels.append(0)
#     elif kmeans.labels_[i] == 4:
#         order_labels.append(1)
#     elif kmeans.labels_[i] == 10:
#         order_labels.append(2)
#     elif kmeans.labels_[i] == 3:
#         order_labels.append(3)
#     elif kmeans.labels_[i] == 8:
#         order_labels.append(4)
#     elif kmeans.labels_[i] == 11:
#         order_labels.append(5)
#     elif kmeans.labels_[i] == 9:
#         order_labels.append(6)
#     elif kmeans.labels_[i] == 0:
#         order_labels.append(7)
#     elif kmeans.labels_[i] == 5:
#         order_labels.append(8)
#     elif kmeans.labels_[i] == 2:
#         order_labels.append(9)
#     elif kmeans.labels_[i] == 1:
#         order_labels.append(10)
#     elif kmeans.labels_[i] == 7:
#         order_labels.append(11)

In [ ]:
plt_markers = []
for i in range(0,len(smallspectra)):
    if smallspectra[i] == 'circle':
        plt_markers.append('o')
    else:
        plt_markers.append('s')

In [ ]:
order_labels = []
for i in range(0,len(classing_set_names)):
    if kmeans.labels_[i] == 5:
        order_labels.append(0)
    elif kmeans.labels_[i] == 4:
        order_labels.append(1)
    elif kmeans.labels_[i] == 3:
        order_labels.append(2)
    elif kmeans.labels_[i] == 7:
        order_labels.append(3)
    elif kmeans.labels_[i] == 9:
        order_labels.append(4)
    elif kmeans.labels_[i] == 0:
        order_labels.append(5)
    elif kmeans.labels_[i] == 8:
        order_labels.append(6)
    elif kmeans.labels_[i] == 2:
        order_labels.append(7)
    elif kmeans.labels_[i] == 1:
        order_labels.append(8)
    elif kmeans.labels_[i] == 6:
        order_labels.append(9)

In [ ]:
f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
colors = plt.cm.jet(np.linspace(0,1,13))
ax[0].scatter(X_proj[:, 0], X_proj[:, 1], s=75, lw=0,c=order_labels,cmap=plt.cm.rainbow)
ax[1].scatter(X_proj[:, 0], X_proj[:, 2], s=75, lw=0,c=order_labels,cmap=plt.cm.rainbow)
# ax[2].scatter(X_proj[:, 1], X_proj[:, 2], s=75, lw=0,c=order_labels,cmap=plt.cm.rainbow)

# for i in range(0,len(classing_set_names)):
#     if smallspectra[i] == 'square':
#         ax[0].scatter(X_proj[i, 0], X_proj[i, 1], s=80, marker='s', lw=0,c='k')
#         ax[1].scatter(X_proj[i, 0], X_proj[i, 2], s=80, marker='s', lw=0,c='k')
#         ax[2].scatter(X_proj[i, 1], X_proj[i, 2], s=80, marker='s', lw=0,c='k')

# plt.colorbar()
ax[0].set_xlabel('Component 1')
ax[0].set_ylabel('Component 2')
ax[1].set_xlabel('Component 1')
ax[1].set_ylabel('Component 3')
# ax[2].set_xlabel('Component 2')
# ax[2].set_ylabel('Component 3')
# plt.savefig('kmeans_3projections.eps')
plt.show()

In [ ]:
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=X_proj[:, 0],  # <-- Put your data instead
    y=X_proj[:, 1],  # <-- Put your data instead
    z=X_proj[:, 2],
    customdata=classing_set_names,#classing_set_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':order_labels,
        'colorscale':'rainbow',
        'symbol':smallspectra
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]


plot_figure = go.Figure(data=data, layout=layout)

plot_figure.update_traces(
    hovertemplate="<br>".join([
        "Comp 1: %{x}",
        "Comp 2: %{y}",
        "Comp 3: %{z}",
        "Name: %{customdata}",
    ])
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
# good_spectra = []
# class_index = []
# good_names = []
# good_smallspectra = []

# for i in range(0,len(store_names)):
#     wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
#                                              store_Av_values[i],store_names[i])
# #     if np.min(wavelength) > 4500 or np.max(wavelength)<9000 or store_names[i] not in final_set_dict.keys():
#     if store_names[i] not in final_set_dict.keys():
#         pass
#     else:
#         if store_names[i] in order_pca.keys():
#             good_smallspectra.append('square')
#         else:
#             good_smallspectra.append('circle')

#         wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

#         interpolated = interpolate.interp1d(wavelength, flux, bounds_error = False, fill_value = 0)
#         good_spectra.append(interpolated(general_wavelength))
#         good_names.append(store_names[i])

# good_spectra = np.asarray(good_spectra)
# good_smallspectra = np.asarray(good_smallspectra)

In [ ]:
# good_X = preprocessing.normalize(good_spectra)
# good_X_proj = rpca.transform(good_X)

In [ ]:
# this set includes all stars that have been classified (the most recent complete classification system)
# keep in mind that stars with incomplete spectra DON'T PCA PROJECT WELL
good_v2_spectra = []
good_v2_names = []
good_v2_smallspectra = []

for i in range(0,len(store_names)):
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])
#     if np.min(wavelength) > 4500 or np.max(wavelength)<9000 or store_names[i] not in final_set_dict.keys():
    if store_names[i] not in final_set_v2_dict.keys():
        pass
    else:
        if store_names[i] in order_pca.keys():
            good_v2_smallspectra.append('square')
        else:
            good_v2_smallspectra.append('circle')

        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

        interpolated = interpolate.interp1d(wavelength, flux, bounds_error = False, fill_value = 0)
        good_v2_spectra.append(interpolated(general_wavelength))
        good_v2_names.append(store_names[i])

good_v2_spectra = np.asarray(good_v2_spectra)
good_v2_smallspectra = np.asarray(good_v2_smallspectra)

In [ ]:
good_v2_X = preprocessing.normalize(good_v2_spectra)
good_v2_X_proj = rpca.transform(good_v2_X)

In [ ]:
#NOT FOR PUBLISHING
f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
colors = plt.cm.jet(np.linspace(0,1,13))
ax[0].scatter(good_v2_X_proj[:, 0], good_v2_X_proj[:, 1], s=75, lw=0,c=final_set_v2_arr,cmap=plt.cm.rainbow)
ax[1].scatter(good_v2_X_proj[:, 0], good_v2_X_proj[:, 2], s=75, lw=0,c=final_set_v2_arr,cmap=plt.cm.rainbow)
# ax[2].scatter(good_v2_X_proj[:, 1], good_v2_X_proj[:, 2], s=75, lw=0,c=final_set_arr,cmap=plt.cm.rainbow)

for i in range(0,len(good_v2_names)):
    if good_v2_smallspectra[i] == 'square':
        ax[0].scatter(good_v2_X_proj[i, 0], good_v2_X_proj[i, 1], s=80, marker='s', lw=0,c='k')
        ax[1].scatter(good_v2_X_proj[i, 0], good_v2_X_proj[i, 2], s=80, marker='s', lw=0,c='k')
#         ax[2].scatter(good_v2_X_proj[i, 1], good_v2_X_proj[i, 2], s=80, marker='s', lw=0,c='k')

# plt.colorbar()
ax[0].set_xlabel('Component 1')
ax[0].set_ylabel('Component 2')
ax[1].set_xlabel('Component 1')
ax[1].set_ylabel('Component 3')
# ax[2].set_v2_Xlabel('Component 2')
# ax[2].set_ylabel('Component 3')
# plt.savefig('final_3projections')
plt.show()

In [ ]:
# j_min_w2_key = []
# # i_min_w2_key = []
# for i in range(0,len(good_names)):
#     if good_names[i] in hdc_j_min_w2.keys():
#         j_min_w2_key.append(hdc_j_min_w2[good_names[i]])
# #         i_min_w2_key.append(hdc_i_min_w2[good_names[i]])
# #         print(good_names[i],hdc_i_min_w2[good_names[i]])
#     else:
#         j_min_w2_key.append(-1.)
# #         i_min_w2_key.append(-1.)

In [ ]:
# print(max((j_min_w2_key)))

In [ ]:
# f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
# colors = plt.cm.rainbow(np.linspace(0,1,int(max(j_min_w2_key)*5)+1))

# for i in range(0,len(good_names)):
#     if j_min_w2_key[i] != -1 and good_names[i] not in ['AO Her']:
#         ax[0].scatter(good_X_proj[i, 0], good_X_proj[i, 1], s=75, lw=0,c=colors[int(j_min_w2_key[i])*5])
#         ax[1].scatter(good_X_proj[i, 0], good_X_proj[i, 2], s=75, lw=0,c=colors[int(j_min_w2_key[i]*5)])
#     else:
#         ax[0].scatter(good_X_proj[i, 0], good_X_proj[i, 1], s=75, lw=0,c='k')
#         ax[1].scatter(good_X_proj[i, 0], good_X_proj[i, 2], s=75, lw=0,c='k')


# ax[0].set_xlabel('Component 1')
# ax[0].set_ylabel('Component 2')
# ax[1].set_xlabel('Component 1')
# ax[1].set_ylabel('Component 3')
# plt.savefig('j_min_w2_pca')
# plt.show()

In [ ]:
# f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
# colors = plt.cm.rainbow(np.linspace(0,1,10))

# for i in range(0,len(good_names)):
#     if i_min_w2_key[i] != -1 and good_names[i] not in ['B566']:
#         ax[0].scatter(good_X_proj[i, 0], good_X_proj[i, 1], s=75, lw=0,c=colors[int(i_min_w2_key[i]*2)])
#         ax[1].scatter(good_X_proj[i, 0], good_X_proj[i, 2], s=75, lw=0,c=colors[int(i_min_w2_key[i]*2)])
#     else:
#         ax[0].scatter(good_X_proj[i, 0], good_X_proj[i, 1], s=75, lw=0,c='k')
#         ax[1].scatter(good_X_proj[i, 0], good_X_proj[i, 2], s=75, lw=0,c='k')


# ax[0].set_xlabel('Component 1')
# ax[0].set_ylabel('Component 2')
# ax[1].set_xlabel('Component 1')
# ax[1].set_ylabel('Component 3')
# plt.savefig('i_min_w2_pca')
# plt.show()

In [ ]:
# all the classified stars with the standard stars labeled
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=good_v2_X_proj[:, 0],  # <-- Put your data instead
    y=good_v2_X_proj[:, 1],  # <-- Put your data instead
    z=good_v2_X_proj[:, 2],
    customdata=good_v2_names,#carbon_index_arr,#classing_set_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':final_set_v2_arr,
        'colorscale':'rainbow',
        'symbol':good_v2_smallspectra
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]


plot_figure = go.Figure(data=data, layout=layout)

plot_figure.update_traces(
    hovertemplate="<br>".join([
        "Comp 1: %{x}",
        "Comp 2: %{y}",
        "Comp 3: %{z}",
        "Name: %{customdata}",
    ])
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
# set of all stars
general_wavelength = np.linspace(4500,9000,10000)
all_resampled_spectra = []
# classing_set_names = []

for i in range(0,len(store_names)):
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])

    wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

    interpolated = interpolate.interp1d(wavelength, flux, bounds_error = False, fill_value = 0)
    all_resampled_spectra.append(interpolated(general_wavelength))
#         classing_set_names.append(store_names[i])

all_resampled_spectra = np.asarray(all_resampled_spectra)

norm_resampled_spectra = preprocessing.normalize(all_resampled_spectra)

In [ ]:
# not_classed_spectra = []
# not_classed_names = []
# for i in range(0,len(store_names)):
#     if store_names[i] not in final_set_dict.keys():
# #         if store_names[i] == 'UW Cen':
# #             print('caught')
#         not_classed_spectra.append(norm_resampled_spectra[i])
#         not_classed_names.append(store_names[i])
# #         deconstructed = rpca.transform([norm_resampled_spectra[i]])
# #         print(deconstructed)
# deconstructed = rpca.transform(not_classed_spectra)

In [ ]:
not_classed_spectra = []
not_classed_names = []
for i in range(0,len(store_names)):
    if store_names[i] not in final_set_v2_dict.keys():
#         if store_names[i] == 'UW Cen':
#             print('caught')
        not_classed_spectra.append(norm_resampled_spectra[i])
        not_classed_names.append(store_names[i])
#         deconstructed = rpca.transform([norm_resampled_spectra[i]])
#         print(deconstructed)
deconstructed = rpca.transform(not_classed_spectra)

In [ ]:
#I'm not sure what I meant by resampled here
# same set as above but only the complete spectra (those where PCA means something)
general_wavelength = np.linspace(4500,9000,10000)
full_resampled_spectra = []
full_set_names = []

for i in range(0,len(store_names)):
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])
    if np.min(wavelength) > 4500 or np.max(wavelength) < 8900 or store_names[i] not in final_set_v2_dict.keys() or store_names[i] in ['FH Sct','J005010.67-694357.7']:
        pass
    else:
        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

        interpolated = interpolate.interp1d(wavelength, flux, bounds_error = False, fill_value = 0)
        full_resampled_spectra.append(interpolated(general_wavelength))
        full_set_names.append(store_names[i])

full_resampled_spectra = np.asarray(full_resampled_spectra)
full_resampled_spectra = preprocessing.normalize(full_resampled_spectra)

full_pca_set = rpca.transform(full_resampled_spectra)

In [ ]:
full_resampled_arr = []
for i in range(0,len(store_names)):
    if store_names[i] in full_set_names:
        if store_names[i] in final_sets_v2[0]:
            full_resampled_arr.append(9)
        elif store_names[i] in final_sets_v2[1]:
            full_resampled_arr.append(8)
        elif store_names[i] in final_sets_v2[2]:
            full_resampled_arr.append(7)
        elif store_names[i] in final_sets_v2[3]:
            full_resampled_arr.append(6)
        elif store_names[i] in final_sets_v2[4]:
            full_resampled_arr.append(5)
        elif store_names[i] in final_sets_v2[5]:
            full_resampled_arr.append(4)
        elif store_names[i] in final_sets_v2[6]:
            full_resampled_arr.append(3)
        elif store_names[i] in final_sets_v2[7]:
            full_resampled_arr.append(2)
        elif store_names[i] in final_sets_v2[8]:
            full_resampled_arr.append(1)
        elif store_names[i] in final_sets_v2[9]:
            full_resampled_arr.append(0)
print(len(full_resampled_arr))

In [ ]:
# plot for the paper!!
f,ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
colors = plt.cm.jet(np.linspace(0,1,13))
ax[0].scatter(full_pca_set[:, 0], full_pca_set[:, 1], s=75, lw=0,c=full_resampled_arr,cmap=plt.cm.rainbow)
ax[1].scatter(full_pca_set[:, 0], full_pca_set[:, 2], s=75, lw=0,c=full_resampled_arr,cmap=plt.cm.rainbow)
# ax[2].scatter(full_pca_set[:, 1], full_pca_set[:, 2], s=75, lw=0,c=final_set_arr,cmap=plt.cm.rainbow)

for i in range(0,len(full_set_names)):
    if full_set_names[i] in order_pca.keys():
        ax[0].scatter(full_pca_set[i, 0], full_pca_set[i, 1], s=80, marker='s', lw=0,c='k')
        ax[1].scatter(full_pca_set[i, 0], full_pca_set[i, 2], s=80, marker='s', lw=0,c='k')
#         ax[2].scatter(full_pca_set[i, 1], full_pca_set[i, 2], s=80, marker='s', lw=0,c='k')

# plt.colorbar()
ax[0].set_xlabel('Component 1')
ax[0].set_ylabel('Component 2')
ax[1].set_xlabel('Component 1')
ax[1].set_ylabel('Component 3')
# ax[2].set_xlabel('Component 2')
# ax[2].set_ylabel('Component 3')
# plt.savefig('full_final_3projections.eps')
plt.show()

In [ ]:
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=full_pca_set[:, 0],  # <-- Put your data instead
    y=full_pca_set[:, 1],  # <-- Put your data instead
    z=full_pca_set[:, 2],
    customdata=full_set_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':full_resampled_arr,
        'colorscale':'rainbow',
        'symbol':'circle'
    }
)

# trace2 = go.Scatter3d(
#     x=deconstructed[:, 0],  # <-- Put your data instead
#     y=deconstructed[:, 1],  # <-- Put your data instead
#     z=deconstructed[:, 2],
#     customdata=not_classed_names,
#     mode='markers',
#     marker={
#         'size': 5,
#         'opacity': 0.8,
#         'color':'black',
#         'colorscale':'Jet',
#         'symbol':'square'
#     }
# )


# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]



plot_figure = go.Figure(data=data, layout=layout)

plot_figure.update_traces(
    hovertemplate="<br>".join([
        "Comp 1: %{x}",
        "Comp 2: %{y}",
        "Comp 3: %{z}",
        "Name: %{customdata}",
    ])
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
# this one includes the stars that aren't classified and the DY Pers
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=good_X_proj[:, 0],  # <-- Put your data instead
    y=good_X_proj[:, 1],  # <-- Put your data instead
    z=good_X_proj[:, 2],
    customdata=good_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':final_set_arr,
        'colorscale':'rainbow',
        'symbol':good_smallspectra
    }
)

trace2 = go.Scatter3d(
    x=deconstructed[:, 0],  # <-- Put your data instead
    y=deconstructed[:, 1],  # <-- Put your data instead
    z=deconstructed[:, 2],
    customdata=not_classed_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':'black',
        'colorscale':'Jet',
        'symbol':'square'
    }
)


# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace,trace2]



plot_figure = go.Figure(data=data, layout=layout)

plot_figure.update_traces(
    hovertemplate="<br>".join([
        "Comp 1: %{x}",
        "Comp 2: %{y}",
        "Comp 3: %{z}",
        "Name: %{customdata}",
    ])
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
# this one includes the stars that aren't classified and the DY Pers are a diff color
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=good_v2_X_proj[:, 0],  # <-- Put your data instead
    y=good_v2_X_proj[:, 1],  # <-- Put your data instead
    z=good_v2_X_proj[:, 2],
    customdata=good_v2_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':final_set_v2_arr,
        'colorscale':'rainbow',
        'symbol':good_v2_smallspectra
    }
)

trace2 = go.Scatter3d(
    x=deconstructed[:-27, 0],  # <-- Put your data instead
    y=deconstructed[:-27, 1],  # <-- Put your data instead
    z=deconstructed[:-27, 2],
    customdata=not_classed_names[:-27],
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':'black',
        'colorscale':'Jet',
        'symbol':'square'
    }
)

trace3 = go.Scatter3d(
    x=deconstructed[-27:-1, 0],  # <-- Put your data instead
    y=deconstructed[-27:-1, 1],  # <-- Put your data instead
    z=deconstructed[-27:-1, 2],
    customdata=not_classed_names[-27:-1],
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':'grey',
        'colorscale':'Jet',
        'symbol':'square'
    }
)


# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace,trace2,trace3]



plot_figure = go.Figure(data=data, layout=layout)

plot_figure.update_traces(
    hovertemplate="<br>".join([
        "Comp 1: %{x}",
        "Comp 2: %{y}",
        "Comp 3: %{z}",
        "Name: %{customdata}",
    ])
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
# this is all the stars but only the red side of the spectrum
# for exploratory purposes
# I would like to mention that I hate how good this actually looks
general_wavelength_v3 = np.linspace(6000,9000,10000)
good_v3_spectra = []
good_v3_names = []
good_v3_smallspectra = []

for i in range(0,len(store_names)):
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])
    if np.min(wavelength) > 6100 or np.max(wavelength)<8500 or store_names[i] not in final_set_v2_dict.keys():
#     if store_names[i] not in final_set_v2_dict.keys():
        pass
#         print(store_names[i])
    else:
        good_v3_smallspectra.append('circle')

        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

        interpolated = interpolate.interp1d(wavelength, flux, bounds_error = False, fill_value = 0)
        good_v3_spectra.append(interpolated(general_wavelength))
        good_v3_names.append(store_names[i])

good_v3_spectra = np.asarray(good_v3_spectra)
good_v3_smallspectra = np.asarray(good_v3_smallspectra)

In [ ]:
red_X = preprocessing.normalize(good_v3_spectra)
# red_X_proj = rpca.transform(red_X)

In [ ]:
rpca_red = PCA(n_components=50, svd_solver='randomized')
# pk.dump(rpca, open("store_pca.pkl","wb"))
red_X_proj = rpca_red.fit_transform(red_X)
red_X_proj.shape

In [ ]:
#to look at the plots and see if they're in carbon index order
carbon_order = []
for i in range(0,len(store_names)):
    if store_names[i] in good_v3_names:
        carbon_order.append(plot_order_v2_sets[store_names[i]])

In [ ]:
classes_arr = []
for i in range(0,len(store_names)):
    if store_names[i] in good_v3_names:
        if store_names[i] in final_sets_v2[0]:
            classes_arr.append(0)
        elif store_names[i] in final_sets_v2[1]:
            classes_arr.append(1)
        elif store_names[i] in final_sets_v2[2]:
            classes_arr.append(2)
        elif store_names[i] in final_sets_v2[3]:
            classes_arr.append(3)
        elif store_names[i] in final_sets_v2[4]:
            classes_arr.append(4)
        elif store_names[i] in final_sets_v2[5]:
            classes_arr.append(5)
        elif store_names[i] in final_sets_v2[6]:
            classes_arr.append(6)
        elif store_names[i] in final_sets_v2[7]:
            classes_arr.append(7)
        elif store_names[i] in final_sets_v2[8]:
            classes_arr.append(8)
        elif store_names[i] in final_sets_v2[9]:
            classes_arr.append(9)


In [ ]:
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=red_X_proj[:, 0],  # <-- Put your data instead
    y=red_X_proj[:, 1],  # <-- Put your data instead
    z=red_X_proj[:, 2],
    customdata=carbon_order,#good_v3_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':classes_arr,
        'colorscale':'rainbow',
        'symbol':good_v3_smallspectra
    }
)


# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]



plot_figure = go.Figure(data=data, layout=layout)

plot_figure.update_traces(
    hovertemplate="<br>".join([
        "Comp 1: %{x}",
        "Comp 2: %{y}",
        "Comp 3: %{z}",
        "Name: %{customdata}",
    ])
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
# this is all the stars but only the h alpha portion of the spectrum
# for exploratory purposes
general_wavelength_v4 = np.linspace(5800,7000,10000)
good_v4_spectra = []
good_v4_names = []
good_v4_smallspectra = []

for i in range(0,len(store_names)):
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])
    if np.min(wavelength) > 5800 or np.max(wavelength)<7000 or store_names[i] not in final_set_v2_dict.keys():
#     if store_names[i] not in final_set_v2_dict.keys():
        pass
#         print(store_names[i])
    else:
        good_v4_smallspectra.append('circle')

        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

        interpolated = interpolate.interp1d(wavelength, flux, bounds_error = False, fill_value = 0)
        good_v4_spectra.append(interpolated(general_wavelength))
        good_v4_names.append(store_names[i])

good_v4_spectra = np.asarray(good_v4_spectra)
good_v4_smallspectra = np.asarray(good_v4_smallspectra)

In [ ]:
small_X = preprocessing.normalize(good_v4_spectra)

In [ ]:
rpca_small = PCA(n_components=50, svd_solver='randomized')
# pk.dump(rpca, open("store_pca.pkl","wb"))
small_X_proj = rpca_small.fit_transform(small_X)
small_X_proj.shape

In [ ]:
carbon_order_small = []
for i in range(0,len(store_names)):
    if store_names[i] in good_v4_names:
        carbon_order_small.append(plot_order_v2_sets[store_names[i]])

In [ ]:
small_classes_arr = []
for i in range(0,len(store_names)):
    if store_names[i] in good_v4_names:
        if store_names[i] in final_sets_v2[0]:
            small_classes_arr.append(0)
        elif store_names[i] in final_sets_v2[1]:
            small_classes_arr.append(1)
        elif store_names[i] in final_sets_v2[2]:
            small_classes_arr.append(2)
        elif store_names[i] in final_sets_v2[3]:
            small_classes_arr.append(3)
        elif store_names[i] in final_sets_v2[4]:
            small_classes_arr.append(4)
        elif store_names[i] in final_sets_v2[5]:
            small_classes_arr.append(5)
        elif store_names[i] in final_sets_v2[6]:
            small_classes_arr.append(6)
        elif store_names[i] in final_sets_v2[7]:
            small_classes_arr.append(7)

In [ ]:
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=small_X_proj[:, 0],  # <-- Put your data instead
    y=small_X_proj[:, 1],  # <-- Put your data instead
    z=small_X_proj[:, 2],
    customdata=carbon_order_small,#carbon_order_small,#good_v4_names,
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color':small_classes_arr,
        'colorscale':'rainbow',
        'symbol':good_v4_smallspectra
    }
)


# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]



plot_figure = go.Figure(data=data, layout=layout)

plot_figure.update_traces(
    hovertemplate="<br>".join([
        "Comp 1: %{x}",
        "Comp 2: %{y}",
        "Comp 3: %{z}",
        "Name: %{customdata}",
    ])
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
# ii = 0
# for i in range(0,len(store_names)):
#     if store_names[i] in not_classed_names:
#         print(store_names[i])
#         wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
#                                              store_Av_values[i],store_names[i])
        
#         reconstructed = rpca.inverse_transform(deconstructed[ii,:])
#         plt.plot(general_wavelength,reconstructed/np.max(reconstructed))
#         plt.plot(wavelength,flux/np.max(flux))
#         plt.title(store_names[i])
#         plt.show()
#         ii = ii+1



In [ ]:
# ii=0
# for i in range(0,len(store_names)):
#     if store_names[i] in not_classed_names:
#         if store_names[i] in ['ASAS-RCB-6','ASAS-RCB-3','A166']:
#     #         print(store_names[i])
#             wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
#                                                  store_Av_values[i],store_names[i])
#             greater = np.where(wavelength>6500)                
#             plt.plot(wavelength,flux/(np.average(flux[greater][:50])))
            
#             reconstructed = rpca.inverse_transform(deconstructed[ii,:])
#             greater_recon = np.where(general_wavelength>6500)
#             plt.plot(general_wavelength,reconstructed/(np.average(reconstructed[greater_recon][:50])))

#             plt.title(store_names[i])
#             plt.show()
#         ii = ii+1


In [ ]:
# order_pca = {'V3795 Sgr':0,'VZ Sgr':1,'HD173409':2,'A223':3,'B42':4,
#              'WX CrA':5,'J174328.50-375029.0':6,'ASAS-RCB-4':7}#MACHO 308.38099.66 'EROS2-LMC-DYPer-3'

plt.figure(figsize=(15,6))
for i in range(0,len(store_names)):
    if store_names[i] in ['MACHO 308.38099.66','ASAS-RCB-4','J182723.38-200830.1']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        if store_names[i] == 'MACHO 308.38099.66':
            wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 6,store_names[i])

        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)
#         if store_names[i] == 'J005113.58-731036.3':
#             wavelength = wavelength + ((350/3e5)*wavelength)
#         if store_names[i] == 'MACHO 308.38099.66':
#             flux = convolve(flux,Gaussian1DKernel(1))
        greater = np.where(wavelength>6860)                
#         plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i])
#         if store_names[i] == 'EROS2-LMC-DYPer-6':
#             plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i],color='tab:red')
#         else:
        plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i])
#         plt.plot(wavelength,flux/np.max(flux),label=store_names[i])
# plt.axvline(wavelength[greater][0])
# plt.axvline(wavelength[greater][25],color='k')
# plt.axvline(6910,color='k')
# plt.axvline(5780,color='k')

# plt.ylim(top=1.75)
# plt.ylim(bottom=-0.5)
plt.xlim(5800,7200)
# plt.xlim(7000,8000)
# plt.xlim(8000,9000)
# plt.xlim(3800,5000)
# plt.xlim(4500,5800)
# plt.legend(bbox_to_anchor=(0,1), loc="lower left")       
plt.yticks([])
plt.xticks(fontsize=15)
# plt.ylim(top=1.55)
# plt.xlim(right=7500)
plt.legend(fontsize=15)
plt.ylabel('Norm. Flux',fontsize=25)
plt.xlabel(r'Wavelength ($\AA$)',fontsize=25)
# plt.savefig('308compare_3')
plt.show()

In [ ]:
# order_pca = {'V3795 Sgr':0,'VZ Sgr':1,'HD173409':2,'A223':3,'B42':4,
#              'ASAS-RCB-2':5,'V517 Oph':6,'ASAS-RCB-18':7} #EROS2-LMC-DYPer-3

plt.figure(figsize=(15,6))
for i in range(0,len(store_names)):
    if store_names[i] in ['A223','C539']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
#         if store_names[i] == 'MACHO 308.38099.66':
#             wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
#                                                  6,store_names[i])

        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)
#         if store_names[i] == 'J005113.58-731036.3':
#             wavelength = wavelength + ((350/3e5)*wavelength)
#         if store_names[i] == 'MACHO 308.38099.66':
#             flux = convolve(flux,Gaussian1DKernel(1))
        greater = np.where(wavelength>6860)                
#         plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i])
#         if store_names[i] == 'EROS2-LMC-DYPer-6':
#             plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i],color='tab:red')
#         else:
        plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i])
#         plt.plot(wavelength,flux/np.max(flux),label=store_names[i])

plt.axvline(6496,color='k') #barium
plt.axvline(6563,color='r') #hydrogen
plt.axvline(6707,color='g') #lithium
plt.axvline(4077,color='k') #strontium
# plt.axvline(4860,color='r') #hbeta
plt.axvline(6140,color='k') #searching
plt.axvline(4554,color='k')

# plt.ylim(top=1.75)
# plt.ylim(bottom=-0.5)
plt.xlim(3800,5000)
plt.xlim(4500,5800)
plt.xlim(4500,5000)
# plt.xlim(5800,7200)
# plt.xlim(7000,8000)
# plt.xlim(8000,9000) 
# plt.legend(bbox_to_anchor=(0,1), loc="lower left")       
plt.yticks([])
plt.xticks(fontsize=15)
# plt.ylim(top=1.55)
# plt.xlim(right=7500)
plt.legend(fontsize=15)
plt.ylabel('Norm. Flux',fontsize=25)
plt.xlabel(r'Wavelength ($\AA$)',fontsize=25)
# plt.savefig('308compare_3')
plt.show()

In [ ]:
# order_pca = {'V3795 Sgr':0,'VZ Sgr':1,'HD173409':2,'A223':3,'B42':4,
#              'WX CrA':5,'J174328.50-375029.0':6,'ASAS-RCB-4':7}#MACHO 308.38099.66 'EROS2-LMC-DYPer-3'

plt.figure(figsize=(15,4))
for i in range(0,len(store_names)):
    if store_names[i] in ['ASAS-RCB-18','ASAS-DYPer-1','EROS2-LMC-DYPer-6']:#'EROS2-SMC-RCB-3','EROS2-LMC-DYPer-6']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
#         if store_names[i] == 'ASAS-DYPer-2':
#             wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
#                                                  0.0,store_names[i])

        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)
#         if store_names[i] == 'J005113.58-731036.3':
#             wavelength = wavelength + ((350/3e5)*wavelength)
#         if store_names[i] == 'MACHO 308.38099.66':
#             flux = convolve(flux,Gaussian1DKernel(1))
        greater = np.where(wavelength>6860)                
#         plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i])
        if store_names[i] == 'EROS2-LMC-DYPer-6':
            plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i],color='tab:red')
        else:
            plt.plot(wavelength,flux/(np.average(flux[greater][:25])),label=store_names[i]+r' (HdC8: C$_2$5 CN+)')
#         plt.plot(wavelength,flux/np.max(flux),label=store_names[i])
# plt.axvline(wavelength[greater][0])
# plt.axvline(wavelength[greater][25],color='k')
# plt.axvline(6910,color='k')
# plt.axvline(5780,color='k')

# plt.ylim(top=1.75)
# plt.ylim(bottom=-0.5)
plt.xlim(5800,7200)
# plt.xlim(7000,8000)
# plt.xlim(8000,9000)
# plt.xlim(3800,5000)
# plt.xlim(4500,5800)
# plt.legend(bbox_to_anchor=(0,1), loc="lower left")       
plt.yticks([])
plt.xticks(fontsize=15)
plt.ylim(top=1.55)
# plt.xlim(right=7500)
plt.legend(fontsize=15)
plt.ylabel('Norm. Flux',fontsize=25)
plt.xlabel(r'Wavelength ($\AA$)',fontsize=25)
# plt.savefig('potential_new_dyper',bbox_inches='tight')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] == 'ASAS-DYPer-1':
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)
        plt.plot(wavelength,flux)
        plt.show()

In [ ]:
print(len(dyper_class))

In [ ]:
dyper_class = ['ASAS-DYPer-1','ASAS-DYPer-2','EROS2-LMC-DYPer-1','EROS2-LMC-DYPer-2',
              'EROS2-LMC-DYPer-3','EROS2-LMC-DYPer-4','EROS2-LMC-DYPer-5','EROS2-LMC-DYPer-6',
              'EROS2-LMC-DYPer-7','EROS2-LMC-DYPer-8','EROS2-LMC-DYPer-9','EROS2-LMC-DYPer-10',
              'EROS2-LMC-DYPer-11','EROS2-LMC-DYPer-12','EROS2-LMC-DYPer-13','EROS2-LMC-DYPer-14',
              'EROS2-LMC-DYPer-15','EROS2-LMC-DYPer-16','EROS2-LMC-DYPer-17','EROS2-CG-RCB-2',
              'EROS2-SMC-DYPer-1','EROS2-SMC-DYPer-2','EROS2-SMC-DYPer-3','EROS2-SMC-DYPer-4',
              'EROS2-SMC-DYPer-5','EROS2-SMC-DYPer-6']

plt.figure(figsize=(12,len(dyper_class)))
x=0
for i in range(0,len(barnbaum_stars_lam)):
    if barnbaum_spec['Name'][i] == 'DY Per':
        plt.plot(barnbaum_stars_lam[i],barnbaum_stars_flux[i]/np.max(barnbaum_stars_flux[i]-x))#,label=r'DY Per (C-Hd4.5 C$_2$6)',color='tab:red')
        plt.annotate('DY Per',xy=(9910,-x+0.2),annotation_clip=False,size=17.5)
        x=x+1
for i in range(0,len(store_names)):
    if store_names[i] in dyper_class:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

#         greater = np.where(wavelength>6900)                
#         plt.plot(wavelength,flux/(np.average(flux[greater][:50]))-x,color='tab:blue')
        plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
        plt.annotate(store_names[i],xy=(9910,-x+0.2),annotation_clip=False,size=17.5)
        x=x+1
        
# plt.axvline(5730,color='k')
# plt.axvline(5250,color='k')

plt.yticks([])
plt.title('DY Pers')
plt.xlim(right=9800)
plt.ylabel('Norm. Flux + Offset')
plt.xlabel(r'Wavelength ($\AA$)')
# plt.xlim(left=5000)
# plt.savefig('dyper_class.eps',bbox_inches='tight')
plt.show()

In [ ]:
# plt.axvline(6855,color='k')
# plt.axvline(6910,color='k')

for i in range(0,len(store_names)):
    if store_names[i] in dyper_class:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])
        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

        region_max = np.where(np.abs(wavelength-6885) < 20)
        avg_max = np.average(flux[region_max],weights=flux[region_max])
        region_c2 = np.where(np.abs(wavelength-6830) < 25)
        avg_c2 = np.average(flux[region_c2],weights=flux[region_c2])
        region_cn = np.where(np.abs(wavelength-6965) < 25)
        avg_cn = np.average(flux[region_cn],weights=flux[region_cn])

        plt.plot(wavelength,flux)
        plt.plot(wavelength[region_max],flux[region_max])
        plt.plot(wavelength[region_c2],flux[region_c2])
        plt.plot(wavelength[region_cn],flux[region_cn])
        plt.axhline(avg_max,color='k')
        plt.axhline(avg_c2,color='k')
        plt.axhline(avg_cn,color='k')
        plt.title(store_names[i])
        plt.xlim(6600,7200)
        plt.show()

In [ ]:
print(len(plot_order_v2_sets))

# Plotting Classes

In [ ]:
final_sets_v2 = {
# 9: ['J182334.24-282957.1',
# #     'EROS2-CG-RCB-7',
#     'J163450.35-380218.5',
#     'J171908.50-435044.6',
# #     'EROS2-LMC-RCB-6',
#    'MACHO 308.38099.66'],
# 8: [#'V517 Oph',
# #    'ASAS-RCB-20',
# #    'EROS2-CG-RCB-11',
# #    'ASAS-RCB-14',
# #    'J173202.75-432906.1',
#    'J005113.58-731036.3'
# #     'EROS2-CG-RCB-9',
# #     'ES Aql',
# #     'EROS2-SMC-RCB-3',
# #     'J173737.07-072828.1',
# #    'J054221.91-690259.3'
#     ],
7: ['AO Her',
  'ASAS-RCB-18',
  #'ASAS-RCB-19',
#   'ASAS-RCB-4',
#   'ASAS-RCB-7',
#   'J132354.47-673720.8',
  'J182723.38-200830.1',
  'J183649.54-113420.7',
    'MACHO 401.48170.2237',
    'EROS2-CG-RCB-5',
    'J150104.50-563325.1',
    'J182235.25-033213.2',
#     'J175521.75-281131.2',
#     'J173202.75-432906.1',
  'J184246.26-125414.7'],
6: ['ASAS-RCB-15',
#   'ASAS-RCB-5',
  'IRAS1813.5-2419',
  'J173819.81-203632.1',
  'J174138.87-161546.4',
  'J174328.50-375029.0',
  'J194218.38-203247.5',
  'SV Sge',
#   'V1157 Sgr',
    'MACHO-81.8394.1358',
    'MACHO-80.6956.207',
#     'EROS2-CG-RCB-13',
    'WISE-ToI-5031',
    'EROS2-CG-RCB-3',
#     'EROS2-CG-RCB-4',
    'J173553.02-364104.3',
#     'EROS2-CG-RCB-14',
#     'EROS2-CG-RCB-6',
#     'J174257.19-362052.1',
    'J175749.98-182522.8',
    'V517 Oph',
    'ASAS-RCB-14',
    'ASAS-RCB-21',
    'ASAS-RCB-4',
    'ES Aql',
    'J132354.47-673720.8'
#   'EROS2-LMC-RCB-4',
#   'EROS2-SMC-RCB-1'
   ],
5: ['ASAS-RCB-1',
  'ASAS-RCB-16',
  'ASAS-RCB-17',
  'ASAS-RCB-2',
#   'ASAS-RCB-9',
  'J160205.48-552741.6',
  'J161156.23-575527.1',
#   'J182943.83-190246.2',
#   'OGLE-GC-RCB-1',
  'S Aps',
  'U Aqr',
  'V1783 Sgr',
#   'V2331 Sgr',
#   'V739 Sgr',
#   'WX CrA',
    'NSV11154',
    'Z Umi',
    'J181538.25-203845.7',
    'J110008.77-600303.6',
    'J170343.87-385126.6',
#     'ES Aql',
    'EROS2-CG-RCB-6',
    'ASAS-RCB-7',
    'ASAS-RCB-3',
#   'EROS2-LMC-RCB-1',
  'EROS2-LMC-RCB-5',
  'EROS2-SMC-RCB-2'],
4: ['ASAS-RCB-11',
  'HD137613',
  'HD175893',
  'EROS2-LMC-RCB-2',
  'EROS2-LMC-RCB-3',
  'J054123.49-705801.8',
    'ASAS J050232-7218.9',
    'J175749.76-075314.9',
    'HE 1015-2050',
    'J181252.50-233304.4',
  'B42',
  'A770',
  'A977',
  'J184158.40-054819.2',
  'V4017 Sgr',
#     'ASAS-RCB-12',
    'M38',
#     'ASAS-RCB-3',
#     'J171815.36-341339.9',
#     'J181836.38-181732.8',
    'dLS114',
    'EROS2-CG-RCB-1',
#     'C542',
#     'M37',
#     'P12',
#     'J180550.49-151301.7',
  'A814'],
3: ['C27',
    'GU Sgr',
  'J174119.57-250621.2',
  'RS Tel',
  'J055643.56-715510.7',
  'A223',
  'A183',
  'C38',
  'C20',
  'A811',
  'C539',
    'dLS17',
    'C528',
    'C542',
    'M37',
    'P12',
#     'J172553.80-312421.1',
    'J174645.90-250314.1',
  'B563'],
2: ['HD148839',
  'HD173409',
    'HD182040',
  'A182',
  'A226',
    'B565',
    'RZ Nor',
    'RT Nor',
    'B566',
    'J174851.29-330617.0',
  'C17',
#     'FH Sct',
    'J164704.67-470817.8',
    'J175031.70-233945.7',
    'J175107.12-242357.3',
#     'J180550.49-151301.7',
    'UV Cas',
#     'V CrA',
    'J005010.67-694357.7',
  'C105'],
1: ['J172447.52-290418.6',
#   'J175558.51-164744.3',
  'VZ Sgr',
    'ASAS-RCB-8',
    'RY Sgr',
  'F75',
  'C526',
  'J185525.52-025145.7',
  'V2552 Oph',
  'V391 Sct',
  'B567',
    'A249',
    'UX Ant',
    'R CrB',
    'F152',
    'J174111.80-281955.3',
#     'UW Cen',
    'V482 Cyg',
    'ASAS-RCB-10',
    'W Men',
  'V854 Cen'],
0: ['J172951.80-101715.9', 
    'V3795 Sgr', 
#     'W Men',
#     'A798',
    'Y Mus', 
    'B564', 
    'A980']}

In [ ]:
# carbon_index_v2_sets = {
# 'J182334.24-282957.1':6.5,
#     'EROS2-CG-RCB-7':'-',
#     'J163450.35-380218.5':'-',
#     'J171908.50-435044.6':'-',
#     'EROS2-LMC-RCB-6':'-',
#    'MACHO 308.38099.66':6.5, ##############
# 'V517 Oph':6,
#    'ASAS-RCB-20':5,
#    'EROS2-CG-RCB-11':'-',
#    'ASAS-RCB-14':5,
#    'J173202.75-432906.1':5,
#    'J005113.58-731036.3':6,
#     'EROS2-CG-RCB-9':'-',
#     'ES Aql':6,
#     'EROS2-SMC-RCB-3':5,
#     'J173737.07-072828.1':'-',
#    'J054221.91-690259.3':6, ##############
#   'ASAS-RCB-18':6,
#   'ASAS-RCB-19':6,
#   'ASAS-RCB-4':6,
#   'ASAS-RCB-7':6,
#   'J132354.47-673720.8':6,
#   'J182723.38-200830.1':5.5,
#   'J183649.54-113420.7':5.5,
#     'EROS2-SMC-RCB-3':6,
#     'AO Her':6,
#     'MACHO 401.48170.2237':6,
#     'EROS2-CG-RCB-5':6,
#     'J150104.50-563325.1':'-',
#     'J175521.75-281131.2':6,
#   'J184246.26-125414.7':5.5, ###########
#   'ASAS-RCB-15':5.5,
#   'ASAS-RCB-5':5,
#   'IRAS1813.5-2419':5,
#   'J173819.81-203632.1':5.5,
#   'J174138.87-161546.4':5.5,
#   'J174328.50-375029.0':5.5,
#   'J194218.38-203247.5':5.5,
#   'SV Sge':5.5,
#   'V1157 Sgr':5.5,
#   'EROS2-LMC-RCB-4':5.5,
#     'MACHO-80.6956.207':5.5,
#     'MACHO-81.8394.1358':5.5,
#     'EROS2-CG-RCB-13':5.5,
#     'EROS2-CG-RCB-3':5.5,
#     'WISE-ToI-5031':5.5,
#     'EROS2-CG-RCB-4':5,
#     'J173553.02-364104.3':5.5,
#     'EROS2-CG-RCB-14':'-',
#     'EROS2-CG-RCB-6':'-',
#     'J174257.19-362052.1':'-',
#     'J175749.98-182522.8':'-',
#   'EROS2-SMC-RCB-1':5.5, #############
#   'ASAS-RCB-1':5,
#   'ASAS-RCB-16':5.5,
#   'ASAS-RCB-17':5,
#   'ASAS-RCB-2':5,
#   'ASAS-RCB-9':5,
#   'J160205.48-552741.6':5,
#   'J161156.23-575527.1':5,
#   'J182943.83-190246.2':5,
#   'OGLE-GC-RCB-1':5.5,
#   'S Aps':5,
#   'U Aqr':5,
#   'V1783 Sgr':5,
#   'V2331 Sgr':5,
#   'V739 Sgr':5,
#   'WX CrA':5,
#   'EROS2-LMC-RCB-1':5,
#   'EROS2-LMC-RCB-5':5.5,
#     'NSV11154':5.5,
#     'Z Umi':5.5,
#     'J181538.25-203845.7':5,
#     'J110008.77-600303.6':5,
#     'J170343.87-385126.6':'-',
#   'EROS2-SMC-RCB-2':5.5, ##########
#   'ASAS-RCB-11':4.5,
#   'HD137613':3.5,
#   'HD175893':3.5,
#   'EROS2-LMC-RCB-2':5,
#   'EROS2-LMC-RCB-3':5,
#   'J054123.49-705801.8':5,
#   'B42':4.5,
#   'A770':5,
#   'A977':4.5,
#   'J184158.40-054819.2':4,
#   'V4017 Sgr':3,
#     'HE 1015-2050':5,
#     'ASAS J050232-7218.9':5,
#     'J175749.76-075314.9':5,
#     'J181836.38-181732.8':5,
#     'J181252.50-233304.4':5,
#     'ASAS-RCB-12':3.5,
#     'M38':4,
#     'ASAS-RCB-3':3.5,
#     'J171815.36-341339.9':'-',
#     'dLS114':5,
#   'A814':4, ##############
#   'HD182040':4, 
#   'C27':4, 
#   'B566':3,
#   'GU Sgr':3,
#   'J174119.57-250621.2':2,
#   'J174851.29-330617.0':2.5,
#   'RS Tel':3,
#   'J055643.56-715510.7':4,
#   'A223':3,
#   'A183':3,
#   'C38':3,
#   'C20':3.5, 
#   'A811':3.5,
#   'C539':3,
#     'dLS17':4.5,
#     'C528':0.5,
#     'C542':2.5,
#     'M37':0.5,
#     'P12':3.5,
#     'J172553.80-312421.1':'-',
#     'J174645.90-250314.1':3,
#   'B563':4.5, ############
#   'RZ Nor':2.5,
#   'HD148839':3,
#   'HD173409':3,
#   'A182':3,
#   'A226':3,
#   'C17':3,
#     'B565':3.5,
#     'RT Nor':3,
#     'B566':3,
#     'J174851.29-330617.0':2.5,
#     'FH Sct':1.5,
#     'J164704.67-470817.8':'-',
#     'J175031.70-233945.7':2.5,
#     'J175107.12-242357.3':'-',
#     'J180550.49-151301.7':0.5,
#     'UV Cas':0.5,
#     'V CrA':2.5,
#     'J005010.67-694357.7':2,
#   'C105':3, ############
#   'VZ Sgr':0.5,
#   'ASAS-RCB-8':1,
#   'RY Sgr':1,
#   'F75':0.5,
#   'C526':0.5,
#   'J185525.52-025145.7':0.5,
#   'V2552 Oph':0.5,
#   'V391 Sct':0.5,
#   'B567':0.5,
#   'V854 Cen':0.5,
#     'UX Ant':2,
#     'A249':2.5,
#     'J175558.51-164744.3':0.5,
#     'J172447.52-290418.6':0.5,
#     'R CrB':2,
#     'F152':1,
#     'J174111.80-281955.3':'-',
#     'UW Cen':0.5,
#     'V482 Cyg':1.5, ###########
#   'J172951.80-101715.9':0,
#   'V3795 Sgr':0.5, 
#   'Y Mus':0.5,
#   'B564':0.5, 
#   'A980':0.5,
#     'A798':0,
#     'W Men':1}

In [ ]:
plot_order_v2_sets = {'Y Mus':1,
                      'A980':2,
                      'B564':3,
                      'V3795 Sgr':4,
                      'J172951.80-101715.9':5,
#                       'A798':6,
                      #################
                      'A249':0,
                      'UX Ant':1,
                      'R CrB':2,
                      'V482 Cyg':3,
                      'ASAS-RCB-8':4,
                      'RY Sgr':5,
                      'F152':6,
                      'V391 Sct':7,
                      'J185525.52-025145.7':8,
                      'V2552 Oph':9,
                      'VZ Sgr':10,
                      'F75':11,
                      'C526':12,
                      'ASAS-RCB-10':13,
                      'W Men':14,
                      'B567':15,
                      'V854 Cen':16,
#                       'J175558.51-164744.3':15,
                      'J172447.52-290418.6':17,
#                       'UW Cen':17,
                      'J174111.80-281955.3':18,
                      ##################
                      'HD182040':0,
                      'B565':1,
                      'C105':2,
                      'C17':3,
                      'RT Nor':4,
                      'A226':5,
                      'HD173409':6,
                      'HD148839':7,
                      'A182':8,
                      'B566':9,
                      'RZ Nor':10,
                      'J174851.29-330617.0':11,
#                       'V CrA':12,
                      'J175031.70-233945.7':12,
                      'J005010.67-694357.7':13,
#                       'FH Sct':14,
                      'UV Cas':14,
#                       'J180550.49-151301.7':17,
                      'J175107.12-242357.3':15,
                      'J164704.67-470817.8':16,
                      ##################
                      'B563':0,
                      'dLS17':1,
                      'C27':2,
                      'J055643.56-715510.7':3,
                      'A811':4,
                      'C20':5,
                      'P12':6,
                      'A223':7,
                      'C38':8,
                      'A183':9,
                      'RS Tel':10,
                      'GU Sgr':11,
                      'C539':12,
                      'J174645.90-250314.1':13,
                      'C542':14,
                      'J174119.57-250621.2':15,
                      'C528':16,
                      'M37':17,
                      'J172553.80-312421.1':18,
                      ##################
#                       'M37':-3,
#                       'P12':-2,
#                       'C542':-1,
                      'J054123.49-705801.8':0,
                      'A770':1,
                      'ASAS J050232-7218.9':2,
                      'EROS2-LMC-RCB-2':3,
                      'EROS2-LMC-RCB-3':4,
                      'HE 1015-2050':5,
                      'J175749.76-075314.9':6,
#                       'J181836.38-181732.8':7,
                      'dLS114':7,
                      'A977':8,
                      'B42':9,
                      'ASAS-RCB-11':10,
                      'J181252.50-233304.4':11,
                      'A814':12,
                      'J184158.40-054819.2':13,
                      'M38':14,
                      'HD175893':15,
                      'HD137613':16,
#                       'ASAS-RCB-12':18,
#                       'ASAS-RCB-3':19,
                      'V4017 Sgr':17,
                      'EROS2-CG-RCB-1':18,
#                       'J171815.36-341339.9':21,
                      ##################
                      'EROS2-SMC-RCB-2':1,
                      'EROS2-LMC-RCB-5':0,
#                       'ES Aql':2,
                      'ASAS-RCB-7':2,
#                       'OGLE-GC-RCB-1':2,
                      'ASAS-RCB-16':3,
                      'NSV11154':4,
                      'Z Umi':5,
                      'ASAS-RCB-17':6,
                      'J161156.23-575527.1':7,
                      'ASAS-RCB-1':8,
#                       'V2331 Sgr':9,
#                       'V739 Sgr':10,
                      'ASAS-RCB-2':9,
#                       'ASAS-RCB-9':12,
                      'J160205.48-552741.6':10,
#                       'J182943.83-190246.2':14,
                      'S Aps':11,
                      'U Aqr':12,
                      'V1783 Sgr':13,
#                       'WX CrA':18,
#                       'EROS2-LMC-RCB-1':17,
                      'ASAS-RCB-3':14,
                      'EROS2-CG-RCB-6':15,
                      'J110008.77-600303.6':16,
                      'J181538.25-203845.7':17,
                      'J170343.87-385126.6':18,
                      ##################
#                       'EROS2-CG-RCB-13':-1,
                      'J173819.81-203632.1':0,
                      'ES Aql':1,
                      'J132354.47-673720.8':2,
                      'V517 Oph':3,
                      'ASAS-RCB-4':4,
                      'ASAS-RCB-14':5,
#                       'V1157 Sgr':0,
                      'MACHO-81.8394.1358':6,
#                       'EROS2-LMC-RCB-4':2,
#                       'EROS2-SMC-RCB-1':3,
                      'ASAS-RCB-15':7,
                      'SV Sge':8,
                      'J174328.50-375029.0':9,
                      'MACHO-80.6956.207':10,
                      'J194218.38-203247.5':11,
                      'WISE-ToI-5031':12,
                      'EROS2-CG-RCB-3':13,
                      'J174138.87-161546.4':14,
                      'ASAS-RCB-21':15,
#                       'EROS2-CG-RCB-13':13,
                      'J173553.02-364104.3':16,
                      'IRAS1813.5-2419':17,
#                       'ASAS-RCB-5':16,
#                       'EROS2-CG-RCB-4':17,
#                       'EROS2-CG-RCB-14':18,
#                       'EROS2-CG-RCB-6':19,
#                       'J174257.19-362052.1':20,
#                       'J175749.98-182522.8':21,
                      ##################
                      'AO Her':0,
#                       'ASAS-RCB-7':1,
#                       'J132354.47-673720.8':2,
#                       'ASAS-RCB-4':3,
                      'ASAS-RCB-18':1,
                      'MACHO 401.48170.2237':2,
#                       'ASAS-RCB-19':6,
                      'EROS2-CG-RCB-5':3,
#                       'J175521.75-281131.2':8,
                      'J184246.26-125414.7':4,
                      'J182723.38-200830.1':5,
                      'J183649.54-113420.7':6,
                      'J150104.50-563325.1':7,
                      'J182235.25-033213.2':8,
                      ##################
#                       'ASAS-RCB-20':0,
#                       'ES Aql':1,
#                       'V517 Oph':2,
                      'J005113.58-731036.3':3,
#                       'J054221.91-690259.3':4,
#                       'ASAS-RCB-14':5,
#                       'J173202.75-432906.1':6,
#                       'EROS2-SMC-RCB-3':7,
#                       'EROS2-CG-RCB-11':8,
#                       'EROS2-CG-RCB-9':9,
#                       'J173737.07-072828.1':10,
                      ##################
                      'J182334.24-282957.1':0,
                      'MACHO 308.38099.66':1,
#                       'EROS2-CG-RCB-7':2,
                      'J163450.35-380218.5':2,
                      'J171908.50-435044.6':3}
#                       'EROS2-LMC-RCB-6':5}

In [ ]:
final_set_v2_dict = {}
final_set_v2_arr = []
for i in range(0,len(store_names)):
    if store_names[i] in final_sets_v2[0]:
        final_set_v2_dict[store_names[i]] = 0
        final_set_v2_arr.append(7)
    elif store_names[i] in final_sets_v2[1]:
        final_set_v2_dict[store_names[i]] = 1
        final_set_v2_arr.append(6)
    elif store_names[i] in final_sets_v2[2]:
        final_set_v2_dict[store_names[i]] = 2
        final_set_v2_arr.append(5)
    elif store_names[i] in final_sets_v2[3]:
        final_set_v2_dict[store_names[i]] = 3
        final_set_v2_arr.append(4)
    elif store_names[i] in final_sets_v2[4]:
        final_set_v2_dict[store_names[i]] = 4
        final_set_v2_arr.append(3)
    elif store_names[i] in final_sets_v2[5]:
        final_set_v2_dict[store_names[i]] = 5
        final_set_v2_arr.append(2)
    elif store_names[i] in final_sets_v2[6]:
        final_set_v2_dict[store_names[i]] = 6
        final_set_v2_arr.append(1)
    elif store_names[i] in final_sets_v2[7]:
        final_set_v2_dict[store_names[i]] = 7
        final_set_v2_arr.append(0)
#     elif store_names[i] in final_sets_v2[8]:
#         final_set_v2_dict[store_names[i]] = 8
#         final_set_v2_arr.append(1)
#     elif store_names[i] in final_sets_v2[9]:
#         final_set_v2_dict[store_names[i]] = 9
#         final_set_v2_arr.append(0)
print(len(final_set_v2_arr))

In [ ]:
full_class_dict = {}
for i in range(0,len(master_w_spec)):
    if master_w_spec['Name'][i] in final_set_v2_dict.keys():
#         print(master_w_spec['Name'][i])
        temperatureclass = master_w_spec['Class'][i]
        if master_w_spec['Carbon'][i] == '-':
            carbonclass = 'C$_2$-'
        else:
            try:
                carbonclass = 'C$_2$'+master_w_spec['Carbon'][i][3:]
            except TypeError:
                carbonclass = 'no C info'
        if type(master_w_spec['Additional'][i]) == str:
            additionalclass = master_w_spec['Additional'][i]
        else:
            additionalclass = ''
#         print(temperatureclass+' '+carbonclass+' '+additionalclass)
        try:
            full_class_dict[master_w_spec['Name'][i]] = temperatureclass+' '+carbonclass+' '+additionalclass
        except TypeError:
            full_class_dict[master_w_spec['Name'][i]] = 'no class info'
    
full_class_dict

In [ ]:
# for i in range(0,len(store_names)):
#     if store_names[i] in not_classed_names and store_names[i] not in final_set_v2_dict.keys():
#         print(store_names[i])

In [ ]:
population_size = []
for xx in range(0,8):
#     print(xx,len(final_sets_v2[xx])
    population_size.append(len(final_sets_v2[xx]))
          
classes = np.linspace(0,7,8)
# population_size = list(data.values())
plt.bar(classes, population_size, color ='tab:blue',
        width = 0.5)
 
plt.xlabel("Class Index")
plt.ylabel("No. of stars")
plt.title("Stars Per Class")
plt.show()

In [ ]:
print(population_size)
print(19+19+19+9)
print(5+19)
print(18+18)
print(sum(population_size))

In [ ]:
labels = ['HdC0', 'HdC1', 'HdC2', 'HdC3', 'HdC4', 'HdC5', 'HdC6', 'HdC7']#, 'HdC8', 'HdC9']
total_count = population_size

x = np.arange(len(labels))  # the label locations
width = 0.4 # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x, total_count, width)#, label='RCB')
# rects2 = ax.bar(x + width/2, dl_count, width, label='dLHdC')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Total No. of Stars')
# ax.set_title('Scores by group and gender')
ax.set_xticks(x)
ax.set_xticklabels(labels)
# ax.legend()

# ax.bar_label(rects1, padding=3)
# ax.bar_label(rects2, padding=3)

fig.tight_layout()

# plt.savefig('class_histogram_total')
plt.show()

In [ ]:
print(sum(rcb_count))
print(sum(dl_count))
print(sum(rcb_count)+sum(dl_count))
print(sum(population_size))
print(population_size)

In [ ]:
labels = ['HdC0', 'HdC1', 'HdC2', 'HdC3', 'HdC4', 'HdC5', 'HdC6', 'HdC7']#, 'HdC8', 'HdC9']
rcb_count = [3, 14, 9, 5, 11, 20, 18, 9]#, 11, 6]
dl_count = [2, 5, 8, 13, 8, 0, 0, 0]#, 0, 0]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(2, 1, figsize=[10,12], constrained_layout=True)
rects1 = ax[0].bar(x - width/2, rcb_count, width, label='RCB')
rects2 = ax[0].bar(x + width/2, dl_count, width, label='dLHdC',color='tab:red')

ax[0].set_ylabel('No. of stars')
ax[0].set_xticks(x)
ax[0].set_xticklabels(labels,Fontsize=15)
ax[0].legend()


rects1 = ax[1].bar(x, total_count, 0.4,color='tab:grey')

ax[1].set_ylabel('Total No. of Stars')
ax[1].set_xticks(x)
ax[1].set_xticklabels(labels,Fontsize=15)

ax[0].set_yticks([0,5,10,15,20])
ax[1].set_yticks([0,5,10,15,20])


fig.tight_layout()

# plt.savefig('subplot_histogram.eps')
plt.show()

In [ ]:
for xx in range(0,8):
    plt.figure(figsize=(12,len(final_sets_v2[xx])))
    ii=0
#     x=0
    for i in range(0,len(store_names)):
        if store_names[i] in final_set_v2_dict.keys():
            if store_names[i] in final_sets_v2[xx]:
                try:
                    x=plot_order_v2_sets[store_names[i]]
                except KeyError:
                    x=0

                wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])
                wavelength, flux = bad_pixel_rejection(wavelength,flux,3)
                
                plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
#                 greater = np.where(wavelength>6900)
#                 plt.plot(wavelength,flux*2/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
                plt.annotate(store_names[i],xy=(9910,-x+0.52),annotation_clip=False,size=13)
#                 plt.annotate(store_names[i],xy=(6910,-x+0.52+2),annotation_clip=False,size=13)
                try:
                    plt.annotate(r'%s' % full_class_dict[store_names[i]],xy=(9910,-x+0.2),annotation_clip=False,size=10)
                except KeyError:
                    plt.annotate('no class info',xy=(9910,-x+0.2),annotation_clip=False,size=10)
#                 x=x+1
            ii = ii+1
#     plt.axvline(6707,color='k')
    plt.yticks([])
    plt.title('HdC ' + np.str(xx))
    plt.xlim(right=9800)
#     plt.xlim(6500,6900)
    plt.ylabel('Norm. Flux + Offset')
    plt.xlabel(r'Wavelength ($\AA$)')
    plt.savefig('final_sets/set_number_' + np.str(xx) + '.eps',bbox_inches='tight')
    plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] == 'ASAS-RCB-12':
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        
        wavelength, flux = bad_pixel_rejection(wavelength,flux)
        plt.plot(wavelength,flux/np.max(flux))
        plt.plot(model_lam,model_flux,color='tab:red')
        plt.plot(model_lam1,model_flux1,color='tab:green')
        plt.xlim(6000,9000)
        plt.show()

In [ ]:
print(len(final_set_v2_dict))

In [ ]:
plt.figure(figsize=(15,4))
for i in range(0,len(store_names)):
    if store_names[i] in ['ASAS-RCB-18','ASAS-DYPer-1']:#'J182334.24-282957.1': #EROS2-SMC-RCB-3
#             x=plot_order_final_sets[store_names[i]]
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

#         flux = convolve(flux,Gaussian1DKernel(1))
        
        greater = np.where(wavelength>6900)
        if store_names[i] == 'ASAS-DYPer-1':
            plt.plot(wavelength,flux*0.75/(np.average(flux[greater][:50])),label=store_names[i])#+r' (HdC7 C$_2$5.5)',color='tab:blue')
        else:
            plt.plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i]+r' (HdC7 C$_2$5.5)')
#             plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
#         plt.annotate(store_names[i],xy=(7010,-x+0.2),annotation_clip=False,size=15)
        
for i in range(0,len(barnbaum_stars_lam)):
    if barnbaum_spec['Name'][i] == 'DY Per':
        plt.plot(barnbaum_stars_lam[i],barnbaum_stars_flux[i]*1.5/np.max(barnbaum_stars_flux[i]),label=r'DY Per (C-Hd4.5 C$_2$6)',color='tab:red')
        
plt.yticks([])
plt.xticks(fontsize=15)
# plt.title('HdC 7')
plt.ylim(top=1.7)
# plt.ylim(bottom=-0.5)
# plt.ylim(top=1.2)
plt.xlim(right=7500)
# plt.xlim(4500,5500)
# plt.xlim(6000,7500)
# plt.xlim(5800,7200)
# plt.xlim(7000,8000)
# plt.xlim(8000,9000)
# plt.xlim(3800,5000)
# plt.xlim(4500,5800)
plt.legend(fontsize=12.5)
plt.ylabel('Norm. Flux',fontsize=25)
plt.xlabel(r'Wavelength ($\AA$)',fontsize=25)
# plt.savefig('DYPer_compare',bbox_inches='tight')
plt.show()

In [ ]:
# plt.figure(figsize=(15,4))
f,ax = plt.subplots(4, 1, figsize=[12,12], constrained_layout=True)
for i in range(0,len(store_names)):
    if store_names[i] in ['ASAS-RCB-18','ASAS-DYPer-1']:#'J182334.24-282957.1': #EROS2-SMC-RCB-3
#             x=plot_order_final_sets[store_names[i]]
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

#         flux = convolve(flux,Gaussian1DKernel(1))
        
        greater = np.where(wavelength>6900)
        if store_names[i] == 'ASAS-DYPer-1':
            ax[0].plot(wavelength,flux*0.75/(np.average(flux[greater][:50])),label=store_names[i])
            ax[1].plot(wavelength,flux*0.75/(np.average(flux[greater][:50])),label=store_names[i])
            ax[2].plot(wavelength,flux*0.75/(np.average(flux[greater][:50])),label=store_names[i])
            ax[3].plot(wavelength,flux*0.75/(np.average(flux[greater][:50])),label=store_names[i])
        else:
            ax[0].plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i]+r' (HdC7 C$_2$5.5)')
            ax[1].plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i]+r' (HdC7 C$_2$5.5)')
            ax[2].plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i]+r' (HdC7 C$_2$5.5)')
            ax[3].plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i]+r' (HdC7 C$_2$5.5)')
#             plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
#         plt.annotate(store_names[i],xy=(7010,-x+0.2),annotation_clip=False,size=15)
        
for i in range(0,len(barnbaum_stars_lam)):
    if barnbaum_spec['Name'][i] == 'DY Per':
        ax[0].plot(barnbaum_stars_lam[i],barnbaum_stars_flux[i]*1.5/np.max(barnbaum_stars_flux[i]),label=r'DY Per (C-Hd4.5 C$_2$6)',color='tab:red')
        ax[1].plot(barnbaum_stars_lam[i],barnbaum_stars_flux[i]*1.5/np.max(barnbaum_stars_flux[i]),label=r'DY Per (C-Hd4.5 C$_2$6)',color='tab:red')
        ax[2].plot(barnbaum_stars_lam[i],barnbaum_stars_flux[i]*1.5/np.max(barnbaum_stars_flux[i]),label=r'DY Per (C-Hd4.5 C$_2$6)',color='tab:red')
        ax[3].plot(barnbaum_stars_lam[i],barnbaum_stars_flux[i]*1.5/np.max(barnbaum_stars_flux[i]),label=r'DY Per (C-Hd4.5 C$_2$6)',color='tab:red')
        
ax[0].set_yticks([])
ax[1].set_yticks([])
ax[2].set_yticks([])
ax[3].set_yticks([])
ax[1].set_xlim(4000,5800)
ax[2].set_xlim(5800,7000)
ax[3].set_xlim(7000,9000)
ax[0].legend(fontsize=12.5)
ax[0].set_ylabel('Norm. Flux',fontsize=25)
ax[1].set_ylabel('Norm. Flux',fontsize=25)
ax[2].set_ylabel('Norm. Flux',fontsize=25)
ax[3].set_ylabel('Norm. Flux',fontsize=25)
ax[3].set_xlabel(r'Wavelength ($\AA$)',fontsize=25)
ax[0].tick_params(axis='both', which='major', labelsize=12)
ax[1].tick_params(axis='both', which='major', labelsize=12)
ax[2].tick_params(axis='both', which='major', labelsize=12)
ax[3].tick_params(axis='both', which='major', labelsize=12)
# plt.savefig('detailed_DYPer_compare.eps',bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
for i in range(0,len(store_names)):
    if store_names[i] in ['EROS2-SMC-RCB-3','EROS2-LMC-DYPer-6']:
#             x=plot_order_final_sets[store_names[i]]
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        wavelength, flux = bad_pixel_rejection(wavelength,flux,3)

#         flux = convolve(flux,Gaussian1DKernel(1))
        
        greater = np.where(wavelength>6900)                
        plt.plot(wavelength,flux/(np.average(flux[greater][:50])),label=store_names[i])#+r' (HdC9 C$_2$6.5 Li)',color='tab:blue')
#             plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
#         plt.annotate(store_names[i],xy=(7010,-x+0.2),annotation_clip=False,size=15)
        
# for i in range(0,len(barnbaum_stars_lam)):
#     if barnbaum_spec['Name'][i] == 'DY Per':
#         plt.plot(barnbaum_stars_lam[i],barnbaum_stars_flux[i]*1.5/np.max(barnbaum_stars_flux[i]),label=r'DY Per (C-Hd4.5 C$_2$6)',color='tab:red')
        
plt.yticks([])
# plt.title('HdC 7')
plt.ylim(top=2.1)
# plt.ylim(bottom=-0.5)
# plt.ylim(top=1.2)
plt.xlim(right=7500)
# plt.xlim(4500,5500)
plt.xlim(8000,9000)
plt.legend(fontsize=15)
plt.ylabel('Norm. Flux')
plt.xlabel(r'Wavelength ($\AA$)')
# plt.savefig('DYPer_compare',bbox_inches='tight')
plt.show()

In [ ]:
for xx in range(0,8):
    plt.figure(figsize=(12,len(final_sets[xx])))
    ii=0
    x=0
    for i in range(0,len(store_names)):
        if store_names[i] in final_set_dict.keys():
            if store_names[i] in final_sets[xx]:
#                 x=plot_order_final_sets[store_names[i]]
                wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])
                wavelength, flux = bad_pixel_rejection(wavelength,flux,3)
                
                plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
                plt.annotate(store_names[i],xy=(9910,-x+0.2),annotation_clip=False,size=15)
                x=x+1
            ii = ii+1

    plt.yticks([])
    plt.title('HdC ' + np.str(xx))
    plt.xlim(right=9800)
#     plt.savefig('final_sets/set_number_' + np.str(xx),bbox_inches='tight')
    plt.show()

In [ ]:
carbon_index_arr = []
for i in range(0,len(good_names)):
    carbon_index_arr.append(carbon_index_final_sets[good_names[i]])
    
print(carbon_index_arr)

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in carbon_index_final_sets.keys():
        print(store_names[i], 'C2', carbon_index_final_sets[store_names[i]])

In [ ]:
for xx in range(0,8):
    plt.figure(figsize=(12,len(final_sets[xx])))
    ii=0
#     x=0
    for i in range(0,len(store_names)):
        if store_names[i] in final_set_dict.keys():
            if store_names[i] in final_sets[xx]:
                x=plot_order_final_sets[store_names[i]]
                wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])
                wavelength, flux = bad_pixel_rejection(wavelength,flux,3)
                
                greater = np.where(wavelength>6500)
                if store_names[i] in bad_list:
                    plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:red')
                else:
                    plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
#                 plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
#                 plt.plot(wavelength,flux/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
                plt.annotate(store_names[i] + ' C2 ' + str(carbon_index_final_sets[store_names[i]]),
                             xy=(10010,-x+0.2),annotation_clip=False,size=15)
#                 x=x+1
            ii = ii+1

#     plt.axvline(6347,color='k')#si
#     plt.axvline(6371,color='k')#si
#     plt.axvline(4215,color='k')#sr
#     plt.axvline(4077,color='k')#sr
    
#     plt.axvline(6563,color='tab:green')
#     plt.axvline(4861,color='tab:green')
#     plt.axvline(4340,color='tab:green')
#     plt.axvline(4101,color='tab:green')
    
#     plt.axvline(5165,color='k')
#     plt.axvline(4737,color='k')
#     plt.axvline(5635,color='k')
    
    plt.yticks([])
    plt.title('HdC ' + np.str(xx))
    plt.xlim(right=9800)
#     plt.xlim(4650,5500)
#     plt.savefig('final_sets/carbon_set_number_' + np.str(xx),bbox_inches='tight')
    plt.show()

In [ ]:
len(balmer)

# Other Index Calibration

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] not in balmer:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux)
        plt.axvline(6563,color='tab:green')
        plt.axvline(4861,color='tab:green')
        plt.axvline(4340,color='tab:green')
        plt.axvline(4101,color='tab:green')
        plt.title(store_names[i])
        plt.xlim(6400,6700)
        plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in balmer:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux)
        plt.axvline(6563,color='tab:green')
#         plt.axvline(4861,color='tab:green')
#         plt.axvline(4300,color='tab:green')
#         plt.axvline(4315,color='tab:green')
        plt.title(store_names[i])
        plt.xlim(6300,6750)
        plt.show()

In [ ]:
balmer = ['V854 Cen','ASAS-RCB-5','J172951.80-101715.9','J185525.52-025145.7',
         'RY Sgr','V2552 Oph','V391 Sct','VZ Sgr','HD148839','A183','A223',
         'A226','A770','A811','A814','A980','B42','B563','B564','C17','C20','C38','C526','F75',
         'HE 1015-2050','B565','A249','UX Ant','J172447.52-290418.6','W Men','RT Nor','J181252.50-233304.4',
         'R CrB','V482 Cyg','UW Cen','B566','V CrA','J164704.67-470817.8','C542','C528','M37','P12',
         'J181836.38-181732.8','M38','ASAS-RCB-12','J110008.77-600303.6','J181538.25-203845.7',
         'MACHO-81.8394.1358','IRAS1813.5-2419','J173202.75-432906.1','J173737.07-072828.1','J171908.50-435044.6',
         'ASAS-RCB-10','ASAS-RCB-3','EROS2-CG-RCB-1','A182','J171815.36-341339.9',
         'J174111.80-281955.3','J174257.19-362052.1','J180550.49-151301.7']#F152

balmer_dict = {'V854 Cen':3,'ASAS-RCB-5':1,'J172951.80-101715.9':1,'J174119.57-250621.2':0.5,'J185525.52-025145.7':1.5,
         'RY Sgr':1.5,'V2552 Oph':1,'V391 Sct':1,'VZ Sgr':0.5,'HD148839':2,'A183':1.5,'A223':0.25,
         'A226':0.25,'A770':0.5,'A811':1.5,'A814':0.5,'A980':1,'B42':0.25,
               'B563':0.5,'B564':0.5,'C17':1.5,'C20':1.5,'C38':0.5,'C526':1,'F75':0.5,
              'HE 1015-2050':1,'B565':1,'A249':1,'UX Ant':1,'J172447.52-290418.6':1,'W Men':0.5,'RT Nor':0.25,
              'J181252.50-233304.4':0.5}

balmer_order = {'V854 Cen':31,'ASAS-RCB-5':14,'J172951.80-101715.9':16,'J174119.57-250621.2':3,'J185525.52-025145.7':23,
         'RY Sgr':25,'V2552 Oph':21,'V391 Sct':24,'VZ Sgr':9,'HD148839':30,'A183':27,'A223':1,
         'A226':2,'A770':4,'A811':26,'A814':8,'A980':20,'B42':0,
               'B563':11,'B564':10,'C17':29,'C20':28,'C38':12,'C526':13,'F75':5,
               'HE 1015-2050':22,'B565':18,'A249':19,'UX Ant':15,'J172447.52-290418.6':17,'W Men':7,'RT Nor':-1,
               'J181252.50-233304.4':6}

In [ ]:
# plt.figure(figsize=(8,30))
f,ax = plt.subplots(1, 4, figsize=[15,len(balmer)+8], constrained_layout=True)
for i in range(0,len(store_names)):
    if store_names[i] in balmer:
        x=balmer_order[store_names[i]]
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        if store_names[i] == 'B565':
            for ind in range(0,len(flux)):
                if flux[ind] > 1.65e-13:
                    flux[ind] = 1.34981181100152e-13
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))

        greater = np.where(wavelength>6500)                
        ax[3].plot(wavelength,flux*1.75/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
        
#         plt.plot(wavelength,flux/np.max(flux)-x,label=store_names[i],color='tab:blue')
        ax[3].annotate(store_names[i]+' H'+str(balmer_dict[store_names[i]]),xy=(6610,-x+1.5),annotation_clip=False,size=15)
        
        greater = np.where(wavelength>4860)                
        ax[2].plot(wavelength,flux*1.5/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
        
        greater = np.where(wavelength>4340) 
        ax[1].plot(wavelength,flux*1.25/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
        
        greater = np.where(wavelength>4100) 
        ax[0].plot(wavelength,flux*1.25/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
        
        
        
ax[3].axvline(6563,color='k')
ax[3].set_xlim(6510,6610)
ax[3].set_ylim(-len(balmer),3.5)
ax[3].set_yticks([])

ax[2].axvline(4861,color='k')
ax[2].set_xlim(4810,4910)
ax[2].set_ylim(-len(balmer),3.5)
ax[2].set_yticks([])

ax[1].axvline(4340,color='k')
ax[1].set_xlim(4290,4390)
ax[1].set_ylim(-len(balmer),3.5)
ax[1].set_yticks([])

ax[0].axvline(4101,color='k')
ax[0].set_xlim(4050,4150)
ax[0].set_ylim(-len(balmer),3.5)
ax[0].set_yticks([])

plt.show()

In [ ]:
# plt.figure(figsize=(8,30))
f,ax = plt.subplots(1, 2, figsize=[15,15], constrained_layout=True)
for i in range(0,len(store_names)):
    if store_names[i] in balmer:
        x=balmer_order[store_names[i]]
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        if store_names[i] == 'B565':
            for ind in range(0,len(flux)):
                if flux[ind] > 1.65e-13:
                    flux[ind] = 1.34981181100152e-13
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))

        greater = np.where(wavelength>6500) 
        if x < 17:
            ax[0].plot(wavelength,flux*1.75/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
            ax[0].annotate(store_names[i]+' H'+str(balmer_dict[store_names[i]]),xy=(6615,-x+1.5),annotation_clip=False,size=15)
        else:
            ax[1].plot(wavelength,flux*1.75/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
            ax[1].annotate(store_names[i]+' H'+str(balmer_dict[store_names[i]]),xy=(6615,-x+1.5),annotation_clip=False,size=15)
        
#         greater = np.where(wavelength>4860)                
#         ax[2].plot(wavelength,flux*1.5/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
        
#         greater = np.where(wavelength>4340) 
#         ax[1].plot(wavelength,flux*1.25/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
        
#         greater = np.where(wavelength>4100) 
#         ax[0].plot(wavelength,flux*1.25/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
        
        
        
ax[1].axvline(6563,color='k')
ax[1].set_xlim(6510,6610)
ax[0].set_ylim(-15,4)
ax[1].set_yticks([])

ax[0].axvline(6563,color='k')
ax[0].set_xlim(6510,6610)
ax[1].set_ylim(-30,-14)
ax[0].set_yticks([])

ax[0].set_xlabel(r'Wavelength ($\AA$)')
ax[1].set_xlabel(r'Wavelength ($\AA$)')

# ax[2].axvline(4861,color='k')
# ax[2].set_xlim(4810,4910)
# ax[2].set_ylim(-25,1.5)
# ax[2].set_yticks([])

# ax[1].axvline(4340,color='k')
# ax[1].set_xlim(4290,4390)
# ax[1].set_ylim(-25,1.5)
# ax[1].set_yticks([])

# ax[0].axvline(4101,color='k')
# ax[0].set_xlim(4050,4150)
# ax[0].set_ylim(-25,1.5)
# ax[0].set_yticks([])

# plt.savefig('halpha',bbox_inches='tight')
plt.show()

In [ ]:
# plt.figure(figsize=(8,30))
f,ax = plt.subplots(1, 2, figsize=[15,30], constrained_layout=True)
x = 0
for i in range(0,len(store_names)):
    if store_names[i] in balmer and store_names[i] in final_set_v2_dict.keys():
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        if store_names[i] == 'B565':
            for ind in range(0,len(flux)):
                if flux[ind] > 1.65e-13:
                    flux[ind] = 1.34981181100152e-13
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))

        greater = np.where(wavelength>6500) 
        if x < len(balmer)/2:
            ax[0].plot(wavelength,flux*1.75/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
#             ax[0].annotate(store_names[i]+' H'+str(balmer_dict[store_names[i]]),xy=(6615,-x+1.5),annotation_clip=False,size=15)
            ax[0].annotate(store_names[i],xy=(6615,-x+1.5),annotation_clip=False,size=15)
        else:
            ax[1].plot(wavelength,flux*1.75/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
#             ax[1].annotate(store_names[i]+' H'+str(balmer_dict[store_names[i]]),xy=(6615,-x+1.5),annotation_clip=False,size=15)
            ax[1].annotate(store_names[i],xy=(6615,-x+1.5),annotation_clip=False,size=15)
        x = x+1
        
        
ax[1].axvline(6563,color='k')
ax[1].set_xlim(6510,6610)
ax[0].set_ylim(-len(balmer)/2,4)
ax[1].set_yticks([])

ax[0].axvline(6563,color='k')
ax[0].set_xlim(6510,6610)
ax[1].set_ylim(-len(balmer),-len(balmer)/2+3)
ax[0].set_yticks([])

ax[0].set_xlabel(r'Wavelength ($\AA$)')
ax[1].set_xlabel(r'Wavelength ($\AA$)')

# plt.savefig('halpha',bbox_inches='tight')
plt.show()

In [ ]:
print(len(balmer))
print(x)

In [ ]:
halpha_area = []

for i in range(0,len(store_names)):
    if store_names[i] in balmer:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                             store_Av_values[i],store_names[i])

        lines = [6563]

        name = store_names[i]
        flux = flux/np.max(flux)

        eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,name,lines,
                                                                              center_tolerance=1,
                                                                              width_tolerance=15,
                                                                              continuum_region=50,
                                                                              continuum_width=0)



        halpha_area.append(eq_width_array[0])

In [ ]:
ii=0
for i in range(0,len(store_names)):
    if store_names[i] in balmer:
        print(store_names[i],halpha_area[ii])
        ii=ii+1

In [ ]:
hbeta_area = []

for i in range(0,len(store_names)):
    if store_names[i] in balmer:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                             store_Av_values[i],store_names[i])

        lines = [4861]

        name = store_names[i]
        flux = flux/np.max(flux)

        eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,name,lines,
                                                                              center_tolerance=1,
                                                                              width_tolerance=15,
                                                                              continuum_region=50,
                                                                              continuum_width=0)



        hbeta_area.append(eq_width_array[0])

In [ ]:
lithium = ['ASAS-RCB-4','ASAS-RCB-5','J132354.47-673720.8','J174138.87-161546.4',
          'J182723.38-200830.1','J184246.26-125414.7','RZ Nor','V391 Sct',
          'UW Cen','EROS2-LMC-RCB-3','EROS2-SMC-RCB-1','A183','A770','A811',
          'A814','A977','A980','B563','B564','C17','C20','C38','F152','F75',
          'MACHO-81.8394.1358','MACHO-80.6956.207','ASAS J050232-7218.9','HE 1015-2050',
          'dLS17','B565','S Aps','MACHO 401.48170.2237','EROS2-CG-RCB-13','EROS2-CG-RCB-3',
          'WISE-ToI-5031','R CrB','ASAS-RCB-14','ASAS-RCB-20','ASAS-RCB-7','ES Aql',
           'C528','C526','V CrA','V2331 Sgr','V517 Oph','HD148839','RY Sgr','OGLE-GC-RCB-1',
          'J173202.75-432906.1','J181538.25-203845.7','J194218.38-203247.5','J173737.07-072828.1']

lithium_dict = {'EROS2-LMC-RCB-3':1,
'EROS2-SMC-RCB-1':2,
'A814':3,
'B564':4,
'A980':5,
'RZ Nor':6,
'ASAS-RCB-5':7,
'J132354.47-673720.8':8,
'C17':9,
'F152':10,
'ASAS-RCB-4':11,
'A183':12,
'F75':13,
'A770':14,
'A811':15,
'A977':16,
'J182723.38-200830.1':17,
'J184246.26-125414.7':18,
'UW Cen':19,
'B563':20,
'J174138.87-161546.4':21,
'C38':22,
'V391 Sct':23,
'C20':24,
'MACHO-81.8394.1358':25,
'MACHO-80.6956.207':26,
'ASAS J050232-7218.9':27,
'HE 1015-2050':28,
'dLS17':29,
'B565':30,
'S Aps':31,
'MACHO 401.48170.2237':32,
'EROS2-CG-RCB-13':33,
'EROS2-CG-RCB-3':34,
'WISE-ToI-5031':35}

In [ ]:
print(len(lithium))

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] not in lithium:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux)
        plt.axvline(6707,color='tab:red')
        plt.axvline(6760,color='tab:green')
        plt.axvline(6672,color='tab:green')
    #     plt.axvline(4101,color='tab:green')
        plt.title(store_names[i])
        plt.xlim(6500,6900)
        plt.show()

In [ ]:
lithium_area = []

for i in range(0,len(store_names)):
    if store_names[i] in lithium:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                             store_Av_values[i],store_names[i])

        lines = [6707]

        name = store_names[i]
        flux = flux/np.max(flux)

        eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,name,lines,
                                                                              center_tolerance=1,
                                                                              width_tolerance=10,
                                                                              continuum_region=50,
                                                                              continuum_width=0)



        lithium_area.append(eq_width_array[0])

In [ ]:
ii=0
for i in range(0,len(store_names)):
    if store_names[i] in lithium:
        print(store_names[i],lithium_area[ii])
        ii=ii+1

In [ ]:
# plt.figure(figsize=(5,30))
f,ax = plt.subplots(1, 2, figsize=[15,20], constrained_layout=True)
x=0
for i in range(0,len(store_names)):
    if store_names[i] in lithium and store_names[i] in final_set_v2_dict.keys():
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        if store_names[i] == 'B565':
            for ind in range(0,len(flux)):
                if flux[ind] > 1.65e-13:
                    flux[ind] = 1.34981181100152e-13
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))

        greater = np.where(wavelength>6700) 
        if x > len(lithium)/2:
            ax[1].plot(wavelength,flux*2/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
            ax[1].annotate(store_names[i],xy=(6790,-x+1.75),
                     annotation_clip=False,size=15)
        else:
            ax[0].plot(wavelength,flux*2/(np.average(flux[greater][:50]))-x,label=store_names[i],color='tab:blue')
            ax[0].annotate(store_names[i],xy=(6790,-x+1.75),
                     annotation_clip=False,size=15)
        x = x+1
        
        
        
        
ax[0].axvline(6707,color='k')
ax[0].set_xlim(6637,6787)
ax[0].set_ylim(-len(lithium)/2,5)
ax[0].set_yticks([])

ax[1].axvline(6707,color='k')
ax[1].set_xlim(6637,6787)
ax[1].set_ylim(-len(lithium),-len(lithium)/2+3)
ax[1].set_yticks([])

ax[0].set_xlabel(r'Wavelength ($\AA$)')
ax[1].set_xlabel(r'Wavelength ($\AA$)')

# plt.savefig('lithium',bbox_inches='tight')
plt.show()

In [ ]:
print(x)

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in ['MACHO 401.48170.2237','EROS2-CG-RCB-13','EROS2-CG-RCB-3','WISE-ToI-5031',
                         'RT Nor','J181252.50-233304.4']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux)
        plt.axvline(4077,color='tab:green')
        plt.title(store_names[i])
        plt.xlim(3800,4150)
        plt.show()

In [ ]:
for i in range(0,len(store_names)):
#     if store_names[i] not in barium:
    wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])
    plt.plot(wavelength,flux)
    plt.axvline(6496,color='tab:red')
    plt.title(store_names[i])
    plt.xlim(6300,6700)
    plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] == 'V3795 Sgr':
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))

        plt.plot(wavelength,flux,label=store_names[i])#,color='tab:blue')
        
# plt.yticks([])
plt.axvline(5168,ymin=0.55,ymax=0.7,color='k',linewidth=2)
# plt.axhline()
plt.plot([5058,5168],[1.05e-12,0.98e-12],linewidth=2,color='k')
plt.annotate(r'C$_2$',xy=(5145,1.27e-12),annotation_clip=False,size=25)

plt.axvline(4670,ymin=0.7,ymax=0.85,color='k',linewidth=2)
# plt.axhline()
plt.plot([4560,4670],[1.295e-12,1.23e-12],linewidth=2,color='k')
plt.annotate(r'C$_2$',xy=(4650,1.53e-12),annotation_clip=False,size=25)
# plt.axvline(4737,color='k')
plt.xlim(4100,5700)
# plt.ylim(bottom=0.1e-12)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'Flux (erg s$^{-1}$cm$^{-2}$$\AA^{-1}$)')#ergs s-1 cm-2 A-1
# plt.savefig('V3795Sgr_band.eps')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] == 'V3795 Sgr':
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                 store_Av_values[i],store_names[i])
        left_of_telluric_lam = wavelength[(wavelength < 7575)]
        left_of_telluric_flux = flux[(wavelength < 7575)]
        right_of_telluric_lam = wavelength[(wavelength > 7700)]
        right_of_telluric_flux = flux[(wavelength > 7700)]
        wavelength = np.concatenate((np.asarray(left_of_telluric_lam),np.asarray(right_of_telluric_lam)))
        flux = np.concatenate((np.asarray(left_of_telluric_flux),np.asarray(right_of_telluric_flux)))

        plt.plot(wavelength,flux,label=store_names[i])#,color='tab:blue')
        
# plt.axvline(6678,color='tab:red')
# plt.axvline(5876,color='tab:red')
# plt.axvline(7065,color='tab:red')
# plt.axvline(3889,color='tab:red')
# plt.axvline(4471,color='tab:red')
# plt.axvline(4388,color='tab:red')
# plt.axvline(4922,color='tab:red')

plt.axvline(6580,color='k')

# plt.axvline(4026,color='b')
# plt.axvline(4471,color='b')
# plt.axvline(4009,color='b')
# plt.axvline(4120,color='b')
# plt.axvline(4144,color='b')
# plt.axvline(4388,color='b')
# plt.axvline(3834,color='b')
# plt.axvline(3867,color='b')
# # plt.axvline(4200,color='g') #heII
# plt.axvline(4713,color='b')
# plt.axvline(4267,color='g')

# # plt.yticks([])
# plt.axvline(5168,ymin=0.55,ymax=0.7,color='k',linewidth=2)
# # plt.axhline()
# plt.plot([5050,5168],[1.1e-12,0.98e-12],linewidth=2,color='k')
# plt.annotate(r'C$_2$',xy=(5145,1.27e-12),annotation_clip=False,size=25)
# # plt.axvline(4737,color='k')
plt.xlim(6400,6800)
# plt.ylim(top=0.5)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'Flux (erg s$^{-1}$cm$^{-2}$$\AA^{-1}$)')#ergs s-1 cm-2 A-1
# plt.savefig('V3795Sgr_band')
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in good_names:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux)
        plt.axvline(4744,color='tab:red')
        plt.axvline(4737,color='tab:green')
    #     plt.axvline(6672,color='tab:green')
    #     plt.axvline(4101,color='tab:green')
        plt.title(store_names[i])
        plt.xlim(4600,4900)
        plt.show()

# Miscellaneous Playing

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in ['RZ Nor','UX Ant']:
        if store_names[i] == 'UX Ant':
            print(store_Av_values[i])
            wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     1.15,store_names[i])
        else:
            print(store_Av_values[i])
            wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux/np.max(flux),label=store_names[i])
#         plt.annotate(store_names[i],xy=(9010),annotation_clip=False,size=15)


plt.yticks([])
# plt.xlim(4500,5500)
plt.legend()
plt.show()

In [ ]:
model = np.loadtxt('models/MODELS_VIS_Courtney-Jan2021-CN-or-C2-ONLY-WithoutBB/5000g1.0z+0.00m1.0t05ptin85_1000-25000_xit1.0_O1618-10.0_C1213-inf_N85__Only-C2.spec_0.3-1.0_res1ang'
                   ,skiprows=3)
model_lam1 = model[:,0]
model_flux1 = model[:,1]
model_flux1 = convolve(model_flux1,Gaussian1DKernel(1))

plt.plot(model_lam1,model_flux1)
plt.xlim(4000,9000)
plt.show()

In [ ]:
model = np.loadtxt('models/MODELS_VIS_Courtney-Jan2021-CN-or-C2-ONLY-WithoutBB/5000g1.0z+0.00m1.0t05ptin85_1000-25000_xit1.0_O1618-10.0_C1213-inf_N85__Only-CN.spec_0.3-1.0_res1ang'
                   ,skiprows=3)
model_lam = model[:,0]
model_flux = model[:,1]
model_flux = convolve(model_flux,Gaussian1DKernel(1))

plt.plot(model_lam,model_flux)
plt.xlim(4000,9000)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in ['RY Sgr','VZ Sgr','SV Sge']:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])
        plt.plot(wavelength,flux/np.max(flux),label=store_names[i])
#         plt.annotate(store_names[i],xy=(9010),annotation_clip=False,size=15)

plt.plot(model_lam,model_flux,color='tab:red')
plt.yticks([])
plt.xlim(3800,7000)
plt.legend()
plt.show()

In [ ]:
obs_C2_5165_band_area = []

for i in range(0,len(store_names)):
    if store_names[i] in classing_set_names:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])

        flux = flux/np.max(flux)

        if np.min(wavelength) > 4600:
            print(store_names[i], 'not enough spectrum, setting EW to -99')
            eq_width = -99
            index = -99
        else:
            try:
                eq_width, index = spec.calculate_band_eq_width_observations(wavelength,flux,store_names[i],
                                                                       [4900,5175],[4740,4835],[5175,5200])
            except TypeError:
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width = -99
                index = -99


        obs_C2_5165_band_area.append(eq_width)
    else:
        obs_C2_5165_band_area.append(-99)

In [ ]:
obs_C2_4300_band_area = []

for i in range(0,len(store_names)):
    if store_names[i] in classing_set_names:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])
        if store_names[i] == 'B565':
            for ind in range(0,len(flux)):
                if flux[ind] > 1.65e-13:
                    flux[ind] = 1.34981181100152e-13

        flux = flux/np.max(flux)

        if np.min(wavelength) > 4150:
            print(store_names[i], 'not enough spectrum, setting EW to -99')
            eq_width = -99
            index = -99
        else:
            try:
                eq_width, index = spec.calculate_band_eq_width_observations(wavelength,flux,store_names[i],
                                                                       [4255,4400],[4220,4250],[4400,4440])
            except TypeError:
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width = -99
                index = -99


        obs_C2_4300_band_area.append(eq_width)
    else:
        obs_C2_4300_band_area.append(-99)

In [ ]:
for xx in range(0,8):
    print('----------------------------')
    for i in range(0,len(store_names)):
        if store_names[i] in classing_set_names:
            if store_names[i] in final_sets[xx]:
                ratio = np.round(obs_C2_5165_band_area[i]/obs_C2_4300_band_area[i],3)
                print(str(store_names[i])+',',str(ratio)+',',str(obs_C2_5165_band_area[i])+',',str(obs_C2_4300_band_area[i])+',')

In [ ]:
labels = ['HdC0', 'HdC1', 'HdC2', 'HdC3', 'HdC4', 'HdC5', 'HdC6', 'HdC7', 'HdC8', 'HdC9']
rcb_count = [4, 15, 11, 6, 14, 23, 22, 13, 11, 6]
dl_count = [3, 4, 9, 13, 8, 0, 0, 0, 0, 0]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, rcb_count, width, label='RCB')
rects2 = ax.bar(x + width/2, dl_count, width, label='dLHdC',color='tab:red')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('# in class')
# ax.set_title('Scores by group and gender')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# ax.bar_label(rects1, padding=3)
# ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.savefig('class_histogram')
plt.show()

In [ ]:
classified_list = [group0_list,group0half_list,group1_list,group1half_list,group2_list,group2half_list,
                  group3_list,group3half_list,group4_list,group4half_list,group5_list,group5half_list,
                  group6_list,group6half_list,group7_list,group7half_list,group8_list,group9_list,
                  group10_list,group10half_list,group11_list]

classified_list = sum(classified_list,[])

single_list = ['V517 Oph','ASAS-RCB-4','S Aps','J194218.38-203247.5','A770','U Aqr',
              'HD182040','HD175893','RT Nor','HD173409','ASAS-RCB-12','RS Tel','CD-51_12650',
              'UX Ant','FH Sct','RZ Nor','ASAS-RCB-8','UW Cen','V2552 Oph','F75',
              'Y Mus','J172951.80-101715.9']

order_singles = {'V517 Oph':0,'ASAS-RCB-4':1,'S Aps':2,'J194218.38-203247.5':3,'A770':4,'U Aqr':5,
              'HD182040':6,'HD175893':7,'RT Nor':8,'HD173409':9,'ASAS-RCB-12':10,'RS Tel':11,'CD-51_12650':12,
              'UX Ant':13,'FH Sct':14,'RZ Nor':15,'ASAS-RCB-8':16,'UW Cen':17,'V2552 Oph':18,'F75':19,
              'Y Mus':20,'J172951.80-101715.9':21}

In [ ]:
classified_index = dict()

for i in range(0,len(classified_list)):
    if classified_list[i] in group0_list:
        classified_index[classified_list[i]] = 0
    elif classified_list[i] in group0half_list:
        classified_index[classified_list[i]] = 1
    elif classified_list[i] in group1half_list:
        classified_index[classified_list[i]] = 2
    elif classified_list[i] in group1_list:
        classified_index[classified_list[i]] = 3
    elif classified_list[i] in group2_list:
        classified_index[classified_list[i]] = 4
    elif classified_list[i] in group3_list:
        classified_index[classified_list[i]] = 5
    elif classified_list[i] in group5_list:
        classified_index[classified_list[i]] = 6
    elif classified_list[i] in group7_list:
        classified_index[classified_list[i]] = 7
    elif classified_list[i] in group7half_list:
        classified_index[classified_list[i]] = 8
    elif classified_list[i] in group9_list:
        classified_index[classified_list[i]] = 9
    elif classified_list[i] in group10_list:
        classified_index[classified_list[i]] = 10
    elif classified_list[i] in group10half_list:
        classified_index[classified_list[i]] = 11
    elif classified_list[i] in group11_list:
        classified_index[classified_list[i]] = 12
    else:
        classified_index[classified_list[i]] = -5

# print(classified_index)

In [ ]:
new_classified_index = dict()

for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        if store_names[i] in group0_list:
            new_classified_index[store_names[i]] = 0
        elif store_names[i] in group0half_list:
            new_classified_index[store_names[i]] = 1
        elif store_names[i] in group1half_list:
            new_classified_index[store_names[i]] = 2
        elif store_names[i] in group1_list:
            new_classified_index[store_names[i]] = 3
        elif store_names[i] in group2_list:
            new_classified_index[store_names[i]] = 4
        elif store_names[i] in group3_list:
            new_classified_index[store_names[i]] = 5
        elif store_names[i] in group5_list:
            new_classified_index[store_names[i]] = 6
        elif store_names[i] in group7_list:
            new_classified_index[store_names[i]] = 7
        elif store_names[i] in group7half_list:
            new_classified_index[store_names[i]] = 8
        elif store_names[i] in group9_list:
            new_classified_index[store_names[i]] = 9
        elif store_names[i] in group10_list:
            new_classified_index[store_names[i]] = 10
        elif store_names[i] in group10half_list:
            new_classified_index[store_names[i]] = 11
        elif store_names[i] in group11_list:
            new_classified_index[store_names[i]] = 12
        elif store_names[i] in group1lowcn_list:
            new_classified_index[store_names[i]] = 13
        elif store_names[i] in group2half_list:
            new_classified_index[store_names[i]] = 14
        elif store_names[i] in group3half_list:
            new_classified_index[store_names[i]] = 15
        elif store_names[i] in group4_list:
            new_classified_index[store_names[i]] = 16
        elif store_names[i] in group4half_list:
            new_classified_index[store_names[i]] = 17
        elif store_names[i] in group5half_list:
            new_classified_index[store_names[i]] = 18
        elif store_names[i] in group6_list:
            new_classified_index[store_names[i]] = 19
        elif store_names[i] in group6half_list:
            new_classified_index[store_names[i]] = 20
        elif store_names[i] in group8_list:
            new_classified_index[store_names[i]] = 21
    else:
        new_classified_index[store_names[i]] = 22

# print(new_classified_index)

In [ ]:
all_classified_index = dict()

for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        if store_names[i] in group0_list:
            all_classified_index[store_names[i]] = 0
        elif store_names[i] in group0half_list:
            all_classified_index[store_names[i]] = 1
        elif store_names[i] in group1half_list:
            all_classified_index[store_names[i]] = 2
        elif store_names[i] in group1_list:
            all_classified_index[store_names[i]] = 3
        elif store_names[i] in group2_list:
            all_classified_index[store_names[i]] = 4
        elif store_names[i] in group3_list:
            all_classified_index[store_names[i]] = 5
        elif store_names[i] in group5_list:
            all_classified_index[store_names[i]] = 6
        elif store_names[i] in group7_list:
            all_classified_index[store_names[i]] = 7
        elif store_names[i] in group7half_list:
            all_classified_index[store_names[i]] = 8
        elif store_names[i] in group9_list:
            all_classified_index[store_names[i]] = 9
        elif store_names[i] in group10_list:
            all_classified_index[store_names[i]] = 10
        elif store_names[i] in group10half_list:
            all_classified_index[store_names[i]] = 11
        elif store_names[i] in group11_list:
            all_classified_index[store_names[i]] = 12
        else:
            all_classified_index[store_names[i]] = -5
    else:
        all_classified_index[store_names[i]] = 17

# print(all_classified_index)

In [ ]:



n = 15
colors = plt.cm.viridis(np.linspace(0,0.8,n))

plt.figure(figsize=(10,25))
# x = 0
for i in range(0,len(store_names)):
#     if store_names[i] not in ['MACHO 135.27132.51','MACHO 301.45783.9','MACHO 308.38099.66','MACHO 401.48170.2237']:
    if store_names[i] in single_list:
        x=2*order_singles[store_names[i]]
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])
        if store_names[i] == 'B565':
            for xx in range(0,len(flux)):
                if flux[xx] > 1.65e-13:
#                     print('caught',xx)
                    flux[xx] = 1.34981181100152e-13
#                     print(flux[xx])
        greater = np.where(wavelength>7100)
        plt.plot(wavelength,(flux*2/(np.average(flux[greater][:50]))-x),color=colors[int(x/2)])
#         plt.plot(wavelength,(flux*2/np.max(flux))-x,color=colors[int(x/2)])
        if x > -1:
            plt.annotate(store_names[i],xy=(6510,-x+2),annotation_clip=False,size=15)
#         x = x+2

            
# # plt.axvline(4326,color='k')
# # plt.axvline(4290,color='k')
# # plt.axvline(4383,color='k')
# # plt.axvline(4046,color='k')
# # plt.axvline(4030,color='k')
# plt.axvline(4737,color='k')
# plt.axvline(5165,color='k')
# # plt.axvline(4861,color='k')
# # plt.axvline(4990,color='k')

plt.axvline(6191,color='k')
plt.axvline(6122,color='k')
plt.axvline(6059,color='k')
plt.axvline(6175,color='r')
plt.axvline(6013,color='r')
plt.axvline(6347.1,color='b')
plt.axvline(6371.4,color='b')
plt.axvline(6455,color='r')
plt.axvline(6156,color='r')

plt.axvline(7002,color='r')
plt.axvline(7115,color='r')
plt.axvline(7774,color='r')

plt.axvline(6530,color='k')



        
# plt.ylim(-len(store_names)*2,2.5)
plt.ylim(-25,3.5)
plt.yticks([])
plt.xlim(6000,6600)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel('Flux')
plt.show()

In [ ]:
# plt.figure(figsize=(10,72.5))
plt.figure(figsize=(10,100))
x = 0
for i in range(0,len(store_names)):
#     if store_names[i] not in ['MACHO 135.27132.51','MACHO 301.45783.9','MACHO 308.38099.66','MACHO 401.48170.2237']:
    if store_names[i] in classified_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                             store_Av_values[i],store_names[i])
        greater = np.where(wavelength>5300)
        plt.plot(wavelength,(flux*2/np.average(flux[greater][:50]))-x)
#         plt.plot(wavelength,(flux*2/np.max(flux))-x)
        plt.annotate(store_names[i],xy=(5300,-x+2),annotation_clip=False)
        x = x+2

            
# plt.axvline(4326,color='k')
# plt.axvline(4046,color='k')
# plt.axvline(4030,color='k')
plt.axvline(4737,color='k')
plt.axvline(4861,color='k')
plt.axvline(4990,color='k')
        
# plt.ylim(-len(store_names)*2,2.5)
plt.ylim(-len(classified_list)*2+2,3.5)
plt.yticks([])
plt.xlim(4500,5300)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel('Flux')
plt.show()

In [ ]:
all_obs_C2_6122_band_area = []

for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                     store_Av_values[i],store_names[i])

        flux = flux/np.max(flux)

        if np.min(wavelength) > 5000:
            print(name, 'not enough spectrum, setting EW to -99')
            eq_width = -99
            index = -99
        else:
            try:
                eq_width, index = spec.calculate_band_eq_width_observations(wavelength,flux,store_names[i],
                                                                       [6078,6130],[6068,6080],[6127,6134])
            except TypeError:
                print(name, 'not enough spectrum, setting EW to -99')
                eq_width = -99
                index = -99


        all_obs_C2_6122_band_area.append(eq_width)
    else:
        all_obs_C2_6122_band_area.append(-99)

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        plt.scatter(classified_index[store_names[i]],all_obs_C2_6122_band_area[i])

            
        

plt.xlabel('Class index')
plt.ylabel('EW of C2 6122')
# plt.ylim(-1,3)
plt.ylim(bottom=-2)
# plt.ylim(bottom=-0.25)
plt.xlim(-1,13)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        plt.scatter(classified_index[store_names[i]],all_obs_C2_6191_band_area[i]/20,color='tab:blue',alpha=0.75)
        plt.scatter(classified_index[store_names[i]],all_obs_C2_6122_band_area[i]/15,color='tab:red',alpha=0.75)
#         plt.scatter(classified_index[store_names[i]],all_obs_6156_area[i]/2,color='k',alpha=0.75)


            
        

plt.xlabel('Class index')
plt.ylabel('EW of C2 6122 and 6191')
# plt.ylim(-1,3)
plt.ylim(bottom=-0.25)
plt.ylim(top = 1.25)
plt.xlim(-1,13)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        if classified_index[store_names[i]] <=5:
            plt.scatter(0,all_obs_C2_6191_band_area[i],color='tab:blue',alpha=0.75)
        else:
            plt.scatter(all_obs_6175_area[i],all_obs_C2_6191_band_area[i],color='tab:blue',alpha=0.75)

            
        

plt.xlabel('EW 6175')
plt.ylabel('EW of C2 6191')
# plt.ylim(-1,3)
# plt.ylim(bottom=-0.25)
# plt.ylim(top = 1.25)
# plt.xlim(-1,13)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list and classified_index[store_names[i]] == 0 and all_obs_C2_6122_band_area[i]/15 <= 0.8:
        print(store_names[i], all_obs_C2_6122_band_area[i], all_obs_C2_6191_band_area[i])

In [ ]:
all_obs_si_6347_area = []
all_obs_si_6371_area = []

for i in range(0,len(store_names)):
    eq_width_array = [0,0]
    if store_names[i] in classified_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])

        flux = flux/np.max(flux)
    #         flux = convolve(flux,Gaussian1DKernel(1))   

        lines = [6347.1,6371.4]

        if np.min(wavelength) > lines[0]-75:
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99
        else:
            try:
#                 if store_names[i] != 'ASAS-RCB-4':
                eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
                                                                          center_tolerance=2,
                                                                          width_tolerance=9,
                                                                          continuum_region=50,
                                                                          continuum_width=0)
#                 else:
#                     eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
#                                                                                   center_tolerance=2,
#                                                                                   width_tolerance=15,
#                                                                                   continuum_region=20,
#                                                                                   continuum_width=0)
            except (TypeError,ValueError):
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99


    else:
        eq_width_array[0] = -99
        eq_width_array[1] = -99
    
    all_obs_si_6347_area.append(eq_width_array[0])
    all_obs_si_6371_area.append(eq_width_array[1])

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        if classified_index[store_names[i]] < 6:
            plt.scatter(classified_index[store_names[i]],0,color='tab:blue',alpha=0.75)
        else:
            plt.scatter(classified_index[store_names[i]],all_obs_si_6347_area[i],color='tab:blue',alpha=0.75)   
        

plt.xlabel('Class index')
plt.ylabel('EW of Si 6347')
# plt.ylim(-1,3)
# plt.ylim(bottom=-2)
plt.xlim(-1,13)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        if classified_index[store_names[i]] < 6:
            plt.scatter(classified_index[store_names[i]],0,color='tab:blue',alpha=0.75)
        else:
            plt.scatter(classified_index[store_names[i]],all_obs_si_6371_area[i],color='tab:blue',alpha=0.75)
        

plt.xlabel('Class index')
plt.ylabel('EW of Si 6371')
# plt.ylim(-1,3)
# plt.ylim(bottom=-2)
plt.xlim(-1,13)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        if classified_index[store_names[i]] < 6:
            plt.scatter(classified_index[store_names[i]],0,color='tab:blue',alpha=0.75)
        else:
            plt.scatter(classified_index[store_names[i]],all_obs_si_6347_area[i],color='tab:blue',alpha=0.75)
            plt.scatter(classified_index[store_names[i]],all_obs_si_6371_area[i],color='tab:red',alpha=0.75)
        

plt.xlabel('Class index')
plt.ylabel('EW of Si 6371 and 6347')
# plt.ylim(-1,3)
# plt.ylim(bottom=-2)
plt.xlim(-1,13)
plt.show()

In [ ]:
all_obs_7115_area = []

for i in range(0,len(store_names)):
    eq_width_array = [0,0]
    if store_names[i] in classified_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])

        flux = flux/np.max(flux)
    #         flux = convolve(flux,Gaussian1DKernel(1))   

        lines = [7115]

        if np.min(wavelength) > lines[0]-75:
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99
        else:
            try:
#                 if store_names[i] != 'ASAS-RCB-4':
                eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
                                                                          center_tolerance=2,
                                                                          width_tolerance=13,
                                                                          continuum_region=80,
                                                                          continuum_width=0)
#                 else:
#                     eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
#                                                                                   center_tolerance=2,
#                                                                                   width_tolerance=15,
#                                                                                   continuum_region=20,
#                                                                                   continuum_width=0)
            except (TypeError,ValueError):
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99


    else:
        eq_width_array[0] = -99
        eq_width_array[1] = -99
    
    all_obs_7115_area.append(eq_width_array[0])

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        plt.scatter(classified_index[store_names[i]],all_obs_7115_area[i])

            
        

plt.xlabel('Class index')
plt.ylabel('EW of 7115')
# plt.ylim(-1,5)
plt.xlim(-1,13)
plt.show()

In [ ]:
all_obs_7774_area = []

for i in range(0,len(store_names)):
    eq_width_array = [0,0]
    if store_names[i] in classified_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])

        flux = flux/np.max(flux)
    #         flux = convolve(flux,Gaussian1DKernel(1))   

        lines = [7774]

        if np.min(wavelength) > lines[0]-75:
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99
        else:
            try:
#                 if store_names[i] != 'ASAS-RCB-4':
                eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
                                                                          center_tolerance=2,
                                                                          width_tolerance=8,
                                                                          continuum_region=75,
                                                                          continuum_width=0)
#                 else:
#                     eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
#                                                                                   center_tolerance=2,
#                                                                                   width_tolerance=15,
#                                                                                   continuum_region=20,
#                                                                                   continuum_width=0)
            except (TypeError,ValueError):
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99


    else:
        eq_width_array[0] = -99
        eq_width_array[1] = -99
    
    all_obs_7774_area.append(eq_width_array[0])

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        plt.scatter(classified_index[store_names[i]],all_obs_7774_area[i])
        
        

plt.xlabel('Class index')
plt.ylabel('EW of 7774')
# plt.ylim(-1,5)
plt.xlim(-1,13)
plt.show()

In [ ]:
all_obs_fe_4326_area = []
all_obs_cr_4290_area = []

for i in range(0,len(store_names)):
    eq_width_array = [0,0]
    if store_names[i] in classified_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])

        flux = flux/np.max(flux)
    #         flux = convolve(flux,Gaussian1DKernel(1))   

        lines = [4290,4326]

        if np.min(wavelength) > lines[0]-75:
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99
        else:
            try:
#                 if store_names[i] != 'ASAS-RCB-4':
                eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
                                                                          center_tolerance=2,
                                                                          width_tolerance=9,
                                                                          continuum_region=50,
                                                                          continuum_width=0)
#                 else:
#                     eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
#                                                                                   center_tolerance=2,
#                                                                                   width_tolerance=15,
#                                                                                   continuum_region=20,
#                                                                                   continuum_width=0)
            except (TypeError,ValueError):
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99


    else:
        eq_width_array[0] = -99
        eq_width_array[1] = -99
    
    all_obs_fe_4326_area.append(eq_width_array[1])
    all_obs_cr_4290_area.append(eq_width_array[0])

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        plt.scatter(classified_index[store_names[i]],all_obs_fe_4326_area[i])
        
        

plt.xlabel('Class index')
plt.ylabel('EW of Fe 4326')
# plt.ylim(-1,5)
plt.ylim(bottom=-0.5)
plt.ylim(top=5)
plt.xlim(-1,13)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in classified_list:
        plt.scatter(classified_index[store_names[i]],all_obs_cr_4290_area[i])
        

plt.xlabel('Class index')
plt.ylabel('EW of Cr 4290')
# plt.ylim(-1,7)
plt.ylim(bottom=-0.5)
plt.xlim(-1,13)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if obs_cr_4290_area[i] != -99 and obs_fe_4326_area[i] != -99 and store_names[i] in single_list:
        plt.scatter(order_singles[store_names[i]],obs_cr_4290_area[i]/obs_fe_4326_area[i])

        

plt.xlabel('Class index')
plt.ylabel('EW of Cr 4290/Fe 4326')
plt.ylim(-1,5)
plt.xlim(-1,13)
plt.show()

In [ ]:
obs_fe_4046_area = []
obs_mn_4030_area = []

for i in range(0,len(store_names)):
    eq_width_array = [0,0]
    if store_names[i] in single_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])

        flux = flux/np.max(flux)
    #         flux = convolve(flux,Gaussian1DKernel(1))   

        lines = [4030,4046]

        if np.min(wavelength) > lines[0]-75:
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99
        else:
            try:
                eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
                                                                          center_tolerance=2,
                                                                          width_tolerance=15,
                                                                          continuum_region=50,
                                                                          continuum_width=0)
            except (TypeError,ValueError,IndexError):
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99


    else:
        eq_width_array[0] = -99
        eq_width_array[1] = -99
    
    obs_fe_4046_area.append(eq_width_array[1])
    obs_mn_4030_area.append(eq_width_array[0])

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in single_list:
        plt.scatter(order_singles[store_names[i]],obs_fe_4046_area[i])
        

plt.xlabel('Class index')
plt.ylabel('EW of Fe 4046')
plt.ylim(-1,5)
plt.xlim(left=-1)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in single_list:
        plt.scatter(order_singles[store_names[i]],obs_mn_4030_area[i])
        

plt.xlabel('Class index')
plt.ylabel('EW of Mn 4030')
plt.ylim(-1,5)
plt.xlim(left=-1)
plt.show()

In [ ]:
for i in range(0,len(store_names)):
    if obs_fe_4046_area[i] != -99 and obs_mn_4030_area[i] != -99 and store_names[i] in single_list:
        plt.scatter(order_singles[store_names[i]],obs_fe_4046_area[i]/obs_mn_4030_area[i])
        

plt.xlabel('Class index')
plt.ylabel('EW of Fe 4046/Mn 4030')
plt.ylim(-1,5)
plt.xlim(left=-1)
plt.show()

In [ ]:
obs_fe_4383_area = []

for i in range(0,len(store_names)):
    eq_width_array = [0,0]
    if store_names[i] in single_list:
        wavelength, flux = spec.apply_extinction(store_rest_wavelength[i],store_rest_flux[i],
                                                         store_Av_values[i],store_names[i])

        flux = flux/np.max(flux)
    #         flux = convolve(flux,Gaussian1DKernel(1))   

        lines = [4383]

        if np.min(wavelength) > lines[0]-75:
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99
        else:
            try:
#                 if store_names[i] != 'ASAS-RCB-4':
                eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
                                                                          center_tolerance=2,
                                                                          width_tolerance=15,
                                                                          continuum_region=50,
                                                                          continuum_width=0)
#                 else:
#                     eq_width_array, voigt_param = spec.calculate_eq_width_observations(wavelength,flux,store_names[i],lines,
#                                                                                   center_tolerance=2,
#                                                                                   width_tolerance=15,
#                                                                                   continuum_region=20,
#                                                                                   continuum_width=0)
            except (TypeError,ValueError):
                print(store_names[i], 'not enough spectrum, setting EW to -99')
                eq_width_array[0] = -99
                eq_width_array[1] = -99


    else:
        eq_width_array[0] = -99
        eq_width_array[1] = -99
    
    obs_fe_4383_area.append(eq_width_array[0])
#     obs_cr_4290_area.append(eq_width_array[0])

In [ ]:
for i in range(0,len(store_names)):
    if store_names[i] in single_list:
        plt.scatter(order_singles[store_names[i]],obs_fe_4383_area[i])
        

plt.xlabel('Class index')
plt.ylabel('EW of Fe 4383')
plt.ylim(-1,5)
plt.xlim(left=-1)
plt.show()